<a href="https://colab.research.google.com/github/autilo/SparkAR-FaceTrackingRecording/blob/master/Embedingi_hybrydowy_skrypty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprawdzanie dostępów do **haseł**

In [8]:
#!/usr/bin/env python3
"""
🔑 NAPRAWA SEKRETÓW COLAB
Test i naprawa pobierania zmiennych z Google Colab userdata
"""

def test_colab_secrets():
    """Testuj różne sposoby pobierania sekretów z Colab"""

    print("🔑 TESTOWANIE POBIERANIA SEKRETÓW Z COLAB")
    print("=" * 50)

    try:
        from google.colab import userdata
        print("✅ Moduł google.colab.userdata zaimportowany")

        # Lista wymaganych sekretów
        required_secrets = [
            'POSTGRESQL_HOST',
            'POSTGRESQL_PORT',
            'POSTGRESQL_DB',
            'POSTGRESQL_USER',
            'POSTGRESQL_PASSWORD',
            'QDRANT_URL',
            'OPENAI_API_KEY'
        ]

        print(f"\n🔍 Sprawdzanie {len(required_secrets)} sekretów...")

        secrets = {}
        missing = []

        for secret_name in required_secrets:
            try:
                # Próbuj pobrać sekret
                value = userdata.get(secret_name)

                if value:
                    # Maskuj wartość dla bezpieczeństwa
                    if len(value) > 8:
                        masked = f"{value[:4]}...{value[-4:]}"
                    else:
                        masked = "***"

                    secrets[secret_name] = value
                    print(f"   ✅ {secret_name}: {masked}")
                else:
                    missing.append(secret_name)
                    print(f"   ❌ {secret_name}: PUSTY")

            except Exception as e:
                missing.append(secret_name)
                print(f"   ❌ {secret_name}: BŁĄD - {e}")

        # Podsumowanie
        print(f"\n📊 WYNIKI:")
        print(f"   ✅ Znalezione sekrety: {len(secrets)}")
        print(f"   ❌ Brakujące sekrety: {len(missing)}")

        if missing:
            print(f"\n❌ BRAKUJĄCE SEKRETY:")
            for secret in missing:
                print(f"   • {secret}")
            print(f"\n💡 INSTRUKCJE:")
            print(f"   1. Kliknij ikonę klucza 🔑 w lewym panelu Colab")
            print(f"   2. Dodaj brakujące sekrety jeden po drugim")
            print(f"   3. Upewnij się że nazwy są DOKŁADNIE takie jak powyżej")

        if len(secrets) == len(required_secrets):
            print(f"\n🎉 WSZYSTKIE SEKRETY DOSTĘPNE!")
            return secrets
        else:
            return None

    except ImportError:
        print("❌ Nie można zaimportować google.colab.userdata")
        print("💡 Upewnij się że uruchamiasz w Google Colab")
        return None
    except Exception as e:
        print(f"❌ Nieoczekiwany błąd: {e}")
        return None

def get_secrets_fixed():
    """Naprawiona funkcja pobierania sekretów"""
    try:
        from google.colab import userdata

        secrets = {}

        # Lista wymaganych sekretów
        secret_names = [
            'POSTGRESQL_HOST',
            'POSTGRESQL_DB',
            'POSTGRESQL_USER',
            'POSTGRESQL_PASSWORD',
            'QDRANT_URL',
            'OPENAI_API_KEY'
        ]

        missing = []

        for name in secret_names:
            try:
                value = userdata.get(name)
                if value and value.strip():
                    secrets[name] = value
                else:
                    missing.append(name)
            except Exception as e:
                print(f"⚠️ Błąd pobierania {name}: {e}")
                missing.append(name)

        # Dodaj domyślny port jeśli nie ma
        if 'POSTGRESQL_PORT' not in secrets:
            secrets['POSTGRESQL_PORT'] = '5432'

        if missing:
            print(f"❌ Brakujące sekrety: {missing}")
            return None

        print("✅ Wszystkie sekrety załadowane pomyślnie")
        return secrets

    except Exception as e:
        print(f"❌ Błąd ładowania sekretów: {e}")
        return None

# ============================================================================
# 🧪 TEST POŁĄCZEŃ
# ============================================================================

def test_connections_with_secrets(secrets):
    """Testuj połączenia z pobranymi sekretami"""

    if not secrets:
        print("❌ Brak sekretów do testowania")
        return

    print(f"\n🔗 TESTOWANIE POŁĄCZEŃ")
    print("=" * 30)

    # Test PostgreSQL
    print("🐘 Test PostgreSQL...")
    try:
        import psycopg2

        conn = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )

        cursor = conn.cursor()
        cursor.execute("SELECT version();")
        version = cursor.fetchone()[0]
        print(f"   ✅ PostgreSQL OK: {version[:50]}...")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"   ❌ PostgreSQL błąd: {e}")

    # Test Qdrant
    print("\n🎯 Test Qdrant...")
    try:
        from qdrant_client import QdrantClient

        client = QdrantClient(url=secrets['QDRANT_URL'])
        collections = client.get_collections()
        print(f"   ✅ Qdrant OK: {len(collections.collections)} kolekcji")

    except Exception as e:
        print(f"   ❌ Qdrant błąd: {e}")

    # Test OpenAI
    print("\n🤖 Test OpenAI...")
    try:
        import openai

        client = openai.OpenAI(api_key=secrets['OPENAI_API_KEY'])

        # Test prostego embeddingu
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input="test"
        )

        embedding = response.data[0].embedding
        print(f"   ✅ OpenAI OK: embedding {len(embedding)} wymiarów")

    except Exception as e:
        print(f"   ❌ OpenAI błąd: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA TESTU
# ============================================================================

def main():
    """Główna funkcja testowa"""

    print("🔧 NAPRAWA I TEST SEKRETÓW COLAB")
    print(f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Test pobierania sekretów
    secrets = test_colab_secrets()

    if secrets:
        # Test połączeń
        test_connections_with_secrets(secrets)

        print(f"\n🎉 WSZYSTKO GOTOWE!")
        print(f"💡 Możesz teraz uruchomić główny skrypt generatora embeddingów")

    else:
        print(f"\n⚠️ DODAJ BRAKUJĄCE SEKRETY I URUCHOM PONOWNIE")

if __name__ == "__main__":
    from datetime import datetime
    main()

🔧 NAPRAWA I TEST SEKRETÓW COLAB
⏰ 2025-06-02 14:28:12
🔑 TESTOWANIE POBIERANIA SEKRETÓW Z COLAB
✅ Moduł google.colab.userdata zaimportowany

🔍 Sprawdzanie 7 sekretów...
   ✅ POSTGRESQL_HOST: 95.2....157
   ✅ POSTGRESQL_PORT: ***
   ✅ POSTGRESQL_DB: ***
   ✅ POSTGRESQL_USER: ***
   ✅ POSTGRESQL_PASSWORD: dify...3456
   ✅ QDRANT_URL: http...6333
   ✅ OPENAI_API_KEY: sk-p...E30A

📊 WYNIKI:
   ✅ Znalezione sekrety: 7
   ❌ Brakujące sekrety: 0

🎉 WSZYSTKIE SEKRETY DOSTĘPNE!

🔗 TESTOWANIE POŁĄCZEŃ
🐘 Test PostgreSQL...
   ✅ PostgreSQL OK: PostgreSQL 15.13 on x86_64-pc-linux-musl, compiled...

🎯 Test Qdrant...
   ✅ Qdrant OK: 0 kolekcji

🤖 Test OpenAI...
   ✅ OpenAI OK: embedding 1536 wymiarów

🎉 WSZYSTKO GOTOWE!
💡 Możesz teraz uruchomić główny skrypt generatora embeddingów


## Naprawa bazy danych

In [23]:
def insert_urls_safe(connection, urls_data):
    """Bezpieczne wstawianie URL-i z obsługą błędów"""

    print(f"💾 Bezpieczne wstawianie {len(urls_data)} URL-i...")

    # Sprawdź istniejące
    cursor = connection.cursor()
    cursor.execute("SELECT link FROM articles")
    existing = {row[0] for row in cursor.fetchall()}
    cursor.close()

    # Filtruj i normalizuj
    safe_urls = []
    for url_data in urls_data:
        url = url_data['url']

        # Skip jeśli już istnieje
        if url in existing:
            continue

        # Normalizuj URL (usuń polskie znaki jeśli są)
        try:
            safe_url = url.encode('ascii', errors='ignore').decode('ascii')
            if len(safe_url) > 450:  # Zostaw margines
                safe_url = safe_url[:450]

            safe_urls.append({
                'link': safe_url,
                'source': url_data['source'],
                'original_url': url
            })
        except:
            print(f"⚠️ Pomijam problematyczny URL: {url[:50]}...")
            continue

    if not safe_urls:
        print("ℹ️ Brak nowych URL-i do wstawienia")
        return 0

    # Wstaw po jednym (autocommit)
    connection.autocommit = True
    cursor = connection.cursor()

    success_count = 0
    error_count = 0

    for url_data in safe_urls:
        try:
            # JSON placeholder
            placeholder = json.dumps({
                "data": {
                    "title": "Brak tytułu",
                    "content": "",
                    "url": url_data['original_url']
                }
            })

            cursor.execute("""
                INSERT INTO articles (link, text, text_status, source, candidates)
                VALUES (%s, %s, 'pending', %s, '[]')
            """, (url_data['link'], placeholder, url_data['source']))

            success_count += 1

        except Exception as e:
            error_count += 1
            if error_count <= 5:  # Pokaż tylko pierwsze 5 błędów
                print(f"⚠️ Błąd: {str(e)[:50]}...")

    cursor.close()
    connection.autocommit = False  # Przywróć normalne transakcje

    print(f"✅ Pomyślnie: {success_count}")
    print(f"❌ Błędy: {error_count}")

    return success_count

# ZASTĄP w głównym kodzie:
# Znajdź linię: insert_urls_to_db(connection, all_urls)
# Zamień na: insert_urls_safe(connection, all_urls)

In [18]:
# FINALNA NAPRAWA BAZY
import psycopg2
from google.colab import userdata

def fix_database_final():
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER', 'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'

    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        connection.autocommit = True  # Automatyczny commit

        cursor = connection.cursor()

        # 1. Usuń tabelę jeśli istnieje (fresh start)
        print("🗑️ Usuwam starą tabelę...")
        cursor.execute("DROP TABLE IF EXISTS articles CASCADE;")

        # 2. Stwórz nową tabelę
        print("🏗️ Tworzę nową tabelę...")
        create_table_query = """
        CREATE TABLE articles (
            id SERIAL PRIMARY KEY,
            link VARCHAR(500) NOT NULL UNIQUE,
            crawled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            text TEXT,
            text_status VARCHAR(50) DEFAULT 'pending',
            embedding TEXT,
            title_embedding TEXT,
            embedding_status VARCHAR(50),
            candidates TEXT,
            candidates_title TEXT,
            re_rank_content TEXT,
            external_links TEXT,
            source VARCHAR(50) DEFAULT 'unknown',
            monthly_views INTEGER DEFAULT 0,
            category VARCHAR(100),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_table_query)

        # 3. Dodaj indexy
        print("📇 Tworzę indexy...")
        cursor.execute("CREATE INDEX idx_articles_link ON articles(link);")
        cursor.execute("CREATE INDEX idx_articles_text_status ON articles(text_status);")
        cursor.execute("CREATE INDEX idx_articles_embedding_status ON articles(embedding_status);")

        cursor.close()
        connection.close()

        print("✅ BAZA CAŁKOWICIE NAPRAWIONA!")
        print("🚀 Uruchom crawler teraz - powinien działać!")

    except Exception as e:
        print(f"❌ Błąd: {e}")

# Uruchom finalną naprawę
fix_database_final()

🗑️ Usuwam starą tabelę...
🏗️ Tworzę nową tabelę...
📇 Tworzę indexy...
✅ BAZA CAŁKOWICIE NAPRAWIONA!
🚀 Uruchom crawler teraz - powinien działać!


# 1 CRAWLER: Sitemap + url_blog.txt → PostgreSQL:**pogrubiony tekst**

In [26]:
#!/usr/bin/env python3
"""
🕷️ NAPRAWIONY CRAWLER - Bez błędów transakcji i kodowania
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import json
import time
from datetime import datetime
from google.colab import userdata
import pandas as pd
from tqdm import tqdm
import urllib.parse

# Instalacja bibliotek
try:
    import advertools as adv
    print("✅ advertools już zainstalowany")
except ImportError:
    print("📦 Instaluję advertools...")
    os.system('pip install advertools')
    import advertools as adv

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD', 'JINA_API_KEY']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🗄️ SETUP BAZY DANYCH
# ============================================================================

def create_articles_table(connection):
    """Stwórz tabelę articles"""

    cursor = connection.cursor()

    try:
        # Usuń starą tabelę
        cursor.execute("DROP TABLE IF EXISTS articles CASCADE;")
        connection.commit()
        print("🗑️ Usunięto starą tabelę")

        # Stwórz nową
        create_query = """
        CREATE TABLE articles (
            id SERIAL PRIMARY KEY,
            link TEXT NOT NULL UNIQUE,
            crawled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            text TEXT,
            text_status VARCHAR(50) DEFAULT 'pending',
            embedding TEXT,
            title_embedding TEXT,
            embedding_status VARCHAR(50),
            candidates TEXT DEFAULT '[]',
            source VARCHAR(50) DEFAULT 'unknown',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_query)
        connection.commit()

        # Dodaj indexy
        cursor.execute("CREATE INDEX idx_articles_text_status ON articles(text_status);")
        cursor.execute("CREATE INDEX idx_articles_source ON articles(source);")
        connection.commit()

        cursor.close()

        print("✅ Tabela 'articles' utworzona")
        return True

    except Exception as e:
        print(f"❌ Błąd tworzenia tabeli: {e}")
        connection.rollback()
        cursor.close()
        return False

def safe_insert_urls(connection, urls_data):
    """Bezpieczne wstawianie URL-i bez błędów transakcji"""

    if not urls_data:
        print("⚠️ Brak URL-i do wstawienia")
        return 0

    print(f"💾 Wstawianie {len(urls_data)} URL-i...")

    # Sprawdź istniejące URL-e
    cursor = connection.cursor()
    cursor.execute("SELECT link FROM articles")
    existing_links = {row[0] for row in cursor.fetchall()}
    cursor.close()

    print(f"🔍 Znaleziono {len(existing_links)} istniejących URL-i")

    # Przygotuj nowe URL-e
    new_urls = []
    duplicate_count = 0
    error_count = 0

    for url_data in urls_data:
        try:
            original_url = url_data['url']

            # Sprawdź czy już istnieje
            if original_url in existing_links:
                duplicate_count += 1
                continue

            # JSON placeholder
            placeholder_text = json.dumps({
                "data": {
                    "title": "Brak tytułu",
                    "content": "",
                    "url": original_url
                }
            }, ensure_ascii=False)

            new_urls.append({
                'link': original_url,
                'text': placeholder_text,
                'source': url_data['source']
            })

        except Exception as e:
            error_count += 1
            if error_count <= 3:
                print(f"⚠️ Błąd przygotowania URL: {str(e)[:50]}...")

    if not new_urls:
        print(f"ℹ️ Brak nowych URL-i (duplikaty: {duplicate_count})")
        return 0

    # Wstaw URL-e jeden po drugim z commit po każdym
    success_count = 0
    insert_errors = 0

    insert_query = """
    INSERT INTO articles (link, text, text_status, source)
    VALUES (%s, %s, 'pending', %s)
    """

    for i, url_data in enumerate(tqdm(new_urls, desc="Bezpieczne wstawianie")):
        try:
            cursor = connection.cursor()
            cursor.execute(insert_query, (
                url_data['link'],
                url_data['text'],
                url_data['source']
            ))
            connection.commit()  # Commit po każdym insercie
            cursor.close()
            success_count += 1

        except Exception as e:
            connection.rollback()  # Rollback w przypadku błędu
            if cursor:
                cursor.close()
            insert_errors += 1
            # Kontynuuj bez wyświetlania błędów

    print(f"📊 WYNIKI WSTAWIANIA:")
    print(f"   ✅ Pomyślnie wstawiono: {success_count}")
    print(f"   ⚠️ Duplikaty pominięte: {duplicate_count}")
    print(f"   ❌ Błędy wstawiania: {insert_errors}")
    print(f"   📊 Łącznie przetworzono: {len(urls_data)}")

    return success_count

# ============================================================================
# 🗺️ POBIERANIE URL-I
# ============================================================================

def get_sitemap_urls(sitemap_url):
    """Pobierz URL-e z sitemap"""
    print(f"🗺️ Pobieranie sitemap: {sitemap_url}")

    try:
        sitemap_df = adv.sitemap_to_df(sitemap_url)

        if not sitemap_df.empty and 'loc' in sitemap_df.columns:
            urls = sitemap_df['loc'].tolist()
            print(f"✅ Znaleziono {len(urls)} URL-i w sitemap")

            urls_data = []
            for url in urls:
                urls_data.append({
                    'url': url,
                    'source': 'sitemap'
                })

            return urls_data
        else:
            print("⚠️ Pusty sitemap")
            return []

    except Exception as e:
        print(f"❌ Błąd sitemap: {e}")
        return []

def get_blog_urls_from_file():
    """Pobierz URL-e z pliku url_blog.xlsx"""
    print("📄 Szukanie pliku url_blog.xlsx...")

    urls_data = []

    try:
        if os.path.exists('url_blog.xlsx'):
            df = pd.read_excel('url_blog.xlsx')
            print(f"✅ Wczytano plik Excel ({len(df)} wierszy)")

            # Znajdź kolumnę z URL-ami
            url_column = None
            for col in df.columns:
                if 'url' in col.lower() or 'link' in col.lower():
                    url_column = col
                    break

            if url_column:
                print(f"🎯 Używam kolumny: {url_column}")

                for _, row in df.iterrows():
                    url = str(row[url_column]).strip()
                    if url and url.startswith('http') and 'nan' not in url.lower():
                        urls_data.append({
                            'url': url,
                            'source': 'blog_file'
                        })

                print(f"✅ Wczytano {len(urls_data)} URL-i z pliku")
            else:
                print("❌ Nie znaleziono kolumny z URL-ami")
                return []
        else:
            print("⚠️ Plik url_blog.xlsx nie znaleziony")
            choice = input("❓ Kontynuować bez pliku? (t/n): ").strip().lower()
            if choice not in ['t', 'tak', 'y', 'yes', '']:
                return None
            return []

    except Exception as e:
        print(f"❌ Błąd pliku Excel: {e}")
        return []

    return urls_data

# ============================================================================
# 🕷️ CRAWLING
# ============================================================================

def crawl_articles(connection, jina_api_key, limit=10):
    """Crawl artykułów z obsługą błędów"""

    # Pobierz pending URL-e
    cursor = connection.cursor(cursor_factory=RealDictCursor)
    cursor.execute("""
        SELECT id, link
        FROM articles
        WHERE text_status = 'pending'
        ORDER BY created_at ASC
        LIMIT %s
    """, (limit,))

    pending_urls = cursor.fetchall()
    cursor.close()

    if not pending_urls:
        print("🤷 Brak URL-i do crawlingu")
        return 0, 0

    print(f"🕷️ Crawling {len(pending_urls)} URL-i...")

    success_count = 0
    error_count = 0

    # Crawl po kolei
    for i, url_data in enumerate(pending_urls):
        print(f"📄 [{i+1}/{len(pending_urls)}] {url_data['link'][:60]}...")

        try:
            # Wywołaj Jina AI
            api_url = f"https://r.jina.ai/{url_data['link']}"
            headers = {
                "Accept": "application/json",
                "Authorization": f"Bearer {jina_api_key}"
            }

            response = requests.get(api_url, headers=headers, timeout=30)

            if response.status_code == 200:
                content = response.text

                # Sprawdź rate limit
                if 'Whoa there, turbo!' in content:
                    print(f"   ⏳ Rate limit - czekam 30s")
                    time.sleep(30)
                    continue

                # Parse treści
                try:
                    json_data = json.loads(content)
                    title = json_data.get('data', {}).get('title', 'Brak tytułu')
                    article_content = json_data.get('data', {}).get('content', content)
                except:
                    title = 'Artykuł'
                    article_content = content

                # Utwórz JSON
                structured_text = json.dumps({
                    "data": {
                        "title": title,
                        "content": article_content,
                        "url": url_data['link']
                    }
                }, ensure_ascii=False)

                # Zapisz do bazy
                cursor = connection.cursor()
                cursor.execute("""
                    UPDATE articles
                    SET text = %s, text_status = 'completed', crawled_at = %s
                    WHERE id = %s
                """, (structured_text, datetime.now(), url_data['id']))
                connection.commit()
                cursor.close()

                print(f"   ✅ Pobrano: {title[:50]}...")
                success_count += 1

            else:
                # Błąd HTTP
                cursor = connection.cursor()
                cursor.execute("""
                    UPDATE articles
                    SET text_status = %s, crawled_at = %s
                    WHERE id = %s
                """, (f'error_http_{response.status_code}', datetime.now(), url_data['id']))
                connection.commit()
                cursor.close()

                print(f"   ❌ HTTP error: {response.status_code}")
                error_count += 1

        except Exception as e:
            # Błąd ogólny
            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = 'error_general', crawled_at = %s
                WHERE id = %s
            """, (datetime.now(), url_data['id']))
            connection.commit()
            cursor.close()

            print(f"   ❌ Error: {str(e)[:50]}...")
            error_count += 1

        # Pauza między requestami
        time.sleep(2)

    return success_count, error_count

# ============================================================================
# 📊 STATYSTYKI
# ============================================================================

def show_stats(connection):
    """Pokaż statystyki bazy"""

    try:
        cursor = connection.cursor()

        # Status breakdown
        cursor.execute("""
            SELECT text_status, COUNT(*) as count
            FROM articles
            GROUP BY text_status
            ORDER BY count DESC
        """)

        print(f"\n📊 STATYSTYKI BAZY DANYCH:")
        print(f"{'Status':<20} {'Liczba':<10}")
        print("-" * 32)

        total = 0
        for row in cursor.fetchall():
            print(f"{row[0]:<20} {row[1]:<10}")
            total += row[1]

        print(f"{'ŁĄCZNIE':<20} {total:<10}")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd statystyk: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja"""

    print("🕷️ NAPRAWIONY CRAWLER")
    print("=" * 40)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # Menu
        print(f"\n🎯 WYBIERZ OPCJĘ:")
        print("1. Setup bazy + dodaj URL-e")
        print("2. Tylko crawling")
        print("3. Tylko statystyki")

        choice = input("\nWybór (1-3): ").strip()

        if choice == '1':
            # Setup + URL-e
            print(f"\n🏗️ SETUP BAZY")
            if not create_articles_table(connection):
                return

            # Pobierz URL-e
            all_urls = []

            sitemap = input("🗺️ URL sitemap (Enter=pomiń): ").strip()
            if sitemap:
                sitemap_urls = get_sitemap_urls(sitemap)
                all_urls.extend(sitemap_urls)

            blog_urls = get_blog_urls_from_file()
            if blog_urls is None:
                return
            all_urls.extend(blog_urls)

            if all_urls:
                safe_insert_urls(connection, all_urls)
            else:
                print("⚠️ Brak URL-i")

        elif choice == '2':
            # Crawling
            if not secrets.get('JINA_API_KEY'):
                print("❌ Brak JINA_API_KEY")
                return

            limit = input("Ile crawlować? (Enter=10): ").strip()
            limit = int(limit) if limit.isdigit() else 10

            success, errors = crawl_articles(connection, secrets['JINA_API_KEY'], limit)

            print(f"\n🎯 WYNIKI CRAWLINGU:")
            print(f"   ✅ Sukces: {success}")
            print(f"   ❌ Błędy: {errors}")

        # Zawsze pokaż statystyki
        show_stats(connection)

        print(f"\n🎉 GOTOWE!")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

✅ advertools już zainstalowany
🕷️ NAPRAWIONY CRAWLER
✅ PostgreSQL połączony

🎯 WYBIERZ OPCJĘ:
1. Setup bazy + dodaj URL-e
2. Tylko crawling
3. Tylko statystyki

Wybór (1-3): 1

🏗️ SETUP BAZY
🗑️ Usunięto starą tabelę
✅ Tabela 'articles' utworzona
🗺️ URL sitemap (Enter=pomiń): https://cateringfoodharmony.pl/sitemap.xml
🗺️ Pobieranie sitemap: https://cateringfoodharmony.pl/sitemap.xml


INFO:root:Getting https://cateringfoodharmony.pl/sitemap.xml


✅ Znaleziono 99 URL-i w sitemap
📄 Szukanie pliku url_blog.xlsx...
✅ Wczytano plik Excel (124 wierszy)
🎯 Używam kolumny: url
✅ Wczytano 124 URL-i z pliku
💾 Wstawianie 223 URL-i...
🔍 Znaleziono 0 istniejących URL-i


Bezpieczne wstawianie: 100%|██████████| 223/223 [01:16<00:00,  2.90it/s]


📊 WYNIKI WSTAWIANIA:
   ✅ Pomyślnie wstawiono: 174
   ⚠️ Duplikaty pominięte: 0
   ❌ Błędy wstawiania: 49
   📊 Łącznie przetworzono: 223

📊 STATYSTYKI BAZY DANYCH:
Status               Liczba    
--------------------------------
pending              174       
ŁĄCZNIE              174       

🎉 GOTOWE!
🔒 Połączenie zamknięte


## 1.1 Sprawdza błędy wstawienia adresów *url*

In [27]:
#!/usr/bin/env python3
"""
🔍 MINI-SKRYPT: DEBUGOWANIE BŁĘDÓW WSTAWIANIA URL-I
Sprawdza dokładnie dlaczego 49 URL-i nie zostało wstawionych
"""

import os
import psycopg2
import json
import pandas as pd
from google.colab import userdata

# Instalacja advertools jeśli potrzeba
try:
    import advertools as adv
except ImportError:
    os.system('pip install advertools')
    import advertools as adv

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER', 'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 📋 POBIERZ WSZYSTKIE URL-E KTÓRE MIAŁY BYĆ WSTAWIONE
# ============================================================================

def get_all_source_urls():
    """Pobierz wszystkie URL-e ze źródeł (sitemap + Excel)"""

    all_urls = []

    # 1. URL-e z sitemap
    print("🗺️ Pobieranie URL-i z sitemap...")
    sitemap_url = "https://cateringfoodharmony.pl/sitemap.xml"  # Z poprzedniego testu

    try:
        sitemap_df = adv.sitemap_to_df(sitemap_url)
        if not sitemap_df.empty and 'loc' in sitemap_df.columns:
            sitemap_urls = sitemap_df['loc'].tolist()
            print(f"✅ Sitemap: {len(sitemap_urls)} URL-i")

            for url in sitemap_urls:
                all_urls.append({
                    'url': url,
                    'source': 'sitemap'
                })
    except Exception as e:
        print(f"❌ Błąd sitemap: {e}")

    # 2. URL-e z Excel
    print("📄 Pobieranie URL-i z Excel...")

    try:
        if os.path.exists('url_blog.xlsx'):
            df = pd.read_excel('url_blog.xlsx')

            # Znajdź kolumnę z URL-ami
            url_column = None
            for col in df.columns:
                if 'url' in col.lower():
                    url_column = col
                    break

            if url_column:
                excel_urls = []
                for _, row in df.iterrows():
                    url = str(row[url_column]).strip()
                    if url and url.startswith('http') and 'nan' not in url.lower():
                        excel_urls.append(url)

                print(f"✅ Excel: {len(excel_urls)} URL-i")

                for url in excel_urls:
                    all_urls.append({
                        'url': url,
                        'source': 'blog_file'
                    })
        else:
            print("⚠️ Brak pliku Excel")

    except Exception as e:
        print(f"❌ Błąd Excel: {e}")

    print(f"📊 Łącznie ze źródeł: {len(all_urls)} URL-i")
    return all_urls

# ============================================================================
# 🗄️ SPRAWDŹ CO JEST W BAZIE
# ============================================================================

def get_database_urls(connection):
    """Pobierz URL-e które są już w bazie"""

    try:
        cursor = connection.cursor()
        cursor.execute("SELECT link, source FROM articles")
        db_urls = cursor.fetchall()
        cursor.close()

        print(f"🗄️ W bazie: {len(db_urls)} URL-i")

        return {row[0]: row[1] for row in db_urls}

    except Exception as e:
        print(f"❌ Błąd pobierania z bazy: {e}")
        return {}

# ============================================================================
# 🔍 ANALIZA BŁĘDÓW
# ============================================================================

def analyze_failed_urls(source_urls, db_urls):
    """Analizuj które URL-e się nie udało wstawić i dlaczego"""

    print(f"\n🔍 ANALIZA BŁĘDÓW WSTAWIANIA")
    print("=" * 40)

    # Znajdź URL-e które nie zostały wstawione
    source_set = {item['url'] for item in source_urls}
    db_set = set(db_urls.keys())

    missing_urls = source_set - db_set
    duplicates_in_source = len(source_urls) - len(source_set)

    print(f"📊 STATYSTYKI:")
    print(f"   📥 URL-e ze źródeł: {len(source_urls)}")
    print(f"   🔄 Duplikaty w źródłach: {duplicates_in_source}")
    print(f"   📥 Unikalne ze źródeł: {len(source_set)}")
    print(f"   🗄️ Wstawione do bazy: {len(db_set)}")
    print(f"   ❌ Nie udało się wstawić: {len(missing_urls)}")

    if not missing_urls:
        print("✅ Wszystkie URL-e zostały wstawione!")
        return

    print(f"\n🔍 ANALIZA {len(missing_urls)} PROBLEMATYCZNYCH URL-I:")

    # Kategoryzuj błędy
    error_categories = {
        'too_long': [],
        'special_chars': [],
        'encoding_issues': [],
        'json_issues': [],
        'other': []
    }

    for url in missing_urls:
        # Znajdź źródło tego URL
        source = next((item['source'] for item in source_urls if item['url'] == url), 'unknown')

        url_info = {
            'url': url,
            'source': source,
            'length': len(url),
            'issues': []
        }

        # Sprawdź długość
        if len(url) > 500:
            error_categories['too_long'].append(url_info)
            url_info['issues'].append(f'Długość: {len(url)} znaków')

        # Sprawdź znaki specjalne
        problematic_chars = ['ą', 'ę', 'ć', 'ł', 'ń', 'ó', 'ś', 'ź', 'ż', '"', "'", '%', '&']
        found_chars = [char for char in problematic_chars if char in url]
        if found_chars:
            error_categories['special_chars'].append(url_info)
            url_info['issues'].append(f'Znaki: {", ".join(found_chars)}')

        # Sprawdź kodowanie
        try:
            url.encode('ascii')
        except UnicodeEncodeError:
            error_categories['encoding_issues'].append(url_info)
            url_info['issues'].append('Problemy z kodowaniem ASCII')

        # Sprawdź JSON
        try:
            test_json = json.dumps({
                "data": {
                    "title": "Test",
                    "content": "Test",
                    "url": url
                }
            }, ensure_ascii=False)
        except Exception as e:
            error_categories['json_issues'].append(url_info)
            url_info['issues'].append(f'JSON error: {str(e)[:30]}')

        # Jeśli nie ma specyficznych problemów
        if not url_info['issues']:
            error_categories['other'].append(url_info)
            url_info['issues'].append('Nieznany błąd')

    # Wyświetl kategorie błędów
    for category, urls in error_categories.items():
        if urls:
            category_names = {
                'too_long': '📏 ZA DŁUGIE URL-E',
                'special_chars': '🔤 ZNAKI SPECJALNE',
                'encoding_issues': '🔀 PROBLEMY KODOWANIA',
                'json_issues': '📄 BŁĘDY JSON',
                'other': '❓ INNE BŁĘDY'
            }

            print(f"\n{category_names[category]} ({len(urls)} URL-i):")

            for i, url_info in enumerate(urls[:5]):  # Pokaż tylko pierwsze 5
                print(f"   {i+1}. {url_info['url'][:80]}...")
                print(f"      Źródło: {url_info['source']}")
                print(f"      Problemy: {'; '.join(url_info['issues'])}")

            if len(urls) > 5:
                print(f"   ... i {len(urls) - 5} więcej")

# ============================================================================
# 🛠️ NAPRAWA BŁĘDÓW
# ============================================================================

def fix_problematic_urls(connection, source_urls, db_urls):
    """Spróbuj naprawić problematyczne URL-e"""

    source_set = {item['url'] for item in source_urls}
    db_set = set(db_urls.keys())
    missing_urls = source_set - db_set

    if not missing_urls:
        print("✅ Brak URL-i do naprawy")
        return

    print(f"\n🛠️ PRÓBA NAPRAWY {len(missing_urls)} URL-I")
    print("=" * 40)

    choice = input("🛠️ Spróbować naprawić URL-e? (t/n): ").strip().lower()
    if choice not in ['t', 'tak', 'y', 'yes']:
        return

    fixed_count = 0
    still_failed = 0

    insert_query = """
    INSERT INTO articles (link, text, text_status, source)
    VALUES (%s, %s, 'pending', %s)
    """

    for url in missing_urls:
        # Znajdź źródło
        source = next((item['source'] for item in source_urls if item['url'] == url), 'unknown')

        try:
            # Skróć URL jeśli za długi
            fixed_url = url
            if len(url) > 450:
                fixed_url = url[:450]

            # Utwórz bezpieczny JSON
            safe_json = json.dumps({
                "data": {
                    "title": "Brak tytułu",
                    "content": "",
                    "url": url  # Zachowaj oryginalny URL w JSON
                }
            }, ensure_ascii=False)

            # Spróbuj wstawić
            cursor = connection.cursor()
            cursor.execute(insert_query, (fixed_url, safe_json, source))
            connection.commit()
            cursor.close()

            fixed_count += 1

        except Exception as e:
            still_failed += 1
            # Rollback i kontynuuj
            connection.rollback()
            if 'cursor' in locals():
                cursor.close()

    print(f"📊 WYNIKI NAPRAWY:")
    print(f"   ✅ Naprawione: {fixed_count}")
    print(f"   ❌ Nadal błędne: {still_failed}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja debugowania"""

    print("🔍 DEBUGOWANIE BŁĘDÓW WSTAWIANIA URL-I")
    print("=" * 50)

    # Połącz z bazą
    secrets = get_secrets()

    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")

        # Pobierz URL-e ze źródeł
        source_urls = get_all_source_urls()

        # Pobierz URL-e z bazy
        db_urls = get_database_urls(connection)

        # Analizuj błędy
        analyze_failed_urls(source_urls, db_urls)

        # Opcja naprawy
        fix_problematic_urls(connection, source_urls, db_urls)

        # Pokaż końcowe statystyki
        print(f"\n📊 KOŃCOWE STATYSTYKI:")
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT(*) FROM articles")
        final_count = cursor.fetchone()[0]
        cursor.close()

        print(f"   🗄️ URL-i w bazie: {final_count}")
        print(f"   📥 URL-i ze źródeł: {len(source_urls)}")
        print(f"   📈 Procent sukcesu: {final_count/len(source_urls)*100:.1f}%")

    except Exception as e:
        print(f"❌ Błąd: {e}")

    finally:
        if 'connection' in locals():
            connection.close()
            print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🔍 DEBUGOWANIE BŁĘDÓW WSTAWIANIA URL-I
✅ PostgreSQL połączony
🗺️ Pobieranie URL-i z sitemap...


INFO:root:Getting https://cateringfoodharmony.pl/sitemap.xml


✅ Sitemap: 99 URL-i
📄 Pobieranie URL-i z Excel...
✅ Excel: 124 URL-i
📊 Łącznie ze źródeł: 223 URL-i
🗄️ W bazie: 174 URL-i

🔍 ANALIZA BŁĘDÓW WSTAWIANIA
📊 STATYSTYKI:
   📥 URL-e ze źródeł: 223
   🔄 Duplikaty w źródłach: 49
   📥 Unikalne ze źródeł: 174
   🗄️ Wstawione do bazy: 174
   ❌ Nie udało się wstawić: 0
✅ Wszystkie URL-e zostały wstawione!
✅ Brak URL-i do naprawy

📊 KOŃCOWE STATYSTYKI:
   🗄️ URL-i w bazie: 174
   📥 URL-i ze źródeł: 223
   📈 Procent sukcesu: 78.0%
🔒 Połączenie zamknięte


# 2. **CRAWLOWANIE**

In [28]:
#!/usr/bin/env python3
"""
🕷️ TEST CRAWLINGU - 10 URL-i
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import json
import time
from datetime import datetime
from google.colab import userdata

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD', 'JINA_API_KEY']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🕷️ TEST CRAWLINGU
# ============================================================================

def test_crawling(connection, jina_api_key, limit=10):
    """Test crawlingu na ograniczonej liczbie URL-i"""

    print(f"🕷️ TEST CRAWLINGU - {limit} URL-i")
    print("=" * 50)

    # Pobierz pending URL-e
    cursor = connection.cursor(cursor_factory=RealDictCursor)
    cursor.execute("""
        SELECT id, link
        FROM articles
        WHERE text_status = 'pending'
        ORDER BY created_at ASC
        LIMIT %s
    """, (limit,))

    pending_urls = cursor.fetchall()
    cursor.close()

    if not pending_urls:
        print("🤷 Brak URL-i do crawlingu (wszystkie już przetworzone)")
        return 0, 0

    print(f"📋 Znaleziono {len(pending_urls)} URL-i do testowania:")
    for i, url_data in enumerate(pending_urls, 1):
        print(f"   {i}. {url_data['link']}")

    print(f"\n🚀 START CRAWLINGU...")
    print("-" * 50)

    success_count = 0
    error_count = 0

    # Crawl po kolei z szczegółowym logowaniem
    for i, url_data in enumerate(pending_urls, 1):
        print(f"\n📄 [{i}/{len(pending_urls)}] Crawling:")
        print(f"   🔗 URL: {url_data['link']}")
        print(f"   🆔 ID: {url_data['id']}")

        try:
            # Wywołaj Jina AI
            api_url = f"https://r.jina.ai/{url_data['link']}"
            headers = {
                "Accept": "application/json",
                "Authorization": f"Bearer {jina_api_key}"
            }

            print(f"   ⏳ Wysyłam request do Jina AI...")
            start_time = time.time()

            response = requests.get(api_url, headers=headers, timeout=30)

            end_time = time.time()
            response_time = end_time - start_time
            print(f"   ⚡ Odpowiedź w {response_time:.2f}s")
            print(f"   📊 Status: {response.status_code}")

            if response.status_code == 200:
                content = response.text
                print(f"   📝 Długość odpowiedzi: {len(content)} znaków")

                # Sprawdź rate limit
                if 'Whoa there, turbo!' in content or 'rate limit' in content.lower():
                    print(f"   ⏳ RATE LIMIT - czekam 30s...")
                    time.sleep(30)
                    error_count += 1
                    continue

                # Parse treści
                try:
                    json_data = json.loads(content)
                    title = json_data.get('data', {}).get('title', 'Brak tytułu')
                    article_content = json_data.get('data', {}).get('content', content)

                    print(f"   📰 Tytuł: {title[:50]}...")
                    print(f"   📄 Treść: {len(article_content)} znaków")

                except json.JSONDecodeError:
                    print(f"   ⚠️ JSON decode error - używam raw content")
                    title = 'Artykuł'
                    article_content = content

                # Utwórz strukturę JSON
                structured_text = json.dumps({
                    "data": {
                        "title": title,
                        "content": article_content,
                        "url": url_data['link']
                    }
                }, ensure_ascii=False)

                # Zapisz do bazy
                cursor = connection.cursor()
                cursor.execute("""
                    UPDATE articles
                    SET text = %s, text_status = 'completed', crawled_at = %s
                    WHERE id = %s
                """, (structured_text, datetime.now(), url_data['id']))
                connection.commit()
                cursor.close()

                print(f"   ✅ SUKCES - zapisano do bazy")
                success_count += 1

            else:
                # Błąd HTTP
                error_msg = f'error_http_{response.status_code}'

                cursor = connection.cursor()
                cursor.execute("""
                    UPDATE articles
                    SET text_status = %s, crawled_at = %s
                    WHERE id = %s
                """, (error_msg, datetime.now(), url_data['id']))
                connection.commit()
                cursor.close()

                print(f"   ❌ HTTP ERROR: {response.status_code}")
                error_count += 1

        except requests.exceptions.Timeout:
            print(f"   ⏰ TIMEOUT - URL nie odpowiedział w 30s")
            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = 'error_timeout', crawled_at = %s
                WHERE id = %s
            """, (datetime.now(), url_data['id']))
            connection.commit()
            cursor.close()
            error_count += 1

        except requests.exceptions.RequestException as e:
            print(f"   🌐 REQUEST ERROR: {str(e)[:100]}...")
            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = 'error_request', crawled_at = %s
                WHERE id = %s
            """, (datetime.now(), url_data['id']))
            connection.commit()
            cursor.close()
            error_count += 1

        except Exception as e:
            print(f"   ⚠️ UNKNOWN ERROR: {str(e)[:100]}...")
            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = 'error_general', crawled_at = %s
                WHERE id = %s
            """, (datetime.now(), url_data['id']))
            connection.commit()
            cursor.close()
            error_count += 1

        # Pauza między requestami
        if i < len(pending_urls):  # Nie czekaj po ostatnim
            print(f"   😴 Pauza 2s przed następnym URL...")
            time.sleep(2)

    return success_count, error_count

# ============================================================================
# 📊 STATYSTYKI PO TEŚCIE
# ============================================================================

def show_test_stats(connection):
    """Pokaż szczegółowe statystyki po teście"""

    print(f"\n📊 SZCZEGÓŁOWE STATYSTYKI PO TEŚCIE:")
    print("=" * 60)

    try:
        cursor = connection.cursor()

        # Status breakdown
        cursor.execute("""
            SELECT text_status, COUNT(*) as count
            FROM articles
            GROUP BY text_status
            ORDER BY count DESC
        """)

        print(f"{'Status':<25} {'Liczba':<10} {'Procent':<10}")
        print("-" * 50)

        total = 0
        stats = cursor.fetchall()

        # Najpierw policz total
        for status, count in stats:
            total += count

        # Potem wyświetl z procentami
        for status, count in stats:
            percent = (count / total * 100) if total > 0 else 0
            print(f"{status:<25} {count:<10} {percent:.1f}%")

        print("-" * 50)
        print(f"{'ŁĄCZNIE':<25} {total:<10} 100.0%")

        # Szczegóły ostatnich crawli
        print(f"\n🔍 OSTATNIE CRAWLE:")
        cursor.execute("""
            SELECT id, link, text_status, crawled_at
            FROM articles
            WHERE crawled_at IS NOT NULL
            ORDER BY crawled_at DESC
            LIMIT 10
        """)

        recent = cursor.fetchall()
        for row in recent:
            status_emoji = "✅" if row[2] == "completed" else "❌"
            print(f"   {status_emoji} {row[2]} | {row[1][:50]}...")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd statystyk: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA TESTOWA
# ============================================================================

def main():
    """Główna funkcja testowa"""

    print("🧪 TEST CRAWLINGU")
    print("=" * 40)

    # Pobierz sekrety
    secrets = get_secrets()

    if not secrets.get('JINA_API_KEY'):
        print("❌ Brak JINA_API_KEY w sekretach!")
        print("💡 Dodaj JINA_API_KEY w ustawieniach Colab")
        return

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # Test crawlingu
        print(f"\n🎯 URUCHAMIAM TEST CRAWLINGU...")

        success, errors = test_crawling(connection, secrets['JINA_API_KEY'], limit=10)

        print(f"\n🎯 WYNIKI TESTU:")
        print("=" * 30)
        print(f"   ✅ Sukces: {success}")
        print(f"   ❌ Błędy: {errors}")
        print(f"   📊 Łącznie: {success + errors}")

        if success + errors > 0:
            success_rate = (success / (success + errors)) * 100
            print(f"   📈 Skuteczność: {success_rate:.1f}%")

        # Pokaż szczegółowe statystyki
        show_test_stats(connection)

        print(f"\n🎉 TEST ZAKOŃCZONY!")

        if errors > 0:
            print(f"\n💡 WSKAZÓWKI:")
            print(f"   • Sprawdź API key Jina AI")
            print(f"   • Niektóre URL-e mogą być niedostępne")
            print(f"   • Rate limiting może spowalniać proces")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🧪 TEST CRAWLINGU
✅ PostgreSQL połączony

🎯 URUCHAMIAM TEST CRAWLINGU...
🕷️ TEST CRAWLINGU - 10 URL-i
📋 Znaleziono 10 URL-i do testowania:
   1. https://cateringfoodharmony.pl/
   2. https://cateringfoodharmony.pl/poznajmy-sie
   3. https://cateringfoodharmony.pl/kontakt
   4. https://cateringfoodharmony.pl/diety
   5. https://cateringfoodharmony.pl/detox
   6. https://cateringfoodharmony.pl/blog
   7. https://cateringfoodharmony.pl/cennik
   8. https://cateringfoodharmony.pl/rejony-dostaw
   9. https://cateringfoodharmony.pl/fit-harmony
   10. https://cateringfoodharmony.pl/free-harmony

🚀 START CRAWLINGU...
--------------------------------------------------

📄 [1/10] Crawling:
   🔗 URL: https://cateringfoodharmony.pl/
   🆔 ID: 1
   ⏳ Wysyłam request do Jina AI...
   ⚡ Odpowiedź w 0.57s
   📊 Status: 200
   📝 Długość odpowiedzi: 28252 znaków
   📰 Tytuł: Catering dietetyczny - dieta pudełkowa...
   📄 Treść: 27208 znaków
   ✅ SUKCES - zapisano do bazy
   😴 Pauza 2s przed następnym URL...


## 2 Crowlowanie wszystko**

In [1]:
#!/usr/bin/env python3
"""
🔧 POPRAWIONY CRAWLER - Fix dla timeout-ów na cateringfoodharmony.pl
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import json
import time
from datetime import datetime, timedelta
from google.colab import userdata
from tqdm import tqdm
import random

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD', 'JINA_API_KEY']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔧 NAPRAWIONY CRAWLING - Anti-Timeout
# ============================================================================

def improved_crawling_with_retries(connection, jina_api_key, batch_size=25):
    """Poprawiony crawling z retry logic dla timeout-ów"""

    print(f"🔧 IMPROVED CRAWLER - Anti-Timeout Settings")
    print("=" * 60)

    # Sprawdź URL-e do retry (timeout + 503 + pending)
    cursor = connection.cursor()
    cursor.execute("""
        SELECT COUNT(*) FROM articles
        WHERE text_status IN ('error_timeout', 'error_http_503', 'pending')
    """)
    total_pending = cursor.fetchone()[0]
    cursor.close()

    if total_pending == 0:
        print("✅ Brak URL-i do przetworzenia!")
        return 0, 0

    print(f"📊 URL-i do przetworzenia: {total_pending}")
    print(f"📦 Rozmiar batcha: {batch_size} (zmniejszony dla stabilności)")
    print(f"🔢 Liczba batchów: {(total_pending + batch_size - 1) // batch_size}")

    # IMPROVED SETTINGS
    print(f"\n🛠️ IMPROVED SETTINGS:")
    print(f"   ⏰ Timeout: 90s (było 30s)")
    print(f"   😴 Pauza między URL: 4-8s (było 1.5s)")
    print(f"   🔄 Max retry: 2x na URL")
    print(f"   📦 Batch size: {batch_size} (było 50)")
    print(f"   🚦 Rate limit handling: Ulepszone")

    # Statystyki globalne
    total_success = 0
    total_errors = 0
    start_time = datetime.now()

    # Przetwarzaj po batchach
    batch_num = 1

    while True:
        print(f"\n🚀 BATCH {batch_num}")
        print("-" * 40)

        # Pobierz następny batch (priorytet timeout-om)
        cursor = connection.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT id, link, text_status
            FROM articles
            WHERE text_status IN ('error_timeout', 'error_http_503', 'pending')
            ORDER BY
                CASE
                    WHEN text_status = 'error_timeout' THEN 1
                    WHEN text_status = 'error_http_503' THEN 2
                    ELSE 3
                END,
                created_at ASC
            LIMIT %s
        """, (batch_size,))

        batch_urls = cursor.fetchall()
        cursor.close()

        if not batch_urls:
            print("✅ Koniec - wszystkie URL-e przetworzone!")
            break

        print(f"📋 Batch zawiera {len(batch_urls)} URL-i")

        # Pokaż breakdown statusów w batchu
        status_count = {}
        for url in batch_urls:
            status = url['text_status']
            status_count[status] = status_count.get(status, 0) + 1

        for status, count in status_count.items():
            print(f"   • {status}: {count}")

        # Crawl batcha z improved settings
        batch_success, batch_errors = improved_crawl_batch(
            connection, jina_api_key, batch_urls, batch_num
        )

        # Aktualizuj statystyki
        total_success += batch_success
        total_errors += batch_errors

        # Podsumowanie batcha
        batch_total = batch_success + batch_errors
        batch_rate = (batch_success / batch_total * 100) if batch_total > 0 else 0

        print(f"\n📊 Podsumowanie Batch {batch_num}:")
        print(f"   ✅ Sukces: {batch_success}")
        print(f"   ❌ Błędy: {batch_errors}")
        print(f"   📈 Skuteczność: {batch_rate:.1f}%")

        # Statystyki globalne
        elapsed = datetime.now() - start_time
        processed_total = total_success + total_errors
        remaining = total_pending - processed_total

        if processed_total > 0:
            global_rate = (total_success / processed_total * 100)
            avg_time_per_url = elapsed.total_seconds() / processed_total
            estimated_remaining = timedelta(seconds=remaining * avg_time_per_url)

            print(f"\n🌍 Statystyki globalne:")
            print(f"   📊 Naprawione: {processed_total}/{total_pending}")
            print(f"   📈 Globalna skuteczność: {global_rate:.1f}%")
            print(f"   ⏱️ Czas na URL: {avg_time_per_url:.1f}s")
            print(f"   🕐 Pozostały czas: ~{estimated_remaining}")

        batch_num += 1

        # Dłuższa pauza między batchami
        if batch_urls:
            print(f"\n😴 Pauza 10s między batchami...")
            time.sleep(10)

    return total_success, total_errors

def improved_crawl_batch(connection, jina_api_key, urls_batch, batch_num):
    """Improved crawl batcha z retry logic"""

    success_count = 0
    error_count = 0

    # Progress bar dla batcha
    pbar = tqdm(urls_batch,
                desc=f"Batch {batch_num}",
                ncols=100,
                bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}")

    for url_data in pbar:
        url_id = url_data['id']
        url = url_data['link']
        original_status = url_data['text_status']

        # Update progress bar
        domain = url.split('/')[2] if '/' in url else 'unknown'
        pbar.set_postfix_str(f"{domain[:25]}... | {original_status}")

        success = False
        max_retries = 2 if original_status == 'error_timeout' else 1

        # Retry loop
        for attempt in range(max_retries + 1):
            try:
                if attempt > 0:
                    retry_delay = 30 + (attempt * 15)  # Progressive delay
                    pbar.write(f"   🔄 Retry #{attempt} dla {domain} (czekam {retry_delay}s)")
                    time.sleep(retry_delay)

                # Improved headers
                api_url = f"https://r.jina.ai/{url}"
                headers = {
                    "Accept": "application/json",
                    "Authorization": f"Bearer {jina_api_key}",
                    "User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)",
                    "Accept-Encoding": "gzip, deflate",
                    "Connection": "keep-alive",
                    "Cache-Control": "no-cache"
                }

                # Improved timeout settings
                timeout_seconds = 90 if original_status == 'error_timeout' else 60

                response = requests.get(
                    api_url,
                    headers=headers,
                    timeout=timeout_seconds,
                    allow_redirects=True
                )

                if response.status_code == 200:
                    content = response.text

                    # Enhanced rate limit detection
                    rate_limit_phrases = [
                        'whoa there, turbo', 'rate limit', 'too many requests',
                        'please slow down', 'exceeded', 'quota'
                    ]

                    if any(phrase in content.lower() for phrase in rate_limit_phrases):
                        delay = 45 + (attempt * 15)
                        pbar.write(f"   ⏳ Rate limit - czekam {delay}s")
                        time.sleep(delay)
                        continue

                    # Quality checks
                    if len(content.strip()) < 100:
                        pbar.write(f"   ⚠️ Zbyt krótka odpowiedź ({len(content)} znaków)")
                        if attempt < max_retries:
                            continue
                        else:
                            break

                    # Parse content
                    try:
                        json_data = json.loads(content)
                        title = json_data.get('data', {}).get('title', 'Brak tytułu')
                        article_content = json_data.get('data', {}).get('content', content)

                        # Content quality validation
                        if len(article_content.strip()) < 200:
                            pbar.write(f"   ⚠️ Słaba jakość treści")
                            if attempt < max_retries:
                                continue

                    except json.JSONDecodeError:
                        if attempt < max_retries:
                            pbar.write(f"   ⚠️ JSON error - retry")
                            continue
                        title = 'Artykuł'
                        article_content = content

                    # Create enhanced JSON structure
                    structured_text = json.dumps({
                        "data": {
                            "title": title,
                            "content": article_content,
                            "url": url,
                            "crawl_info": {
                                "retry_attempt": attempt + 1,
                                "original_status": original_status,
                                "timeout_used": timeout_seconds,
                                "content_length": len(article_content)
                            }
                        }
                    }, ensure_ascii=False)

                    # Save to database
                    cursor = connection.cursor()
                    cursor.execute("""
                        UPDATE articles
                        SET text = %s, text_status = 'completed', crawled_at = %s
                        WHERE id = %s
                    """, (structured_text, datetime.now(), url_id))
                    connection.commit()
                    cursor.close()

                    success = True
                    success_count += 1

                    if attempt > 0:
                        pbar.write(f"   ✅ RETRY SUKCES po {attempt + 1} próbach!")

                    break  # Success - exit retry loop

                elif response.status_code == 429:
                    # Enhanced rate limiting handling
                    delay = 60 + (attempt * 30)
                    pbar.write(f"   🚦 Rate limit 429 - czekam {delay}s")
                    time.sleep(delay)
                    continue

                elif response.status_code in [503, 502, 504, 408]:
                    # Server errors - retry with longer delay
                    if attempt < max_retries:
                        delay = 45 + (attempt * 20)
                        pbar.write(f"   🚨 Server error {response.status_code} - czekam {delay}s")
                        time.sleep(delay)
                        continue
                    else:
                        pbar.write(f"   💀 Final server error {response.status_code}")
                        break

                else:
                    # Other HTTP errors
                    pbar.write(f"   ❌ HTTP {response.status_code}")
                    break

            except requests.exceptions.Timeout:
                if attempt < max_retries:
                    delay = 60 + (attempt * 30)
                    pbar.write(f"   ⏰ Timeout #{attempt + 1} - czekam {delay}s")
                    time.sleep(delay)
                    continue
                else:
                    pbar.write(f"   💀 Final timeout po {max_retries + 1} próbach")
                    break

            except requests.exceptions.ConnectionError as e:
                if attempt < max_retries:
                    delay = 45 + (attempt * 15)
                    pbar.write(f"   🌐 Connection error - czekam {delay}s")
                    time.sleep(delay)
                    continue
                else:
                    pbar.write(f"   💀 Final connection error")
                    break

            except Exception as e:
                pbar.write(f"   ⚠️ Unexpected: {str(e)[:40]}...")
                break

        # Handle failed attempts
        if not success:
            error_count += 1

            # Determine final error status
            final_status = f"{original_status}_final" if original_status.startswith('error_') else 'error_final'

            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = %s, crawled_at = %s
                WHERE id = %s
            """, (final_status, datetime.now(), url_id))
            connection.commit()
            cursor.close()

        # Random delay between requests (anti-detection)
        delay = random.uniform(4, 8)  # Increased from 1.5s
        time.sleep(delay)

    pbar.close()
    return success_count, error_count

# ============================================================================
# 📊 KOŃCOWE STATYSTYKI Z ANALIZĄ
# ============================================================================

def show_improved_stats(connection):
    """Pokaż końcowe statystyki z analizą ulepszeń"""

    print(f"\n📊 STATYSTYKI PO IMPROVED CRAWLINGU:")
    print("=" * 60)

    try:
        cursor = connection.cursor()

        # Status breakdown
        cursor.execute("""
            SELECT text_status, COUNT(*) as count
            FROM articles
            GROUP BY text_status
            ORDER BY count DESC
        """)

        stats = cursor.fetchall()
        total = sum(count for status, count in stats)

        print(f"{'Status':<30} {'Liczba':<10} {'Procent':<10}")
        print("-" * 55)

        for status, count in stats:
            percent = (count / total * 100) if total > 0 else 0
            emoji = "✅" if status == "completed" else "❌"
            print(f"{emoji} {status:<27} {count:<10} {percent:.1f}%")

        print("-" * 55)
        print(f"{'ŁĄCZNIE':<30} {total:<10} 100.0%")

        # Analiza napraw
        cursor.execute("""
            SELECT
                COUNT(CASE WHEN text_status = 'completed' THEN 1 END) as fixed,
                COUNT(CASE WHEN text_status LIKE '%timeout%' THEN 1 END) as still_timeout,
                COUNT(CASE WHEN text_status LIKE '%503%' THEN 1 END) as still_503
            FROM articles
        """)

        repair_stats = cursor.fetchone()

        print(f"\n🔧 ANALIZA NAPRAW:")
        print(f"   ✅ Naprawione URL-e: {repair_stats[0]}")
        print(f"   ⏰ Nadal timeout: {repair_stats[1]}")
        print(f"   🚨 Nadal 503: {repair_stats[2]}")

        if repair_stats[1] > 0:
            print(f"\n💡 ZALECENIA dla pozostałych timeout-ów:")
            print(f"   • Spróbuj crawlować w godzinach nocnych (mniej obciążenia)")
            print(f"   • Zwiększ timeout do 120-180s")
            print(f"   • Sprawdź czy serwer nie blokuje IP")
            print(f"   • Rozważ użycie proxy/VPN")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd statystyk: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja improved crawlingu"""

    print("🔧 IMPROVED CRAWLER - Fix dla Timeout-ów")
    print("=" * 50)

    # Pobierz sekrety
    secrets = get_secrets()

    if not secrets.get('JINA_API_KEY'):
        print("❌ Brak JINA_API_KEY w sekretach!")
        return

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # Improved crawling
        print(f"\n🚀 URUCHAMIAM IMPROVED CRAWLING...")

        start_time = datetime.now()
        success, errors = improved_crawling_with_retries(
            connection,
            secrets['JINA_API_KEY'],
            batch_size=25  # Zmniejszony batch size
        )
        end_time = datetime.now()

        # Podsumowanie końcowe
        total_processed = success + errors
        duration = end_time - start_time

        print(f"\n🎯 PODSUMOWANIE IMPROVED CRAWLINGU:")
        print("=" * 45)
        print(f"   ✅ Naprawione: {success}")
        print(f"   ❌ Nadal błędy: {errors}")
        print(f"   📊 Łącznie: {total_processed}")
        print(f"   ⏱️ Czas: {duration}")

        if total_processed > 0:
            success_rate = (success / total_processed) * 100
            rate_per_min = total_processed / (duration.total_seconds() / 60) if duration.total_seconds() > 0 else 0
            print(f"   📈 Skuteczność: {success_rate:.1f}%")
            print(f"   ⚡ Prędkość: {rate_per_min:.1f} URL/min")

        # Szczegółowe statystyki
        show_improved_stats(connection)

        print(f"\n🎉 IMPROVED CRAWLING ZAKOŃCZONY!")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🔧 IMPROVED CRAWLER - Fix dla Timeout-ów
✅ PostgreSQL połączony

🚀 URUCHAMIAM IMPROVED CRAWLING...
🔧 IMPROVED CRAWLER - Anti-Timeout Settings
📊 URL-i do przetworzenia: 138
📦 Rozmiar batcha: 25 (zmniejszony dla stabilności)
🔢 Liczba batchów: 6

🛠️ IMPROVED SETTINGS:
   ⏰ Timeout: 90s (było 30s)
   😴 Pauza między URL: 4-8s (było 1.5s)
   🔄 Max retry: 2x na URL
   📦 Batch size: 25 (było 50)
   🚦 Rate limit handling: Ulepszone

🚀 BATCH 1
----------------------------------------
📋 Batch zawiera 25 URL-i
   • error_timeout: 25


Batch 1: 100%|█████████████████████| 25/25 [14:44<00:00] , cateringfoodharmony.pl... | error_timeout



📊 Podsumowanie Batch 1:
   ✅ Sukces: 25
   ❌ Błędy: 0
   📈 Skuteczność: 100.0%

🌍 Statystyki globalne:
   📊 Naprawione: 25/138
   📈 Globalna skuteczność: 100.0%
   ⏱️ Czas na URL: 35.4s
   🕐 Pozostały czas: ~1:06:39.209885

😴 Pauza 10s między batchami...

🚀 BATCH 2
----------------------------------------
📋 Batch zawiera 25 URL-i
   • error_timeout: 25


Batch 2: 100%|█████████████████████| 25/25 [13:40<00:00] , cateringfoodharmony.pl... | error_timeout



📊 Podsumowanie Batch 2:
   ✅ Sukces: 25
   ❌ Błędy: 0
   📈 Skuteczność: 100.0%

🌍 Statystyki globalne:
   📊 Naprawione: 50/138
   📈 Globalna skuteczność: 100.0%
   ⏱️ Czas na URL: 34.3s
   🕐 Pozostały czas: ~0:50:19.040211

😴 Pauza 10s między batchami...

🚀 BATCH 3
----------------------------------------
📋 Batch zawiera 25 URL-i
   • error_timeout: 25


...


📊 Podsumowanie Batch 4:
   ✅ Sukces: 25
   ❌ Błędy: 0
   📈 Skuteczność: 100.0%

🌍 Statystyki globalne:
   📊 Naprawione: 100/138
   📈 Globalna skuteczność: 100.0%
   ⏱️ Czas na URL: 34.7s
   🕐 Pozostały czas: ~0:21:56.702503

😴 Pauza 10s między batchami...

🚀 BATCH 5
----------------------------------------
📋 Batch zawiera 25 URL-i
   • error_timeout: 25


Batch 5: 100%|█████████████████████| 25/25 [14:53<00:00] , cateringfoodharmony.pl... | error_timeout



📊 Podsumowanie Batch 5:
   ✅ Sukces: 25
   ❌ Błędy: 0
   📈 Skuteczność: 100.0%

🌍 Statystyki globalne:
   📊 Naprawione: 125/138
   📈 Globalna skuteczność: 100.0%
   ⏱️ Czas na URL: 35.0s
   🕐 Pozostały czas: ~0:07:34.373355

😴 Pauza 10s między batchami...

🚀 BATCH 6
----------------------------------------
📋 Batch zawiera 13 URL-i
   • error_timeout: 12
   • error_http_503: 1


Batch 6: 100%|████████████████████| 13/13 [08:16<00:00] , cateringfoodharmony.pl... | error_http_503



📊 Podsumowanie Batch 6:
   ✅ Sukces: 13
   ❌ Błędy: 0
   📈 Skuteczność: 100.0%

🌍 Statystyki globalne:
   📊 Naprawione: 138/138
   📈 Globalna skuteczność: 100.0%
   ⏱️ Czas na URL: 35.3s
   🕐 Pozostały czas: ~0:00:00

😴 Pauza 10s między batchami...

🚀 BATCH 7
----------------------------------------
✅ Koniec - wszystkie URL-e przetworzone!

🎯 PODSUMOWANIE IMPROVED CRAWLINGU:
   ✅ Naprawione: 138
   ❌ Nadal błędy: 0
   📊 Łącznie: 138
   ⏱️ Czas: 1:21:27.387575
   📈 Skuteczność: 100.0%
   ⚡ Prędkość: 1.7 URL/min

📊 STATYSTYKI PO IMPROVED CRAWLINGU:
Status                         Liczba     Procent   
-------------------------------------------------------
✅ completed                   174        100.0%
-------------------------------------------------------
ŁĄCZNIE                        174        100.0%

🔧 ANALIZA NAPRAW:
   ✅ Naprawione URL-e: 174
   ⏰ Nadal timeout: 0
   🚨 Nadal 503: 0

🎉 IMPROVED CRAWLING ZAKOŃCZONY!
🔒 Połączenie zamknięte


## 2.2 *Crowlowanie* stron z proxy Geonode

In [2]:
#!/usr/bin/env python3
"""
🇵🇱 CRAWLER Z POLSKIM RESIDENTIAL PROXY
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import json
import time
from datetime import datetime, timedelta
from google.colab import userdata
from tqdm import tqdm
import random

# ============================================================================
# 🔑 KONFIGURACJA Z PROXY
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD', 'JINA_API_KEY']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

def get_proxy_config():
    """Konfiguracja polskiego residential proxy"""
    return {
        'host': '192.155.103.209',
        'port': '9000',
        'username': 'geonode_M9FqVRJQ7A-type-residential-country-pl',
        'password': '5b2a8768-bddc-4f2d-ac14-b28af8e669c3'
    }

def create_proxy_session():
    """Utwórz sesję z polskim proxy"""
    proxy_config = get_proxy_config()

    # Proxy URL
    proxy_url = f"http://{proxy_config['username']}:{proxy_config['password']}@{proxy_config['host']}:{proxy_config['port']}"

    # Create session
    session = requests.Session()
    session.proxies = {
        'http': proxy_url,
        'https': proxy_url
    }

    # Enhanced headers for Polish traffic
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'pl-PL,pl;q=0.9,en;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    })

    return session

def test_proxy_connection():
    """Test połączenia z proxy"""
    print("🧪 TESTOWANIE POLSKIEGO PROXY...")

    try:
        session = create_proxy_session()

        # Test 1: Check IP
        print("   🔍 Sprawdzanie IP...")
        ip_response = session.get('https://httpbin.org/ip', timeout=30)
        if ip_response.status_code == 200:
            ip_data = ip_response.json()
            print(f"   ✅ IP: {ip_data['origin']}")

        # Test 2: Check geolocation
        print("   🌍 Sprawdzanie lokalizacji...")
        geo_response = session.get('https://httpbin.org/headers', timeout=30)
        if geo_response.status_code == 200:
            print(f"   ✅ Headers OK")

        # Test 3: Test target domain
        print("   🎯 Test domeny docelowej...")
        test_response = session.get('https://cateringfoodharmony.pl', timeout=45)
        print(f"   📊 Status: {test_response.status_code}")
        print(f"   ⏱️ Response time: {test_response.elapsed.total_seconds():.2f}s")

        if test_response.status_code == 200:
            print("   ✅ PROXY DZIAŁA ŚWIETNIE! 🇵🇱")
            return True
        else:
            print("   ⚠️ Proxy działa, ale domena ma problemy")
            return True

    except Exception as e:
        print(f"   ❌ Błąd proxy: {e}")
        return False

# ============================================================================
# 🇵🇱 PROXY CRAWLER
# ============================================================================

def proxy_crawling_with_polish_ip(connection, jina_api_key, batch_size=30):
    """Crawling z polskim residential proxy"""

    print(f"🇵🇱 PROXY CRAWLER - Polski Residential IP")
    print("=" * 60)

    # Test proxy przed startem
    if not test_proxy_connection():
        print("❌ Proxy nie działa - przerwano!")
        return 0, 0

    # Sprawdź URL-e do przetworzenia
    cursor = connection.cursor()
    cursor.execute("""
        SELECT COUNT(*) FROM articles
        WHERE text_status IN ('error_timeout', 'error_http_503', 'pending', 'error_timeout_final')
    """)
    total_pending = cursor.fetchone()[0]
    cursor.close()

    if total_pending == 0:
        print("✅ Brak URL-i do przetworzenia!")
        return 0, 0

    print(f"📊 URL-i do przetworzenia: {total_pending}")
    print(f"📦 Rozmiar batcha: {batch_size}")

    # PROXY SETTINGS
    print(f"\n🇵🇱 PROXY SETTINGS:")
    print(f"   🌍 Country: Poland")
    print(f"   🏠 Type: Residential")
    print(f"   ⏰ Timeout: 60s")
    print(f"   😴 Delay: 2-4s (szybsze dzięki proxy)")
    print(f"   🔄 Max retry: 1x (proxy powinno rozwiązać problemy)")

    # Create proxy session
    session = create_proxy_session()

    # Statystyki
    total_success = 0
    total_errors = 0
    start_time = datetime.now()
    batch_num = 1

    while True:
        print(f"\n🚀 BATCH {batch_num}")
        print("-" * 40)

        # Pobierz następny batch
        cursor = connection.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT id, link, text_status
            FROM articles
            WHERE text_status IN ('error_timeout', 'error_http_503', 'pending', 'error_timeout_final')
            ORDER BY
                CASE
                    WHEN text_status LIKE '%timeout%' THEN 1
                    WHEN text_status = 'error_http_503' THEN 2
                    ELSE 3
                END,
                created_at ASC
            LIMIT %s
        """, (batch_size,))

        batch_urls = cursor.fetchall()
        cursor.close()

        if not batch_urls:
            print("✅ Koniec - wszystkie URL-e przetworzone!")
            break

        print(f"📋 Batch zawiera {len(batch_urls)} URL-i")

        # Crawl batch z proxy
        batch_success, batch_errors = proxy_crawl_batch(
            connection, jina_api_key, batch_urls, batch_num, session
        )

        total_success += batch_success
        total_errors += batch_errors

        # Podsumowanie batcha
        batch_total = batch_success + batch_errors
        batch_rate = (batch_success / batch_total * 100) if batch_total > 0 else 0

        print(f"\n📊 Podsumowanie Batch {batch_num}:")
        print(f"   ✅ Sukces: {batch_success}")
        print(f"   ❌ Błędy: {batch_errors}")
        print(f"   📈 Skuteczność: {batch_rate:.1f}%")

        batch_num += 1

        # Krótsza pauza dzięki proxy
        if batch_urls:
            print(f"\n😴 Pauza 3s między batchami...")
            time.sleep(3)

    return total_success, total_errors

def proxy_crawl_batch(connection, jina_api_key, urls_batch, batch_num, session):
    """Crawl batch z polskim proxy"""

    success_count = 0
    error_count = 0

    pbar = tqdm(urls_batch,
                desc=f"🇵🇱 Batch {batch_num}",
                ncols=100,
                bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}")

    for url_data in pbar:
        url_id = url_data['id']
        url = url_data['link']
        original_status = url_data['text_status']

        domain = url.split('/')[2] if '/' in url else 'unknown'
        pbar.set_postfix_str(f"🇵🇱 {domain[:25]}... | {original_status}")

        success = False
        max_retries = 1  # Proxy powinno rozwiązać większość problemów

        for attempt in range(max_retries + 1):
            try:
                if attempt > 0:
                    pbar.write(f"   🔄 Retry #{attempt} dla {domain}")
                    time.sleep(20)

                # Jina AI request przez polski proxy
                api_url = f"https://r.jina.ai/{url}"
                headers = {
                    "Accept": "application/json",
                    "Authorization": f"Bearer {jina_api_key}",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
                }

                # Request przez proxy session
                response = session.get(
                    api_url,
                    headers=headers,
                    timeout=60
                )

                if response.status_code == 200:
                    content = response.text

                    # Rate limit check
                    if any(phrase in content.lower() for phrase in [
                        'whoa there, turbo', 'rate limit', 'too many requests'
                    ]):
                        pbar.write(f"   ⏳ Rate limit - czekam 20s")
                        time.sleep(20)
                        continue

                    # Quality check
                    if len(content.strip()) < 100:
                        pbar.write(f"   ⚠️ Krótka odpowiedź ({len(content)} znaków)")
                        if attempt < max_retries:
                            continue
                        else:
                            break

                    # Parse content
                    try:
                        json_data = json.loads(content)
                        title = json_data.get('data', {}).get('title', 'Brak tytułu')
                        article_content = json_data.get('data', {}).get('content', content)
                    except json.JSONDecodeError:
                        title = 'Artykuł'
                        article_content = content

                    # Create JSON with proxy info
                    structured_text = json.dumps({
                        "data": {
                            "title": title,
                            "content": article_content,
                            "url": url,
                            "crawl_info": {
                                "proxy_used": "polish_residential",
                                "original_status": original_status,
                                "retry_attempt": attempt + 1,
                                "content_length": len(article_content)
                            }
                        }
                    }, ensure_ascii=False)

                    # Save to database
                    cursor = connection.cursor()
                    cursor.execute("""
                        UPDATE articles
                        SET text = %s, text_status = 'completed', crawled_at = %s
                        WHERE id = %s
                    """, (structured_text, datetime.now(), url_id))
                    connection.commit()
                    cursor.close()

                    success = True
                    success_count += 1

                    if original_status.startswith('error_'):
                        pbar.write(f"   ✅ PROXY NAPRAWIŁ {original_status}!")

                    break

                elif response.status_code in [429, 503]:
                    delay = 30 + (attempt * 15)
                    pbar.write(f"   🚦 {response.status_code} - czekam {delay}s")
                    time.sleep(delay)
                    continue

                else:
                    pbar.write(f"   ❌ HTTP {response.status_code}")
                    break

            except requests.exceptions.Timeout:
                if attempt < max_retries:
                    pbar.write(f"   ⏰ Timeout - retry")
                    time.sleep(30)
                    continue
                else:
                    pbar.write(f"   💀 Final timeout (nawet z proxy)")
                    break

            except Exception as e:
                pbar.write(f"   ⚠️ Error: {str(e)[:40]}...")
                break

        if not success:
            error_count += 1
            final_status = f"{original_status}_proxy_failed"

            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET text_status = %s, crawled_at = %s
                WHERE id = %s
            """, (final_status, datetime.now(), url_id))
            connection.commit()
            cursor.close()

        # Szybsze delay dzięki proxy
        delay = random.uniform(2, 4)  # Zmniejszone z 4-8s
        time.sleep(delay)

    pbar.close()
    return success_count, error_count

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja proxy crawlingu"""

    print("🇵🇱 PROXY CRAWLER - Polski Residential")
    print("=" * 50)

    # Pobierz sekrety
    secrets = get_secrets()

    if not secrets.get('JINA_API_KEY'):
        print("❌ Brak JINA_API_KEY!")
        return

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # Proxy crawling
        start_time = datetime.now()
        success, errors = proxy_crawling_with_polish_ip(
            connection,
            secrets['JINA_API_KEY'],
            batch_size=30
        )
        end_time = datetime.now()

        # Podsumowanie
        total_processed = success + errors
        duration = end_time - start_time

        print(f"\n🎯 WYNIKI PROXY CRAWLINGU:")
        print("=" * 40)
        print(f"   🇵🇱 Proxy: Polskie Residential")
        print(f"   ✅ Naprawione: {success}")
        print(f"   ❌ Nadal błędy: {errors}")
        print(f"   📊 Łącznie: {total_processed}")
        print(f"   ⏱️ Czas: {duration}")

        if total_processed > 0:
            success_rate = (success / total_processed) * 100
            print(f"   📈 Skuteczność z proxy: {success_rate:.1f}%")

        print(f"\n🎉 PROXY CRAWLING ZAKOŃCZONY!")

        if success > 100:  # Jeśli naprawił dużo URL-i
            print(f"\n🚀 PROXY BYŁO GAME CHANGEREM! 🇵🇱")
            print(f"   💰 Inwestycja w proxy się opłaciła")
            print(f"   🎯 Kolejne kroki: embeddingi i analiza")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🇵🇱 PROXY CRAWLER - Polski Residential
✅ PostgreSQL połączony
🇵🇱 PROXY CRAWLER - Polski Residential IP
🧪 TESTOWANIE POLSKIEGO PROXY...
   🔍 Sprawdzanie IP...
   ✅ IP: 94.75.105.242
   🌍 Sprawdzanie lokalizacji...
   ✅ Headers OK
   🎯 Test domeny docelowej...
   📊 Status: 200
   ⏱️ Response time: 4.41s
   ✅ PROXY DZIAŁA ŚWIETNIE! 🇵🇱
✅ Brak URL-i do przetworzenia!

🎯 WYNIKI PROXY CRAWLINGU:
   🇵🇱 Proxy: Polskie Residential
   ✅ Naprawione: 0
   ❌ Nadal błędy: 0
   📊 Łącznie: 0
   ⏱️ Czas: 0:00:08.325772

🎉 PROXY CRAWLING ZAKOŃCZONY!
🔒 Połączenie zamknięte


# 3.**DIAGNOSTYKA BAZY I DANYCh**

## 3.1 Sprawdzenie czy wszystkie dane są w bazie

In [3]:
#!/usr/bin/env python3
"""
🔍 SPRAWDZENIE STRUKTURY BAZY POSTGRESQL
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
from google.colab import userdata
from datetime import datetime

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔍 SPRAWDZENIE BAZY
# ============================================================================

def check_database_structure():
    """Sprawdź strukturę bazy danych"""

    print("🔍 SPRAWDZENIE STRUKTURY BAZY POSTGRESQL")
    print("=" * 60)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return False

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        # 1. SPRAWDŹ CZY TABELA ARTICLES ISTNIEJE
        print(f"\n📋 1. SPRAWDZANIE TABELI 'articles':")
        print("-" * 40)

        cursor.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'public'
                AND table_name = 'articles'
            );
        """)

        table_exists = cursor.fetchone()[0]

        if not table_exists:
            print("❌ Tabela 'articles' nie istnieje!")
            print("💡 Musisz uruchomić setup bazy z pierwszego skryptu")
            return False
        else:
            print("✅ Tabela 'articles' istnieje")

        # 2. SPRAWDŹ STRUKTURĘ KOLUMN
        print(f"\n📊 2. STRUKTURA KOLUMN:")
        print("-" * 40)

        cursor.execute("""
            SELECT column_name, data_type, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_schema = 'public'
            AND table_name = 'articles'
            ORDER BY ordinal_position;
        """)

        columns = cursor.fetchall()

        required_columns = {
            'id': 'integer',
            'link': 'text',
            'text': 'text',
            'text_status': 'character varying',
            'embedding': 'text',
            'title_embedding': 'text',
            'embedding_status': 'character varying',
            'crawled_at': 'timestamp'
        }

        print(f"{'Kolumna':<20} {'Typ':<25} {'Nullable':<10} {'Default':<15}")
        print("-" * 75)

        existing_columns = {}
        for col in columns:
            col_name = col['column_name']
            col_type = col['data_type']
            nullable = col['is_nullable']
            default = col['column_default'] or 'NULL'

            existing_columns[col_name] = col_type

            # Check if required
            emoji = "✅" if col_name in required_columns else "ℹ️"
            print(f"{emoji} {col_name:<18} {col_type:<23} {nullable:<8} {str(default)[:13]}")

        # 3. SPRAWDŹ BRAKUJĄCE KOLUMNY
        print(f"\n🔍 3. ANALIZA WYMAGANYCH KOLUMN:")
        print("-" * 40)

        missing_columns = []
        wrong_type_columns = []

        for req_col, req_type in required_columns.items():
            if req_col not in existing_columns:
                missing_columns.append(req_col)
                print(f"❌ Brakująca kolumna: {req_col} ({req_type})")
            elif req_type not in existing_columns[req_col] and req_col not in ['embedding', 'title_embedding']:
                # embedding kolumny mogą być różne typy (TEXT, JSON, etc.)
                wrong_type_columns.append((req_col, existing_columns[req_col], req_type))
                print(f"⚠️ Zły typ: {req_col} jest {existing_columns[req_col]}, powinno być {req_type}")
            else:
                print(f"✅ {req_col}: OK")

        # 4. SPRAWDŹ DANE W TABELI
        print(f"\n📊 4. ANALIZA DANYCH:")
        print("-" * 40)

        # Podstawowe statystyki
        cursor.execute("SELECT COUNT(*) FROM articles")
        total_rows = cursor.fetchone()[0]
        print(f"📦 Łączna liczba rekordów: {total_rows}")

        if total_rows == 0:
            print("⚠️ Tabela jest pusta - brak danych do przetworzenia!")
            return False

        # Status breakdown
        cursor.execute("""
            SELECT text_status, COUNT(*) as count
            FROM articles
            GROUP BY text_status
            ORDER BY count DESC
        """)

        text_statuses = cursor.fetchall()

        print(f"\n📈 Status text_status:")
        for status_row in text_statuses:
            status = status_row['text_status']
            count = status_row['count']
            percent = (count / total_rows * 100) if total_rows > 0 else 0
            emoji = "✅" if status == "completed" else "⚠️"
            print(f"   {emoji} {status}: {count} ({percent:.1f}%)")

        # Embedding status
        cursor.execute("""
            SELECT
                embedding_status,
                COUNT(*) as count
            FROM articles
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        embedding_statuses = cursor.fetchall()

        print(f"\n🔤 Status embedding_status:")
        for status_row in embedding_statuses:
            status = status_row['embedding_status'] or 'NULL'
            count = status_row['count']
            percent = (count / total_rows * 100) if total_rows > 0 else 0
            emoji = "✅" if status == "completed" else "📝" if status == "NULL" else "❌"
            print(f"   {emoji} {status}: {count} ({percent:.1f}%)")

        # 5. SPRAWDŹ PRZYKŁADOWE DANE
        print(f"\n📄 5. PRZYKŁADOWE DANE:")
        print("-" * 40)

        cursor.execute("""
            SELECT id, link, text_status, embedding_status,
                   LENGTH(text) as text_length,
                   LENGTH(embedding) as embedding_length
            FROM articles
            WHERE text_status = 'completed'
            ORDER BY id ASC
            LIMIT 3
        """)

        sample_rows = cursor.fetchall()

        for i, row in enumerate(sample_rows, 1):
            print(f"\n📄 Przykład {i}:")
            print(f"   🆔 ID: {row['id']}")
            print(f"   🔗 URL: {row['link'][:50]}...")
            print(f"   📊 Text status: {row['text_status']}")
            print(f"   🔤 Embedding status: {row['embedding_status'] or 'NULL'}")
            print(f"   📏 Text length: {row['text_length'] or 0} znaków")
            print(f"   📐 Embedding length: {row['embedding_length'] or 0} znaków")

            # Sprawdź strukturę JSON
            if row['text_length'] and row['text_length'] > 0:
                cursor.execute("SELECT text FROM articles WHERE id = %s", (row['id'],))
                text_content = cursor.fetchone()['text']

                try:
                    json_data = json.loads(text_content)
                    if 'data' in json_data:
                        title = json_data['data'].get('title', 'Brak')[:50]
                        content_len = len(json_data['data'].get('content', ''))
                        print(f"   📰 Title: {title}...")
                        print(f"   📄 Content: {content_len} znaków")
                    else:
                        print(f"   ⚠️ JSON bez 'data' key")
                except json.JSONDecodeError:
                    print(f"   ❌ Nieprawidłowy JSON")

        # 6. GOTOWOŚĆ NA EMBEDDINGI
        print(f"\n🎯 6. GOTOWOŚĆ NA EMBEDDINGI:")
        print("-" * 40)

        # Sprawdź ile artykułów ma completed text ale brak embeddingów
        cursor.execute("""
            SELECT COUNT(*)
            FROM articles
            WHERE text_status = 'completed'
            AND (embedding_status IS NULL OR embedding_status != 'completed')
        """)

        ready_for_embeddings = cursor.fetchone()[0]

        print(f"📊 Artykuły gotowe na embeddingi: {ready_for_embeddings}")

        if ready_for_embeddings == 0:
            print("⚠️ Brak artykułów gotowych na embeddingi!")
            print("💡 Sprawdź czy crawling się zakończył pomyślnie")
            return False
        elif ready_for_embeddings < 10:
            print("⚠️ Mało artykułów - sprawdź czy wszystko OK z crawlingiem")
        else:
            print("✅ Wystarczająco artykułów do przetworzenia")

        # 7. REKOMENDACJE
        print(f"\n💡 7. REKOMENDACJE:")
        print("-" * 40)

        issues_found = len(missing_columns) + len(wrong_type_columns)

        if issues_found == 0 and ready_for_embeddings > 0:
            print("🎉 BAZA GOTOWA NA EMBEDDINGI!")
            print("🚀 Możesz uruchomić generator embeddingów")
        else:
            if missing_columns:
                print(f"❌ Dodaj brakujące kolumny: {', '.join(missing_columns)}")
            if wrong_type_columns:
                print(f"⚠️ Popraw typy kolumn")
            if ready_for_embeddings == 0:
                print(f"📝 Najpierw ukończ crawling artykułów")

        return issues_found == 0 and ready_for_embeddings > 0

    except Exception as e:
        print(f"❌ Błąd sprawdzania bazy: {e}")
        return False

    finally:
        cursor.close()
        connection.close()
        print("\n🔒 Połączenie zamknięte")

# ============================================================================
# 🔧 NAPRAWY STRUKTURY
# ============================================================================

def fix_database_structure():
    """Napraw strukturę bazy jeśli potrzeba"""

    print("\n🔧 NAPRAWY STRUKTURY BAZY")
    print("=" * 40)

    # Pobierz sekrety
    secrets = get_secrets()

    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )

        cursor = connection.cursor()

        # Dodaj brakujące kolumny jeśli potrzeba
        alterations = [
            "ALTER TABLE articles ADD COLUMN IF NOT EXISTS embedding TEXT;",
            "ALTER TABLE articles ADD COLUMN IF NOT EXISTS title_embedding TEXT;",
            "ALTER TABLE articles ADD COLUMN IF NOT EXISTS embedding_status VARCHAR(50);",
            "ALTER TABLE articles ADD COLUMN IF NOT EXISTS candidates TEXT DEFAULT '[]';",
            "CREATE INDEX IF NOT EXISTS idx_articles_embedding_status ON articles(embedding_status);",
            "CREATE INDEX IF NOT EXISTS idx_articles_text_status ON articles(text_status);"
        ]

        for sql in alterations:
            try:
                cursor.execute(sql)
                print(f"✅ {sql[:50]}...")
            except Exception as e:
                print(f"⚠️ {sql[:30]}... - {e}")

        connection.commit()
        cursor.close()
        connection.close()

        print("✅ Naprawy struktury zakończone")
        return True

    except Exception as e:
        print(f"❌ Błąd napraw: {e}")
        return False

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja sprawdzenia"""

    # Sprawdź strukturę
    is_ready = check_database_structure()

    if not is_ready:
        print(f"\n🔧 Próbuję naprawić strukturę...")
        if fix_database_structure():
            print(f"\n🔄 Sprawdzam ponownie...")
            is_ready = check_database_structure()

    print(f"\n" + "=" * 60)
    if is_ready:
        print("🎉 BAZA JEST GOTOWA NA EMBEDDINGI!")
        print("🚀 Możesz uruchomić generator embeddingów")
    else:
        print("❌ BAZA WYMAGA NAPRAW")
        print("💡 Sprawdź błędy powyżej i popraw ręcznie")
    print("=" * 60)

if __name__ == "__main__":
    main()

🔍 SPRAWDZENIE STRUKTURY BAZY POSTGRESQL
✅ PostgreSQL połączony

📋 1. SPRAWDZANIE TABELI 'articles':
----------------------------------------
❌ Błąd sprawdzania bazy: 0

🔒 Połączenie zamknięte

🔧 Próbuję naprawić strukturę...

🔧 NAPRAWY STRUKTURY BAZY
✅ ALTER TABLE articles ADD COLUMN IF NOT EXISTS embe...
✅ ALTER TABLE articles ADD COLUMN IF NOT EXISTS titl...
✅ ALTER TABLE articles ADD COLUMN IF NOT EXISTS embe...
✅ ALTER TABLE articles ADD COLUMN IF NOT EXISTS cand...
✅ CREATE INDEX IF NOT EXISTS idx_articles_embedding_...
✅ CREATE INDEX IF NOT EXISTS idx_articles_text_statu...
✅ Naprawy struktury zakończone

🔄 Sprawdzam ponownie...
🔍 SPRAWDZENIE STRUKTURY BAZY POSTGRESQL
✅ PostgreSQL połączony

📋 1. SPRAWDZANIE TABELI 'articles':
----------------------------------------
❌ Błąd sprawdzania bazy: 0

🔒 Połączenie zamknięte

❌ BAZA WYMAGA NAPRAW
💡 Sprawdź błędy powyżej i popraw ręcznie


## 3.2 Diagonstyka bazy postgress

In [4]:
#!/usr/bin/env python3
"""
🔍 DIAGNOSTYKA BAZY POSTGRESQL - Szczegółowe sprawdzenie
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
from google.colab import userdata
import json

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔍 SZCZEGÓŁOWA DIAGNOSTYKA
# ============================================================================

def diagnose_database(connection):
    """Szczegółowa diagnostyka bazy danych"""

    print(f"🔍 SZCZEGÓŁOWA DIAGNOSTYKA BAZY")
    print("=" * 60)

    try:
        cursor = connection.cursor()

        # 1. Sprawdź podstawowe info o bazie
        print(f"\n📊 1. INFORMACJE O BAZIE:")
        print("-" * 40)

        cursor.execute("SELECT version();")
        version = cursor.fetchone()[0]
        print(f"   🗄️ PostgreSQL: {version}")

        cursor.execute("SELECT current_database();")
        db_name = cursor.fetchone()[0]
        print(f"   📂 Baza danych: {db_name}")

        cursor.execute("SELECT current_user;")
        user = cursor.fetchone()[0]
        print(f"   👤 Użytkownik: {user}")

        # 2. Sprawdź czy tabela 'articles' istnieje
        print(f"\n📋 2. SPRAWDZANIE TABELI 'articles':")
        print("-" * 40)

        cursor.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'public'
                AND table_name = 'articles'
            );
        """)

        table_exists = cursor.fetchone()[0]
        print(f"   📊 Tabela 'articles' istnieje: {table_exists}")

        if not table_exists:
            print("   ❌ TABELA 'articles' NIE ISTNIEJE!")
            print("   💡 Trzeba ją utworzyć")
            return False

        # 3. Sprawdź strukturę tabeli
        print(f"\n🏗️ 3. STRUKTURA TABELI 'articles':")
        print("-" * 40)

        cursor.execute("""
            SELECT
                column_name,
                data_type,
                character_maximum_length,
                is_nullable,
                column_default
            FROM information_schema.columns
            WHERE table_name = 'articles'
            ORDER BY ordinal_position;
        """)

        columns = cursor.fetchall()

        if not columns:
            print("   ❌ BRAK KOLUMN W TABELI!")
            return False

        print(f"   📊 Znaleziono {len(columns)} kolumn:")

        expected_columns = {
            'id', 'link', 'crawled_at', 'text', 'text_status',
            'embedding', 'title_embedding', 'embedding_status',
            'candidates', 'source', 'created_at'
        }

        found_columns = set()

        for col in columns:
            col_name = col[0]
            data_type = col[1]
            max_length = col[2] if col[2] else 'unlimited'
            nullable = "NULL" if col[3] == 'YES' else "NOT NULL"
            default = col[4] if col[4] else 'none'

            print(f"   • {col_name:<20} {data_type:<15} {nullable:<10}")
            found_columns.add(col_name)

        # 4. Sprawdź brakujące kolumny
        missing_columns = expected_columns - found_columns
        extra_columns = found_columns - expected_columns

        if missing_columns:
            print(f"\n⚠️ BRAKUJĄCE KOLUMNY ({len(missing_columns)}):")
            for col in sorted(missing_columns):
                print(f"   ❌ {col}")

        if extra_columns:
            print(f"\n🔧 DODATKOWE KOLUMNY ({len(extra_columns)}):")
            for col in sorted(extra_columns):
                print(f"   ➕ {col}")

        # 5. Sprawdź zawartość tabeli
        print(f"\n📊 4. ZAWARTOŚĆ TABELI:")
        print("-" * 40)

        cursor.execute("SELECT COUNT(*) FROM articles;")
        total_rows = cursor.fetchone()[0]
        print(f"   📄 Łączna liczba wierszy: {total_rows}")

        if total_rows > 0:
            # Status breakdown
            cursor.execute("""
                SELECT text_status, COUNT(*) as count
                FROM articles
                GROUP BY text_status
                ORDER BY count DESC;
            """)

            statuses = cursor.fetchall()
            print(f"   📈 Breakdown statusów:")
            for status, count in statuses:
                status_display = status if status else 'NULL'
                print(f"      • {status_display:<20} {count:>6}")

            # Przykładowe dane
            cursor.execute("SELECT id, link, text_status FROM articles LIMIT 3;")
            samples = cursor.fetchall()
            print(f"   📝 Przykładowe wiersze:")
            for sample in samples:
                link_short = sample[1][:40] + '...' if len(sample[1]) > 40 else sample[1]
                print(f"      • ID:{sample[0]:<5} {sample[2]:<15} {link_short}")

        # 6. Sprawdź indeksy
        print(f"\n🗂️ 5. INDEKSY:")
        print("-" * 40)

        cursor.execute("""
            SELECT
                indexname,
                indexdef
            FROM pg_indexes
            WHERE tablename = 'articles';
        """)

        indexes = cursor.fetchall()
        print(f"   📊 Znaleziono {len(indexes)} indeksów:")
        for idx_name, idx_def in indexes:
            print(f"   • {idx_name}")

        cursor.close()
        return True

    except Exception as e:
        print(f"❌ Błąd diagnostyki: {e}")
        print(f"🔍 Typ błędu: {type(e).__name__}")
        import traceback
        print(f"📝 Traceback:\n{traceback.format_exc()}")
        return False

# ============================================================================
# 🛠️ NAPRAWA TABELI
# ============================================================================

def create_or_fix_table(connection):
    """Utwórz lub napraw tabelę articles"""

    print(f"\n🛠️ TWORZENIE/NAPRAWA TABELI 'articles'")
    print("=" * 50)

    try:
        cursor = connection.cursor()

        # Sprawdź czy tabela istnieje
        cursor.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'public'
                AND table_name = 'articles'
            );
        """)

        table_exists = cursor.fetchone()[0]

        if not table_exists:
            print("📝 Tworzenie nowej tabeli 'articles'...")

            create_query = """
            CREATE TABLE articles (
                id SERIAL PRIMARY KEY,
                link TEXT NOT NULL UNIQUE,
                crawled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                text TEXT,
                text_status VARCHAR(50) DEFAULT 'pending',
                embedding TEXT,
                title_embedding TEXT,
                embedding_status VARCHAR(50),
                candidates TEXT DEFAULT '[]',
                source VARCHAR(50) DEFAULT 'unknown',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """

            cursor.execute(create_query)
            connection.commit()
            print("✅ Tabela 'articles' utworzona")

        else:
            print("🔧 Dodawanie brakujących kolumn...")

            # Lista kolumn do dodania (jeśli nie istnieją)
            columns_to_add = [
                ("embedding", "TEXT"),
                ("title_embedding", "TEXT"),
                ("embedding_status", "VARCHAR(50)"),
                ("candidates", "TEXT DEFAULT '[]'"),
                ("source", "VARCHAR(50) DEFAULT 'unknown'")
            ]

            for col_name, col_def in columns_to_add:
                try:
                    alter_query = f"ALTER TABLE articles ADD COLUMN IF NOT EXISTS {col_name} {col_def};"
                    cursor.execute(alter_query)
                    connection.commit()
                    print(f"   ✅ Kolumna '{col_name}' dodana/sprawdzona")
                except Exception as e:
                    print(f"   ⚠️ Problem z kolumną '{col_name}': {e}")

        # Dodaj indeksy
        print("🗂️ Tworzenie indeksów...")

        indexes = [
            "CREATE INDEX IF NOT EXISTS idx_articles_text_status ON articles(text_status);",
            "CREATE INDEX IF NOT EXISTS idx_articles_source ON articles(source);",
            "CREATE INDEX IF NOT EXISTS idx_articles_embedding_status ON articles(embedding_status);"
        ]

        for idx_query in indexes:
            try:
                cursor.execute(idx_query)
                connection.commit()
                print(f"   ✅ Indeks utworzony")
            except Exception as e:
                print(f"   ⚠️ Problem z indeksem: {e}")

        cursor.close()
        print("✅ Struktura tabeli gotowa!")
        return True

    except Exception as e:
        print(f"❌ Błąd tworzenia/naprawy tabeli: {e}")
        connection.rollback()
        return False

# ============================================================================
# 🧪 TEST FUNKCJONALNOŚCI
# ============================================================================

def test_table_functionality(connection):
    """Przetestuj podstawowe operacje na tabeli"""

    print(f"\n🧪 TEST FUNKCJONALNOŚCI TABELI")
    print("=" * 40)

    try:
        cursor = connection.cursor()

        # Test 1: Insert
        print("1. Test INSERT...")
        test_url = "https://test-diagnostyka.example.com"

        cursor.execute("""
            INSERT INTO articles (link, text, text_status, source)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (link) DO NOTHING
            RETURNING id;
        """, (test_url, '{"test": "data"}', 'test', 'diagnostic'))

        result = cursor.fetchone()
        if result:
            test_id = result[0]
            print(f"   ✅ INSERT OK - ID: {test_id}")
        else:
            print(f"   ⚠️ INSERT - rekord już istniał")
            cursor.execute("SELECT id FROM articles WHERE link = %s", (test_url,))
            test_id = cursor.fetchone()[0]

        # Test 2: Update
        print("2. Test UPDATE...")
        cursor.execute("""
            UPDATE articles
            SET text_status = 'diagnostic_test', embedding_status = 'test'
            WHERE link = %s
        """, (test_url,))

        print(f"   ✅ UPDATE OK - zmieniono {cursor.rowcount} wierszy")

        # Test 3: Select
        print("3. Test SELECT...")
        cursor.execute("""
            SELECT id, link, text_status, embedding_status
            FROM articles
            WHERE link = %s
        """, (test_url,))

        row = cursor.fetchone()
        if row:
            print(f"   ✅ SELECT OK - znaleziono rekord")
            print(f"      ID: {row[0]}, Status: {row[2]}, Embedding: {row[3]}")

        # Test 4: Delete test record
        print("4. Czyszczenie test...")
        cursor.execute("DELETE FROM articles WHERE link = %s", (test_url,))
        print(f"   ✅ DELETE OK - usunięto {cursor.rowcount} wierszy")

        connection.commit()
        cursor.close()

        print("🎉 Wszystkie testy przeszły pomyślnie!")
        return True

    except Exception as e:
        print(f"❌ Błąd testu: {e}")
        connection.rollback()
        return False

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja diagnostyczna"""

    print("🔬 DIAGNOSTYKA BAZY POSTGRESQL")
    print("=" * 50)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd połączenia z PostgreSQL: {e}")
        return

    try:
        # 1. Diagnostyka
        print("\n" + "="*60)
        print("ETAP 1: DIAGNOSTYKA")
        print("="*60)

        diagnosis_ok = diagnose_database(connection)

        # 2. Naprawa (jeśli potrzebna)
        if not diagnosis_ok:
            print("\n" + "="*60)
            print("ETAP 2: NAPRAWA")
            print("="*60)

            fix_ok = create_or_fix_table(connection)

            if fix_ok:
                print("\n🔄 Ponowna diagnostyka po naprawie...")
                diagnosis_ok = diagnose_database(connection)

        # 3. Test funkcjonalności
        if diagnosis_ok:
            print("\n" + "="*60)
            print("ETAP 3: TEST FUNKCJONALNOŚCI")
            print("="*60)

            test_ok = test_table_functionality(connection)

            if test_ok:
                print(f"\n🎉 BAZA GOTOWA DO UŻYCIA!")
                print(f"✅ Możesz uruchomić crawling")
            else:
                print(f"\n⚠️ PROBLEMY Z FUNKCJONALNOŚCIĄ")

        else:
            print(f"\n❌ BAZA WYMAGA RĘCZNEJ NAPRAWY")
            print(f"💡 Sprawdź błędy powyżej")

    finally:
        connection.close()
        print("\n🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🔬 DIAGNOSTYKA BAZY POSTGRESQL
✅ PostgreSQL połączony

ETAP 1: DIAGNOSTYKA
🔍 SZCZEGÓŁOWA DIAGNOSTYKA BAZY

📊 1. INFORMACJE O BAZIE:
----------------------------------------
   🗄️ PostgreSQL: PostgreSQL 15.13 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit
   📂 Baza danych: dify
   👤 Użytkownik: postgres

📋 2. SPRAWDZANIE TABELI 'articles':
----------------------------------------
   📊 Tabela 'articles' istnieje: True

🏗️ 3. STRUKTURA TABELI 'articles':
----------------------------------------
   📊 Znaleziono 11 kolumn:
   • id                   integer         NOT NULL  
   • link                 text            NOT NULL  
   • crawled_at           timestamp without time zone NULL      
   • text                 text            NULL      
   • text_status          character varying NULL      
   • embedding            text            NULL      
   • title_embedding      text            NULL      
   • embedding_status     character varying NULL      
   • candida

# 4.GENEROWANIE EMBEDINGÓW OPENAI

In [10]:
#!/usr/bin/env python3
"""
🔤 SKRYPT 1: GENEROWANIE EMBEDDINGÓW OPENAI
Dla bazy PostgreSQL z tabelą 'articles'
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
import time
import logging
from datetime import datetime, timedelta
from google.colab import userdata
from tqdm import tqdm
import tiktoken
from collections import deque
from typing import Deque
import nest_asyncio

# Zastosuj nest_asyncio dla Colab
nest_asyncio.apply()

# Instalacja wymaganych bibliotek
try:
    from llama_index.embeddings.openai import OpenAIEmbedding
    print("✅ llama_index już zainstalowany")
except ImportError:
    print("📦 Instaluję llama_index...")
    os.system('pip install llama-index-embeddings-openai')
    from llama_index.embeddings.openai import OpenAIEmbedding

# ============================================================================
# 📝 KONFIGURACJA LOGOWANIA
# ============================================================================

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)
logger = logging.getLogger(__name__)

# ============================================================================
# 🔑 KONFIGURACJA ŚRODOWISKA
# ============================================================================

def setup_environment():
    """Ustaw zmienne środowiskowe z ukrytych sekretów"""
    try:
        os.environ['POSTGRESQL_HOST'] = userdata.get('POSTGRESQL_HOST')
        os.environ['POSTGRESQL_PORT'] = '5432'
        os.environ['POSTGRESQL_USER'] = userdata.get('POSTGRESQL_USER')
        os.environ['POSTGRESQL_PASSWORD'] = userdata.get('POSTGRESQL_PASSWORD')
        os.environ['POSTGRESQL_DB'] = userdata.get('POSTGRESQL_DB')
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

        print("✅ Zmienne środowiskowe ustawione")

        # Sprawdź czy klucze są ustawione
        if not os.environ.get('OPENAI_API_KEY'):
            raise ValueError("Brak OPENAI_API_KEY!")

        return True

    except Exception as e:
        print(f"❌ Błąd konfiguracji: {e}")
        return False

# ============================================================================
# ⏱️ RATE LIMITER DLA OPENAI API
# ============================================================================

class RateLimiter:
    """Rate limiter dla OpenAI API - 3000 RPM"""

    def __init__(self, rpm_limit: int):
        self.rpm_limit = rpm_limit
        self.requests: Deque[float] = deque()
        self.window_size = 60  # 1 minuta w sekundach

    def wait_if_needed(self):
        """Czekaj jeśli zbliżamy się do limitu"""
        now = time.time()

        # Usuń stare requesty spoza okna czasowego
        while self.requests and now - self.requests[0] > self.window_size:
            self.requests.popleft()

        # Jeśli osiągnęliśmy limit, czekaj
        if len(self.requests) >= self.rpm_limit:
            sleep_time = self.requests[0] + self.window_size - now
            if sleep_time > 0:
                logger.info(f"⏳ Rate limit - czekam {sleep_time:.2f}s")
                time.sleep(sleep_time)

        # Dodaj obecny request
        self.requests.append(now)

# ============================================================================
# 🧠 PROCESOR EMBEDDINGÓW
# ============================================================================

class EmbeddingProcessor:
    """Główna klasa do generowania embeddingów"""

    def __init__(self):
        self.tokenizer = tiktoken.encoding_for_model("text-embedding-3-small")
        self.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
        self.rate_limiter = RateLimiter(rpm_limit=2500)  # Bezpieczny limit 2500 RPM

        print("✅ EmbeddingProcessor zainicjalizowany")
        print(f"   📊 Model: text-embedding-3-small")
        print(f"   ⏱️ Rate limit: 2500 RPM")

    def get_db_connection(self):
        """Połączenie z PostgreSQL"""
        try:
            connection = psycopg2.connect(
                host=os.environ['POSTGRESQL_HOST'],
                port=os.environ['POSTGRESQL_PORT'],
                user=os.environ['POSTGRESQL_USER'],
                password=os.environ['POSTGRESQL_PASSWORD'],
                database=os.environ['POSTGRESQL_DB']
            )
            logger.info("✅ Połączono z PostgreSQL")
            return connection
        except Exception as err:
            logger.error(f"❌ Błąd połączenia z PostgreSQL: {err}")
            return None

    def fetch_pending_articles(self, connection, batch_size=50):
        """Pobierz artykuły do przetworzenia"""
        try:
            cursor = connection.cursor(cursor_factory=RealDictCursor)
            cursor.execute("""
                SELECT id, text, link FROM articles
                WHERE text_status = 'completed'
                AND (embedding IS NULL OR embedding_status IS NULL)
                ORDER BY created_at ASC
                LIMIT %s
            """, (batch_size,))

            rows = cursor.fetchall()
            cursor.close()

            logger.info(f"📥 Pobrano {len(rows)} artykułów do przetworzenia")
            return rows

        except Exception as err:
            logger.error(f"❌ Błąd pobierania danych: {err}")
            return []

    def truncate_text_to_token_limit(self, text: str, token_limit: int = 8000) -> str:
        """Skróć tekst do limitu tokenów"""
        tokens = self.tokenizer.encode(text)
        if len(tokens) > token_limit:
            truncated_tokens = tokens[:token_limit]
            return self.tokenizer.decode(truncated_tokens)
        return text

    def get_text_embedding(self, text: str):
        """Wygeneruj embedding dla tekstu"""
        try:
            # Zastosuj rate limiting
            self.rate_limiter.wait_if_needed()

            # Wygeneruj embedding
            embedding = self.embed_model.get_text_embedding(text)
            return embedding

        except Exception as e:
            logger.error(f"❌ Błąd generowania embeddingu: {e}")
            return None

    def update_embeddings_in_db(self, connection, article_id: int, content_embedding, title_embedding, status: str):
        """Zapisz embeddingi do bazy"""
        try:
            cursor = connection.cursor()

            # Konwertuj embeddingi do JSON
            content_embedding_json = json.dumps(content_embedding) if content_embedding else None
            title_embedding_json = json.dumps(title_embedding) if title_embedding else None

            cursor.execute("""
                UPDATE articles
                SET embedding = %s, title_embedding = %s, embedding_status = %s
                WHERE id = %s
            """, (content_embedding_json, title_embedding_json, status, article_id))

            connection.commit()
            cursor.close()

            logger.info(f"✅ Zaktualizowano embeddingi dla artykułu ID: {article_id}")

        except Exception as err:
            logger.error(f"❌ Błąd aktualizacji bazy dla ID {article_id}: {err}")
            connection.rollback()

    def process_single_article(self, article: dict, connection):
        """Przetwórz pojedynczy artykuł"""
        article_id = article['id']
        text_data = article['text']
        link = article['link']

        if not text_data:
            logger.warning(f"⚠️ Brak tekstu dla artykułu ID: {article_id}")
            self.update_embeddings_in_db(connection, article_id, None, None, 'error_no_text')
            return False

        try:
            # Parse JSON z tekstem
            json_data = json.loads(text_data)

            if not json_data or 'data' not in json_data:
                logger.warning(f"⚠️ Nieprawidłowy JSON dla ID: {article_id}")
                self.update_embeddings_in_db(connection, article_id, None, None, 'error_invalid_json')
                return False

            content = json_data['data'].get('content', '')
            title = json_data['data'].get('title', '')

            if not content or not title:
                logger.warning(f"⚠️ Brak treści lub tytułu dla ID: {article_id}")
                self.update_embeddings_in_db(connection, article_id, None, None, 'error_missing_content')
                return False

            # Skróć do limitu tokenów
            truncated_content = self.truncate_text_to_token_limit(content, 8000)
            truncated_title = self.truncate_text_to_token_limit(title, 500)

            logger.info(f"🔤 Generuję embeddingi dla: {title[:50]}...")

            # Wygeneruj embeddingi
            content_embedding = self.get_text_embedding(truncated_content)
            title_embedding = self.get_text_embedding(truncated_title)

            if content_embedding and title_embedding:
                self.update_embeddings_in_db(connection, article_id, content_embedding, title_embedding, 'completed')
                return True
            else:
                self.update_embeddings_in_db(connection, article_id, None, None, 'error_embedding_failed')
                return False

        except json.JSONDecodeError as e:
            logger.error(f"❌ Błąd JSON dla ID {article_id}: {e}")
            self.update_embeddings_in_db(connection, article_id, None, None, 'error_json_decode')
            return False

        except Exception as e:
            logger.error(f"❌ Nieoczekiwany błąd dla ID {article_id}: {e}")
            self.update_embeddings_in_db(connection, article_id, None, None, 'error_unexpected')
            return False

    def process_batch(self, connection, batch_size=50):
        """Przetwórz batch artykułów"""
        articles = self.fetch_pending_articles(connection, batch_size)

        if not articles:
            logger.info("🤷 Brak artykułów do przetworzenia")
            return 0, 0

        success_count = 0
        error_count = 0

        logger.info(f"🚀 Przetwarzam batch {len(articles)} artykułów...")

        # Progress bar
        pbar = tqdm(articles, desc="Generowanie embeddingów", ncols=80)

        for article in pbar:
            # Aktualizuj opis progress bara
            title = ""
            try:
                if article['text']:
                    json_data = json.loads(article['text'])
                    title = json_data.get('data', {}).get('title', '')[:30] + '...'
            except:
                title = f"ID: {article['id']}"

            pbar.set_postfix_str(title)

            # Przetwórz artykuł
            if self.process_single_article(article, connection):
                success_count += 1
            else:
                error_count += 1

            # Krótka pauza między artykułami
            time.sleep(0.1)

        pbar.close()

        logger.info(f"✅ Batch zakończony: {success_count} sukces, {error_count} błędów")
        return success_count, error_count

    def run_full_processing(self, batch_size=50, max_batches=None):
        """Uruchom pełne przetwarzanie embeddingów"""
        connection = self.get_db_connection()
        if not connection:
            return

        total_success = 0
        total_errors = 0
        batch_count = 0
        start_time = datetime.now()

        try:
            logger.info(f"🔤 ROZPOCZYNAM GENEROWANIE EMBEDDINGÓW")
            logger.info(f"📦 Rozmiar batcha: {batch_size}")

            while True:
                batch_count += 1

                logger.info(f"\n🚀 BATCH #{batch_count}")
                logger.info("-" * 40)

                success, errors = self.process_batch(connection, batch_size)

                if success == 0 and errors == 0:
                    logger.info("✅ Wszystkie artykuły przetworzone!")
                    break

                total_success += success
                total_errors += errors

                # Statystyki
                elapsed = datetime.now() - start_time
                rate = total_success / (elapsed.total_seconds() / 60) if elapsed.total_seconds() > 0 else 0

                logger.info(f"📊 Statystyki po batch #{batch_count}:")
                logger.info(f"   ✅ Łączne sukcesy: {total_success}")
                logger.info(f"   ❌ Łączne błędy: {total_errors}")
                logger.info(f"   ⚡ Prędkość: {rate:.1f} artykułów/min")
                logger.info(f"   ⏱️ Czas trwania: {elapsed}")

                # Sprawdź limit batchów
                if max_batches and batch_count >= max_batches:
                    logger.info(f"🔚 Osiągnięto limit {max_batches} batchów")
                    break

                # Pauza między batchami
                logger.info("😴 Pauza 5s między batchami...")
                time.sleep(5)

        finally:
            connection.close()
            logger.info("🔒 Połączenie zamknięte")

        # Podsumowanie końcowe
        duration = datetime.now() - start_time
        total_processed = total_success + total_errors

        logger.info(f"\n🎉 PRZETWARZANIE ZAKOŃCZONE!")
        logger.info("=" * 50)
        logger.info(f"✅ Sukcesy: {total_success}")
        logger.info(f"❌ Błędy: {total_errors}")
        logger.info(f"📊 Łącznie: {total_processed}")
        logger.info(f"⏱️ Czas trwania: {duration}")

        if total_processed > 0:
            success_rate = (total_success / total_processed) * 100
            rate_per_min = total_processed / (duration.total_seconds() / 60)
            logger.info(f"📈 Skuteczność: {success_rate:.1f}%")
            logger.info(f"⚡ Prędkość: {rate_per_min:.1f} artykułów/min")

# ============================================================================
# 📊 FUNKCJE POMOCNICZE
# ============================================================================

def check_embedding_status(connection):
    """Sprawdź status embeddingów w bazie"""
    try:
        cursor = connection.cursor()

        # Status breakdown
        cursor.execute("""
            SELECT
                embedding_status,
                COUNT(*) as count
            FROM articles
            WHERE text_status = 'completed'
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        results = cursor.fetchall()
        cursor.close()

        print(f"\n📊 STATUS EMBEDDINGÓW:")
        print("-" * 40)

        total = 0
        for status, count in results:
            status_display = status if status else 'NULL (pending)'
            print(f"   {status_display:<20} {count:>6}")
            total += count

        print("-" * 40)
        print(f"   {'ŁĄCZNIE':<20} {total:>6}")

        return results

    except Exception as e:
        print(f"❌ Błąd sprawdzania statusu: {e}")
        return []

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja programu"""

    print("🔤 GENERATOR EMBEDDINGÓW OPENAI")
    print("=" * 50)

    # Konfiguracja środowiska
    if not setup_environment():
        print("❌ Błąd konfiguracji - sprawdź sekrety!")
        return

    # Stwórz procesor
    try:
        processor = EmbeddingProcessor()
    except Exception as e:
        print(f"❌ Błąd inicjalizacji: {e}")
        return

    # Sprawdź status przed rozpoczęciem
    connection = processor.get_db_connection()
    if connection:
        check_embedding_status(connection)
        connection.close()

    # Menu opcji
    print(f"\n🎯 OPCJE:")
    print("1. Test na 1 batchu (50 artykułów)")
    print("2. Pełne przetwarzanie wszystkich artykułów")
    print("3. Przetworzenie określonej liczby batchów")
    print("4. Tylko sprawdzenie statusu")

    choice = input("\nWybierz opcję (1-4): ").strip()

    if choice == '1':
        # Test na 1 batchu
        print(f"\n🧪 TEST NA 1 BATCHU")
        processor.run_full_processing(batch_size=50, max_batches=1)

    elif choice == '2':
        # Pełne przetwarzanie
        print(f"\n🚀 PEŁNE PRZETWARZANIE")
        processor.run_full_processing(batch_size=50)

    elif choice == '3':
        # Określona liczba batchów
        try:
            max_batches = int(input("Ile batchów? "))
            print(f"\n🎯 PRZETWARZANIE {max_batches} BATCHÓW")
            processor.run_full_processing(batch_size=50, max_batches=max_batches)
        except ValueError:
            print("❌ Nieprawidłowa liczba!")

    elif choice == '4':
        # Tylko status
        connection = processor.get_db_connection()
        if connection:
            check_embedding_status(connection)
            connection.close()

    else:
        print("❌ Nieprawidłowy wybór!")

if __name__ == "__main__":
    main()

✅ llama_index już zainstalowany
🔤 GENERATOR EMBEDDINGÓW OPENAI
✅ Zmienne środowiskowe ustawione
✅ EmbeddingProcessor zainicjalizowany
   📊 Model: text-embedding-3-small
   ⏱️ Rate limit: 2500 RPM


2025-06-03 08:49:53,290 - INFO - ✅ Połączono z PostgreSQL



📊 STATUS EMBEDDINGÓW:
----------------------------------------
   completed               172
   NULL (pending)            2
----------------------------------------
   ŁĄCZNIE                 174

🎯 OPCJE:
1. Test na 1 batchu (50 artykułów)
2. Pełne przetwarzanie wszystkich artykułów
3. Przetworzenie określonej liczby batchów
4. Tylko sprawdzenie statusu

Wybierz opcję (1-4): 2

🚀 PEŁNE PRZETWARZANIE


2025-06-03 08:49:59,815 - INFO - ✅ Połączono z PostgreSQL
2025-06-03 08:49:59,816 - INFO - 🔤 ROZPOCZYNAM GENEROWANIE EMBEDDINGÓW
2025-06-03 08:49:59,816 - INFO - 📦 Rozmiar batcha: 50
2025-06-03 08:49:59,817 - INFO - 
🚀 BATCH #1
2025-06-03 08:49:59,817 - INFO - ----------------------------------------
2025-06-03 08:50:00,532 - INFO - 📥 Pobrano 2 artykułów do przetworzenia
2025-06-03 08:50:00,534 - INFO - 🚀 Przetwarzam batch 2 artykułów...
Generowanie embeddingów:   0%| | 0/2 [00:00<?, ?it/s, It helps determine if the 2025-06-03 08:50:00,540 - INFO - 🔤 Generuję embeddingi dla: It helps determine if the user's browser can displ...
2025-06-03 08:50:01,339 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-03 08:50:01,769 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-03 08:50:02,732 - INFO - ✅ Zaktualizowano embeddingi dla artykułu ID: 85
Generowanie embeddingów:  50%|▌| 1/2 [00:02<00:02,  2.30s/it, It help

## 4.1 Sprawdzanie danych w przypadku błędów




In [11]:
#!/usr/bin/env python3
"""
🔍 DEBUG: Sprawdzenie struktury artykułów w bazie
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
from google.colab import userdata

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔍 FUNKCJE DEBUGOWANIA
# ============================================================================

def debug_article_structure(connection):
    """Sprawdź strukturę przykładowych artykułów"""

    print(f"🔍 DEBUGOWANIE STRUKTURY ARTYKUŁÓW")
    print("=" * 60)

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        # Pobierz problematyczne artykuły
        cursor.execute("""
            SELECT id, link, text, text_status, embedding_status
            FROM articles
            WHERE text_status = 'completed'
            AND (embedding_status IS NULL OR embedding_status LIKE 'error%')
            ORDER BY id
            LIMIT 5
        """)

        problem_articles = cursor.fetchall()

        print(f"📊 Znaleziono {len(problem_articles)} problematycznych artykułów")
        print()

        for i, article in enumerate(problem_articles, 1):
            print(f"🔍 ARTYKUŁ #{i} (ID: {article['id']})")
            print("-" * 50)
            print(f"🔗 URL: {article['link']}")
            print(f"📊 Text Status: {article['text_status']}")
            print(f"🔤 Embedding Status: {article['embedding_status']}")

            # Sprawdź strukturę tekstu
            text_data = article['text']
            print(f"📝 Text Length: {len(text_data) if text_data else 0}")

            if text_data:
                print(f"📄 Raw text preview:")
                print(f"   {text_data[:200]}...")

                try:
                    # Spróbuj sparsować JSON
                    json_data = json.loads(text_data)
                    print(f"✅ JSON válido")

                    # Sprawdź strukturę
                    print(f"🏗️ JSON Structure:")
                    print(f"   Keys: {list(json_data.keys())}")

                    if 'data' in json_data:
                        data_section = json_data['data']
                        print(f"   Data keys: {list(data_section.keys()) if isinstance(data_section, dict) else 'Not a dict'}")

                        if isinstance(data_section, dict):
                            title = data_section.get('title', '')
                            content = data_section.get('content', '')
                            url = data_section.get('url', '')

                            print(f"   📰 Title: '{title[:50]}...' (length: {len(title)})")
                            print(f"   📄 Content: '{content[:50]}...' (length: {len(content)})")
                            print(f"   🔗 URL: '{url}'")

                            # Sprawdź czy są puste
                            if not title or not content:
                                print(f"   ⚠️ PROBLEM: Pusty tytuł lub treść!")

                                if not title:
                                    print(f"      ❌ Brak tytułu")
                                if not content:
                                    print(f"      ❌ Brak treści")
                        else:
                            print(f"   ❌ 'data' nie jest obiektem!")
                    else:
                        print(f"   ❌ Brak klucza 'data'!")

                except json.JSONDecodeError as e:
                    print(f"❌ JSON Error: {e}")
                    print(f"   Raw content: {text_data}")

            else:
                print(f"❌ Brak tekstu!")

            print()

        # Sprawdź artykuły, które działają
        print(f"\n✅ SPRAWDZANIE DZIAŁAJĄCYCH ARTYKUŁÓW")
        print("=" * 50)

        cursor.execute("""
            SELECT id, link, text, embedding_status
            FROM articles
            WHERE text_status = 'completed'
            AND embedding_status = 'completed'
            ORDER BY id
            LIMIT 3
        """)

        working_articles = cursor.fetchall()

        for i, article in enumerate(working_articles, 1):
            print(f"✅ WORKING ARTYKUŁ #{i} (ID: {article['id']})")

            text_data = article['text']
            if text_data:
                try:
                    json_data = json.loads(text_data)
                    if 'data' in json_data:
                        data_section = json_data['data']
                        title = data_section.get('title', '')
                        content = data_section.get('content', '')

                        print(f"   📰 Title: '{title[:30]}...' (length: {len(title)})")
                        print(f"   📄 Content: '{content[:30]}...' (length: {len(content)})")

                except Exception as e:
                    print(f"   ❌ Error: {e}")
            print()

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd debugowania: {e}")

def check_embedding_statistics(connection):
    """Sprawdź statystyki embeddingów"""

    print(f"📊 STATYSTYKI EMBEDDINGÓW")
    print("=" * 40)

    try:
        cursor = connection.cursor()

        # Status breakdown
        cursor.execute("""
            SELECT
                embedding_status,
                COUNT(*) as count
            FROM articles
            WHERE text_status = 'completed'
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        results = cursor.fetchall()

        total = 0
        for status, count in results:
            status_display = status if status else 'NULL (pending)'
            print(f"   {status_display:<25} {count:>6}")
            total += count

        print("-" * 40)
        print(f"   {'ŁĄCZNIE':<25} {total:>6}")

        # Breakdown błędów
        print(f"\n🚨 SZCZEGÓŁY BŁĘDÓW:")
        cursor.execute("""
            SELECT
                embedding_status,
                COUNT(*) as count
            FROM articles
            WHERE text_status = 'completed'
            AND embedding_status LIKE 'error%'
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        error_results = cursor.fetchall()

        for status, count in error_results:
            print(f"   ❌ {status:<25} {count:>6}")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd statystyk: {e}")

def sample_raw_content(connection):
    """Pokaż surową treść kilku artykułów"""

    print(f"\n📄 PRZYKŁADY SUROWEJ TREŚCI")
    print("=" * 50)

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        cursor.execute("""
            SELECT id, link, text
            FROM articles
            WHERE text_status = 'completed'
            ORDER BY RANDOM()
            LIMIT 3
        """)

        samples = cursor.fetchall()

        for i, article in enumerate(samples, 1):
            print(f"\n📄 SAMPLE #{i} (ID: {article['id']})")
            print(f"🔗 {article['link'][:60]}...")
            print("📝 Raw text:")

            if article['text']:
                # Pokaż pierwsze 500 znaków
                raw_text = article['text'][:500]
                print(f"```")
                print(raw_text)
                print(f"```")

                # Sprawdź czy to valid JSON
                try:
                    json.loads(article['text'])
                    print("✅ Valid JSON")
                except:
                    print("❌ Invalid JSON")
            else:
                print("❌ NULL text")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja debugowania"""

    print("🔍 DEBUG ARTYKUŁÓW W BAZIE")
    print("=" * 50)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # 1. Sprawdź statystyki
        check_embedding_statistics(connection)

        # 2. Debug problematycznych artykułów
        debug_article_structure(connection)

        # 3. Pokaż przykłady surowej treści
        sample_raw_content(connection)

        print(f"\n🔧 MOŻLIWE PRZYCZYNY PROBLEMÓW:")
        print("1. Pusty title/content w JSON")
        print("2. Nieprawidłowa struktura JSON")
        print("3. Problemy z crawlingiem niektórych URL-i")
        print("4. Encoding issues")

        print(f"\n💡 SUGEROWANE ROZWIĄZANIA:")
        print("1. Sprawdź jakość crawlingu")
        print("2. Dodaj fallback dla pustych pól")
        print("3. Improved error handling")
        print("4. Re-crawl problematycznych URL-i")

    finally:
        connection.close()
        print("\n🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🔍 DEBUG ARTYKUŁÓW W BAZIE
✅ PostgreSQL połączony
📊 STATYSTYKI EMBEDDINGÓW
   completed                    174
----------------------------------------
   ŁĄCZNIE                      174

🚨 SZCZEGÓŁY BŁĘDÓW:
🔍 DEBUGOWANIE STRUKTURY ARTYKUŁÓW
📊 Znaleziono 0 problematycznych artykułów


✅ SPRAWDZANIE DZIAŁAJĄCYCH ARTYKUŁÓW
✅ WORKING ARTYKUŁ #1 (ID: 1)
   📰 Title: 'Catering dietetyczny - dieta p...' (length: 38)
   📄 Content: 'Cenimy prywatność użytkowników...' (length: 27208)

✅ WORKING ARTYKUŁ #2 (ID: 2)
   📰 Title: 'Zespół Catering Food Harmony...' (length: 28)
   📄 Content: 'Zespół Catering Food Harmony

...' (length: 25141)

✅ WORKING ARTYKUŁ #3 (ID: 3)
   📰 Title: 'Kontakt...' (length: 7)
   📄 Content: 'Używamy plików cookie, aby pom...' (length: 11192)


📄 PRZYKŁADY SUROWEJ TREŚCI

📄 SAMPLE #1 (ID: 222)
🔗 https://cateringfoodharmony.pl/wymiana-posilkow-w-cateringu-...
📝 Raw text:
```
{"data": {"title": "It helps determine if the user's browser can display emojis properly", "content

## 4.2 Naprawa pustych tytułów

In [9]:
#!/usr/bin/env python3
"""
🛠️ NAPRAWA PUSTYCH TYTUŁÓW
Generuje fallback tytuły dla artykułów z pustymi tytułami
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
import re
from google.colab import userdata

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🛠️ FUNKCJE NAPRAWCZE
# ============================================================================

def generate_title_from_url(url):
    """Wygeneruj tytuł z URL"""
    try:
        # Wyciągnij ostatnią część URL (slug)
        slug = url.split('/')[-1]

        # Usuń rozszerzenia i parametry
        slug = slug.split('?')[0].split('#')[0]

        # Zamień myślniki i podkreślenia na spacje
        title = slug.replace('-', ' ').replace('_', ' ')

        # Usuń cyfry na końcu (timestampy)
        title = re.sub(r'-?\d{10,}$', '', title)

        # Capitalize words
        title = ' '.join(word.capitalize() for word in title.split())

        # Dodaj prefix jeśli jest zbyt krótki
        if len(title) < 10:
            title = f"Artykuł - {title}"

        return title[:100]  # Max 100 znaków

    except Exception as e:
        print(f"❌ Błąd generowania tytułu z URL: {e}")
        return "Artykuł"

def generate_title_from_content(content):
    """Wygeneruj tytuł z pierwszych słów treści"""
    try:
        # Usuń cookie notices i inne standardowe teksty
        content = re.sub(r'Używamy plików cookie.*?podstawowych funkcji witryny\.', '', content, flags=re.DOTALL)
        content = re.sub(r'Cenimy prywatność użytkowników.*?plików cookie\.', '', content, flags=re.DOTALL)

        # Podziel na zdania
        sentences = re.split(r'[.!?]+', content)

        # Znajdź pierwsze znaczące zdanie
        for sentence in sentences:
            sentence = sentence.strip()

            # Pomiń bardzo krótkie fragmenty
            if len(sentence) < 20:
                continue

            # Pomiń fragmenty z cookie/privacy
            if any(word in sentence.lower() for word in ['cookie', 'prywatność', 'zgoda', 'reklam']):
                continue

            # Skróć do rozsądnej długości tytułu
            if len(sentence) > 60:
                # Spróbuj skrócić do pierwszego przecinka lub średnika
                short = sentence.split(',')[0].split(';')[0]
                if len(short) > 20:
                    sentence = short
                else:
                    sentence = sentence[:60] + '...'

            return sentence.strip()

        # Fallback - pierwsze 50 znaków
        clean_content = re.sub(r'\s+', ' ', content[:200])
        return clean_content[:50] + '...' if len(clean_content) > 50 else clean_content

    except Exception as e:
        print(f"❌ Błąd generowania tytułu z treści: {e}")
        return "Artykuł bez tytułu"

def fix_empty_titles(connection):
    """Napraw artykuły z pustymi tytułami"""

    print(f"🛠️ NAPRAWA PUSTYCH TYTUŁÓW")
    print("=" * 50)

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        # Znajdź artykuły z pustymi tytułami
        cursor.execute("""
            SELECT id, link, text, embedding_status
            FROM articles
            WHERE text_status = 'completed'
            AND (embedding_status = 'error_missing_content'
                 OR embedding_status IS NULL)
        """)

        articles_to_fix = cursor.fetchall()

        print(f"🔍 Znaleziono {len(articles_to_fix)} artykułów do naprawy")

        fixed_count = 0

        for article in articles_to_fix:
            article_id = article['id']
            url = article['link']
            text_data = article['text']

            print(f"\n🔧 Naprawiam artykuł ID: {article_id}")
            print(f"   🔗 URL: {url}")

            if not text_data:
                print(f"   ❌ Brak danych tekstowych - pomijam")
                continue

            try:
                # Parse JSON
                json_data = json.loads(text_data)

                if 'data' not in json_data:
                    print(f"   ❌ Brak sekcji 'data' - pomijam")
                    continue

                data_section = json_data['data']
                current_title = data_section.get('title', '')
                content = data_section.get('content', '')

                print(f"   📰 Obecny tytuł: '{current_title}' (długość: {len(current_title)})")

                # Sprawdź czy rzeczywiście pusty tytuł
                if current_title and len(current_title.strip()) > 0:
                    print(f"   ✅ Tytuł już istnieje - pomijam")
                    continue

                # Wygeneruj nowy tytuł
                new_title = None

                # Metoda 1: Z treści (preferowana)
                if content and len(content) > 50:
                    new_title = generate_title_from_content(content)
                    print(f"   🎯 Tytuł z treści: '{new_title}'")

                # Metoda 2: Z URL (fallback)
                if not new_title or len(new_title) < 10:
                    new_title = generate_title_from_url(url)
                    print(f"   🎯 Tytuł z URL: '{new_title}'")

                # Aktualizuj JSON
                data_section['title'] = new_title
                updated_text = json.dumps(json_data, ensure_ascii=False)

                # Zapisz do bazy
                update_cursor = connection.cursor()
                update_cursor.execute("""
                    UPDATE articles
                    SET text = %s, embedding_status = NULL
                    WHERE id = %s
                """, (updated_text, article_id))

                connection.commit()
                update_cursor.close()

                print(f"   ✅ Zaktualizowano tytuł: '{new_title}'")
                fixed_count += 1

            except json.JSONDecodeError as e:
                print(f"   ❌ Błąd JSON: {e}")
                continue
            except Exception as e:
                print(f"   ❌ Błąd: {e}")
                continue

        cursor.close()

        print(f"\n📊 PODSUMOWANIE NAPRAWY:")
        print(f"   🔧 Naprawiono: {fixed_count} artykułów")
        print(f"   📊 Sprawdzono: {len(articles_to_fix)} artykułów")

        return fixed_count

    except Exception as e:
        print(f"❌ Błąd naprawy: {e}")
        return 0

def verify_fixes(connection):
    """Sprawdź czy naprawy zadziałały"""

    print(f"\n✅ WERYFIKACJA NAPRAW")
    print("=" * 30)

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        # Sprawdź czy nadal są puste tytuły
        cursor.execute("""
            SELECT id, link, text
            FROM articles
            WHERE text_status = 'completed'
            AND embedding_status IS NULL
            LIMIT 5
        """)

        pending_articles = cursor.fetchall()

        print(f"🔍 Artykuły gotowe do embeddingów: {len(pending_articles)}")

        for article in pending_articles:
            try:
                json_data = json.loads(article['text'])
                title = json_data['data'].get('title', '')
                print(f"   ✅ ID {article['id']}: '{title[:40]}...'")
            except:
                print(f"   ❌ ID {article['id']}: Błąd parsowania")

        # Sprawdź czy nadal są błędy
        cursor.execute("""
            SELECT embedding_status, COUNT(*) as count
            FROM articles
            WHERE text_status = 'completed'
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        status_results = cursor.fetchall()

        print(f"\n📊 Nowe statystyki embeddingów:")
        for status, count in status_results:
            status_display = status if status else 'NULL (gotowe do przetworzenia)'
            print(f"   {status_display:<30} {count:>3}")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd weryfikacji: {e}")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja naprawy"""

    print("🛠️ NAPRAWA PUSTYCH TYTUŁÓW")
    print("=" * 40)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # Napraw puste tytuły
        fixed_count = fix_empty_titles(connection)

        if fixed_count > 0:
            # Weryfikuj naprawy
            verify_fixes(connection)

            print(f"\n🎉 NAPRAWA ZAKOŃCZONA!")
            print(f"✅ Naprawiono {fixed_count} artykułów")
            print(f"🔤 Możesz teraz kontynuować generowanie embeddingów")
        else:
            print(f"\n🤷 Brak artykułów do naprawy")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

🛠️ NAPRAWA PUSTYCH TYTUŁÓW
✅ PostgreSQL połączony
🛠️ NAPRAWA PUSTYCH TYTUŁÓW
🔍 Znaleziono 2 artykułów do naprawy

🔧 Naprawiam artykuł ID: 85
   🔗 URL: https://cateringfoodharmony.pl/dieta-w-prezencie-1621254208
   📰 Obecny tytuł: '' (długość: 0)
   🎯 Tytuł z treści: 'It helps determine if the user's browser can display emojis properly'
   ✅ Zaktualizowano tytuł: 'It helps determine if the user's browser can display emojis properly'

🔧 Naprawiam artykuł ID: 222
   🔗 URL: https://cateringfoodharmony.pl/wymiana-posilkow-w-cateringu-dietetycznym
   📰 Obecny tytuł: '' (długość: 0)
   🎯 Tytuł z treści: 'It helps determine if the user's browser can display emojis properly'
   ✅ Zaktualizowano tytuł: 'It helps determine if the user's browser can display emojis properly'

📊 PODSUMOWANIE NAPRAWY:
   🔧 Naprawiono: 2 artykułów
   📊 Sprawdzono: 2 artykułów

✅ WERYFIKACJA NAPRAW
🔍 Artykuły gotowe do embeddingów: 2
   ✅ ID 85: 'It helps determine if the user's browser...'
   ✅ ID 222: 'It helps deter

##4.3 Sprawdzenie statusu embedingów

In [12]:
#!/usr/bin/env python3
"""
✅ SPRAWDZENIE STATUSU EMBEDDINGÓW
Kompletna weryfikacja bazy po generowaniu embeddingów
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
from google.colab import userdata

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 📊 FUNKCJE SPRAWDZAJĄCE
# ============================================================================

def check_overall_status(connection):
    """Sprawdź ogólny status bazy"""

    print(f"📊 OGÓLNY STATUS BAZY DANYCH")
    print("=" * 50)

    try:
        cursor = connection.cursor()

        # Total counts
        cursor.execute("SELECT COUNT(*) FROM articles")
        total_articles = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM articles WHERE text_status = 'completed'")
        crawled_articles = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM articles WHERE embedding_status = 'completed'")
        embedded_articles = cursor.fetchone()[0]

        print(f"📄 Łączna liczba artykułów: {total_articles}")
        print(f"🕷️ Artykuły po crawlingu: {crawled_articles}")
        print(f"🔤 Artykuły z embeddingami: {embedded_articles}")

        # Percentages
        if total_articles > 0:
            crawl_percent = (crawled_articles / total_articles) * 100
            embed_percent = (embedded_articles / total_articles) * 100

            print(f"📈 % crawlingu: {crawl_percent:.1f}%")
            print(f"📈 % embeddingów: {embed_percent:.1f}%")

        cursor.close()

        return {
            'total': total_articles,
            'crawled': crawled_articles,
            'embedded': embedded_articles
        }

    except Exception as e:
        print(f"❌ Błąd sprawdzania statusu: {e}")
        return None

def check_embedding_status_breakdown(connection):
    """Szczegółowy breakdown statusów embeddingów"""

    print(f"\n🔤 BREAKDOWN STATUSÓW EMBEDDINGÓW")
    print("=" * 50)

    try:
        cursor = connection.cursor()

        cursor.execute("""
            SELECT
                embedding_status,
                COUNT(*) as count,
                ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as percentage
            FROM articles
            WHERE text_status = 'completed'
            GROUP BY embedding_status
            ORDER BY count DESC
        """)

        results = cursor.fetchall()

        print(f"{'Status':<30} {'Liczba':<10} {'Procent':<10}")
        print("-" * 55)

        for status, count, percentage in results:
            status_display = status if status else 'NULL (pending)'
            emoji = "✅" if status == "completed" else "❌" if status and "error" in status else "⏳"
            print(f"{emoji} {status_display:<27} {count:<10} {percentage}%")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd breakdown: {e}")

def check_embedding_data_quality(connection):
    """Sprawdź jakość danych embeddingów"""

    print(f"\n🔍 JAKOŚĆ DANYCH EMBEDDINGÓW")
    print("=" * 40)

    try:
        cursor = connection.cursor(cursor_factory=RealDictCursor)

        # Sprawdź przykładowe embeddingi
        cursor.execute("""
            SELECT id, link, embedding, title_embedding
            FROM articles
            WHERE embedding_status = 'completed'
            AND embedding IS NOT NULL
            AND title_embedding IS NOT NULL
            LIMIT 5
        """)

        samples = cursor.fetchall()

        print(f"🔬 Sprawdzam {len(samples)} przykładowych embeddingów:")

        for sample in samples:
            article_id = sample['id']
            link = sample['link']

            try:
                # Parse embeddingów
                content_embedding = json.loads(sample['embedding'])
                title_embedding = json.loads(sample['title_embedding'])

                print(f"\n✅ Artykuł ID: {article_id}")
                print(f"   🔗 URL: {link[:50]}...")
                print(f"   📄 Content embedding: {len(content_embedding)} wymiarów")
                print(f"   📰 Title embedding: {len(title_embedding)} wymiarów")

                # Sprawdź czy wymiary są OK
                if len(content_embedding) != 1536:
                    print(f"   ⚠️ Nieprawidłowy rozmiar content embedding!")
                if len(title_embedding) != 1536:
                    print(f"   ⚠️ Nieprawidłowy rozmiar title embedding!")

                # Sprawdź czy wartości są numeryczne
                if not all(isinstance(x, (int, float)) for x in content_embedding[:5]):
                    print(f"   ⚠️ Nieprawidłowe wartości w content embedding!")

            except json.JSONDecodeError:
                print(f"   ❌ Błąd parsowania JSON dla ID: {article_id}")
            except Exception as e:
                print(f"   ❌ Błąd: {e}")

        cursor.close()

    except Exception as e:
        print(f"❌ Błąd sprawdzania jakości: {e}")

def check_ready_for_similarity(connection):
    """Sprawdź gotowość do similarity search"""

    print(f"\n🎯 GOTOWOŚĆ DO SIMILARITY SEARCH")
    print("=" * 40)

    try:
        cursor = connection.cursor()

        # Artykuły gotowe do similarity
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE text_status = 'completed'
            AND embedding_status = 'completed'
            AND embedding IS NOT NULL
            AND title_embedding IS NOT NULL
        """)

        ready_count = cursor.fetchone()[0]

        # Sprawdź rozmiary embeddingów
        cursor.execute("""
            SELECT
                AVG(LENGTH(embedding)) as avg_embedding_size,
                AVG(LENGTH(title_embedding)) as avg_title_size
            FROM articles
            WHERE embedding_status = 'completed'
        """)

        size_stats = cursor.fetchone()

        print(f"🚀 Artykuły gotowe do similarity: {ready_count}")
        print(f"📊 Średni rozmiar embedding: {size_stats[0]:.0f} znaków")
        print(f"📊 Średni rozmiar title embedding: {size_stats[1]:.0f} znaków")

        if ready_count >= 100:
            print(f"✅ GOTOWE! Wystarczająco artykułów do similarity search")
        elif ready_count >= 50:
            print(f"⚠️ OK - można rozpocząć, ale więcej artykułów byłoby lepiej")
        else:
            print(f"❌ Za mało artykułów do sensownego similarity search")

        cursor.close()

        return ready_count

    except Exception as e:
        print(f"❌ Błąd sprawdzania gotowości: {e}")
        return 0

def check_potential_issues(connection):
    """Sprawdź potencjalne problemy"""

    print(f"\n🚨 SPRAWDZANIE POTENCJALNYCH PROBLEMÓW")
    print("=" * 45)

    try:
        cursor = connection.cursor()

        issues_found = 0

        # 1. Artykuły z błędami
        cursor.execute("""
            SELECT embedding_status, COUNT(*)
            FROM articles
            WHERE text_status = 'completed'
            AND embedding_status LIKE 'error%'
            GROUP BY embedding_status
        """)

        errors = cursor.fetchall()
        if errors:
            print(f"❌ Artykuły z błędami embeddingów:")
            for status, count in errors:
                print(f"   • {status}: {count} artykułów")
                issues_found += count

        # 2. Null embeddingi przy statusie completed
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE embedding_status = 'completed'
            AND (embedding IS NULL OR title_embedding IS NULL)
        """)

        null_embeddings = cursor.fetchone()[0]
        if null_embeddings > 0:
            print(f"❌ Artykuły ze statusem 'completed' ale NULL embeddings: {null_embeddings}")
            issues_found += null_embeddings

        # 3. Bardzo krótkie embeddingi
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE embedding_status = 'completed'
            AND (LENGTH(embedding) < 1000 OR LENGTH(title_embedding) < 1000)
        """)

        short_embeddings = cursor.fetchone()[0]
        if short_embeddings > 0:
            print(f"⚠️ Artykuły z podejrzanie krótkimi embeddingami: {short_embeddings}")

        # 4. Pending artykuły
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE text_status = 'completed'
            AND embedding_status IS NULL
        """)

        pending = cursor.fetchone()[0]
        if pending > 0:
            print(f"⏳ Artykuły oczekujące na embeddingi: {pending}")

        cursor.close()

        if issues_found == 0 and pending == 0:
            print(f"✅ Brak problemów! Wszystko wygląda świetnie!")

        return issues_found

    except Exception as e:
        print(f"❌ Błąd sprawdzania problemów: {e}")
        return -1

def show_next_steps(ready_count, issues_count):
    """Pokaż następne kroki"""

    print(f"\n🎯 NASTĘPNE KROKI")
    print("=" * 30)

    if issues_count > 0:
        print(f"🔧 1. Rozwiąż {issues_count} problemów z embeddingami")
        print(f"🔄 2. Ponownie uruchom generowanie embeddingów")

    if ready_count >= 50:
        print(f"🚀 GOTOWE DO NASTĘPNEGO ETAPU!")
        print(f"")
        print(f"📋 PIPELINE - Następne kroki:")
        print(f"   2️⃣ ✅ Embeddingi OpenAI - UKOŃCZONE")
        print(f"   3️⃣ 🔍 Similarity Search (candidates)")
        print(f"   4️⃣ 📊 Re-ranking Cohere")
        print(f"   5️⃣ 📈 Analiza i eksport")
        print(f"")
        print(f"💡 Uruchom: Skrypt 2 - Similarity Search")
    else:
        print(f"⚠️ Za mało gotowych artykułów ({ready_count})")
        print(f"💡 Potrzebujesz minimum 50 artykułów do similarity search")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja sprawdzająca"""

    print("✅ SPRAWDZENIE STATUSU EMBEDDINGÓW")
    print("=" * 50)

    # Pobierz sekrety
    secrets = get_secrets()

    # Połącz z bazą
    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd PostgreSQL: {e}")
        return

    try:
        # 1. Status ogólny
        overall = check_overall_status(connection)

        # 2. Breakdown embeddingów
        check_embedding_status_breakdown(connection)

        # 3. Jakość danych
        check_embedding_data_quality(connection)

        # 4. Gotowość do similarity
        ready_count = check_ready_for_similarity(connection)

        # 5. Problemy
        issues_count = check_potential_issues(connection)

        # 6. Następne kroki
        show_next_steps(ready_count, issues_count)

    finally:
        connection.close()
        print("\n🔒 Połączenie zamknięte")

if __name__ == "__main__":
    main()

✅ SPRAWDZENIE STATUSU EMBEDDINGÓW
✅ PostgreSQL połączony
📊 OGÓLNY STATUS BAZY DANYCH
📄 Łączna liczba artykułów: 174
🕷️ Artykuły po crawlingu: 174
🔤 Artykuły z embeddingami: 174
📈 % crawlingu: 100.0%
📈 % embeddingów: 100.0%

🔤 BREAKDOWN STATUSÓW EMBEDDINGÓW
Status                         Liczba     Procent   
-------------------------------------------------------
✅ completed                   174        100.0%

🔍 JAKOŚĆ DANYCH EMBEDDINGÓW
🔬 Sprawdzam 5 przykładowych embeddingów:

✅ Artykuł ID: 2
   🔗 URL: https://cateringfoodharmony.pl/poznajmy-sie...
   📄 Content embedding: 1536 wymiarów
   📰 Title embedding: 1536 wymiarów

✅ Artykuł ID: 3
   🔗 URL: https://cateringfoodharmony.pl/kontakt...
   📄 Content embedding: 1536 wymiarów
   📰 Title embedding: 1536 wymiarów

✅ Artykuł ID: 4
   🔗 URL: https://cateringfoodharmony.pl/diety...
   📄 Content embedding: 1536 wymiarów
   📰 Title embedding: 1536 wymiarów

✅ Artykuł ID: 5
   🔗 URL: https://cateringfoodharmony.pl/detox...
   📄 Content embe

# **5. SIMILARITY SEARCH**

In [16]:
#!/usr/bin/env python3
"""
🔍 NAPRAWIONY SKRYPT 2: SIMILARITY SEARCH
Pełna wersja bez błędów cursor handling
"""

import os
import psycopg2
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import logging
from datetime import datetime
from google.colab import userdata
from tqdm import tqdm

# Instalacja wymaganych bibliotek
try:
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    print("✅ numpy i sklearn już zainstalowane")
except ImportError:
    print("📦 Instaluję numpy i scikit-learn...")
    os.system('pip install numpy scikit-learn')
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

# ============================================================================
# 📝 KONFIGURACJA LOGOWANIA
# ============================================================================

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)
logger = logging.getLogger(__name__)

# ============================================================================
# 🔑 KONFIGURACJA ŚRODOWISKA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔍 NAPRAWIONA KLASA SIMILARITY SEARCH
# ============================================================================

class FixedSimilaritySearchProcessor:
    """Naprawiona klasa do similarity search"""

    def __init__(self):
        self.connection = None
        self.articles_data = []
        self.embeddings_matrix = None
        self.top_k = 20  # Top 20 podobnych artykułów

        print("✅ FixedSimilaritySearchProcessor zainicjalizowany")
        print(f"   🎯 Top K kandydatów: {self.top_k}")

    def get_db_connection(self):
        """Połączenie z PostgreSQL"""
        try:
            secrets = get_secrets()
            connection = psycopg2.connect(
                host=secrets['POSTGRESQL_HOST'],
                port=secrets['POSTGRESQL_PORT'],
                database=secrets['POSTGRESQL_DB'],
                user=secrets['POSTGRESQL_USER'],
                password=secrets['POSTGRESQL_PASSWORD']
            )
            logger.info("✅ Połączono z PostgreSQL")
            return connection
        except Exception as err:
            logger.error(f"❌ Błąd połączenia z PostgreSQL: {err}")
            return None

    def load_all_articles_with_embeddings(self, connection):
        """Załaduj wszystkie artykuły z embeddingami - NAPRAWIONA WERSJA"""

        logger.info("📥 Ładowanie wszystkich artykułów z embeddingami...")

        try:
            cursor = connection.cursor()

            # Pobierz wszystkie artykuły z embeddingami - PROSTY CURSOR
            cursor.execute("""
                SELECT id, link, embedding, title_embedding
                FROM articles
                WHERE text_status = 'completed'
                AND embedding_status = 'completed'
                AND embedding IS NOT NULL
                AND title_embedding IS NOT NULL
                ORDER BY id
            """)

            all_articles = cursor.fetchall()
            cursor.close()

            logger.info(f"📊 Pobrано z bazy: {len(all_articles)} artykułów")

            if len(all_articles) == 0:
                logger.error("❌ Brak artykułów z embeddingami!")
                return False

            # Przygotuj dane do przetwarzania
            self.articles_data = []
            embeddings_list = []

            logger.info("🔧 Przygotowywanie embeddingów...")

            invalid_count = 0

            for article in tqdm(all_articles, desc="Parsowanie embeddingów"):
                try:
                    article_id, link, embedding_str, title_embedding_str = article

                    # Parse embeddingu
                    content_embedding = json.loads(embedding_str)

                    # Sprawdź wymiary
                    if len(content_embedding) != 1536:
                        logger.warning(f"⚠️ Nieprawidłowy rozmiar embeddingu dla artykułu {article_id}: {len(content_embedding)}")
                        invalid_count += 1
                        continue

                    # Sprawdź czy wszystkie wartości są numeryczne
                    if not all(isinstance(x, (int, float)) for x in content_embedding[:10]):
                        logger.warning(f"⚠️ Nieprawidłowe wartości embeddingu dla artykułu {article_id}")
                        invalid_count += 1
                        continue

                    # Dodaj do list
                    self.articles_data.append({
                        'id': article_id,
                        'link': link,
                        'embedding': content_embedding
                    })

                    embeddings_list.append(content_embedding)

                except json.JSONDecodeError as e:
                    logger.warning(f"⚠️ Błąd JSON dla artykułu {article_id}: {e}")
                    invalid_count += 1
                    continue
                except Exception as e:
                    logger.warning(f"⚠️ Błąd dla artykułu {article_id}: {e}")
                    invalid_count += 1
                    continue

            # Konwertuj do numpy array
            if embeddings_list:
                self.embeddings_matrix = np.array(embeddings_list)
                logger.info(f"✅ Przygotowano macierz embeddingów: {self.embeddings_matrix.shape}")
                logger.info(f"📊 Prawidłowych artykułów: {len(self.articles_data)}")
                logger.info(f"⚠️ Nieprawidłowych artykułów: {invalid_count}")
                return True
            else:
                logger.error("❌ Brak prawidłowych embeddingów!")
                return False

        except Exception as e:
            logger.error(f"❌ Błąd ładowania artykułów: {e}")
            return False

    def calculate_similarity_batch(self, batch_size=1000):
        """Oblicz macierz podobieństwa z batch processing"""

        logger.info("🧮 Obliczanie macierzy podobieństwa...")

        try:
            if self.embeddings_matrix is None:
                logger.error("❌ Brak macierzy embeddingów!")
                return None

            num_articles = len(self.articles_data)
            logger.info(f"📊 Liczba artykułów: {num_articles}")

            # Jeśli mało artykułów, oblicz wszystko na raz
            if num_articles <= 1000:
                start_time = datetime.now()
                similarity_matrix = cosine_similarity(self.embeddings_matrix)
                end_time = datetime.now()

                duration = (end_time - start_time).total_seconds()
                logger.info(f"✅ Macierz podobieństwa obliczona w {duration:.2f}s")
                logger.info(f"📊 Rozmiar macierzy: {similarity_matrix.shape}")

                return similarity_matrix

            else:
                # Dla większych zbiorów - batch processing
                logger.info(f"📦 Duży zbiór - używam batch processing")
                # TODO: Implementacja batch processing dla bardzo dużych zbiorów
                similarity_matrix = cosine_similarity(self.embeddings_matrix)
                return similarity_matrix

        except Exception as e:
            logger.error(f"❌ Błąd obliczania podobieństwa: {e}")
            return None

    def find_candidates_for_all_articles(self, similarity_matrix):
        """Znajdź kandydatów dla wszystkich artykułów"""

        logger.info(f"🔍 Wyszukiwanie top {self.top_k} podobnych artykułów...")

        candidates_data = []

        with tqdm(range(len(self.articles_data)), desc="Wyszukiwanie kandydatów") as pbar:
            for i in pbar:
                try:
                    article = self.articles_data[i]

                    # Znajdź indeksy najbardziej podobnych artykułów
                    similarity_scores = similarity_matrix[i]

                    # Sortuj według podobieństwa (malejąco)
                    similar_indices = np.argsort(similarity_scores)[::-1]

                    # Zbierz kandydatów (pomijając sam siebie)
                    candidates = []
                    seen_links = set()

                    for idx in similar_indices:
                        if len(candidates) >= self.top_k:
                            break

                        # Pomiń sam siebie
                        if idx == i:
                            continue

                        similar_article = self.articles_data[idx]
                        similar_link = similar_article['link']
                        similarity_score = float(similarity_scores[idx])

                        # Pomiń duplikaty URL-i
                        if similar_link in seen_links:
                            continue

                        # Pomiń artykuły z bardzo niskim podobieństwem
                        if similarity_score < 0.05:  # Próg podobieństwa
                            continue

                        candidates.append({
                            'link': similar_link,
                            'score': similarity_score
                        })

                        seen_links.add(similar_link)

                    # Dodaj do wyników
                    candidates_data.append({
                        'article_id': article['id'],
                        'article_link': article['link'],
                        'candidates': candidates
                    })

                    # Aktualizuj progress bar
                    if i % 10 == 0:
                        pbar.set_postfix_str(f"Avg: {len(candidates)} kandydatów")

                except Exception as e:
                    logger.warning(f"⚠️ Błąd dla artykułu {i}: {e}")
                    continue

        logger.info(f"✅ Znaleziono kandydatów dla {len(candidates_data)} artykułów")
        return candidates_data

    def save_all_candidates_to_db(self, connection, candidates_data):
        """Zapisz wszystkich kandydatów do bazy danych"""

        logger.info("💾 Zapisywanie kandydatów do bazy...")

        try:
            success_count = 0
            error_count = 0

            with tqdm(candidates_data, desc="Zapisywanie kandydatów") as pbar:
                for data in pbar:
                    try:
                        article_id = data['article_id']
                        candidates = data['candidates']

                        # Konwertuj kandydatów do JSON
                        candidates_json = json.dumps(candidates, ensure_ascii=False)

                        # Zapisz do bazy
                        cursor = connection.cursor()
                        cursor.execute("""
                            UPDATE articles
                            SET candidates = %s
                            WHERE id = %s
                        """, (candidates_json, article_id))

                        connection.commit()
                        cursor.close()

                        success_count += 1

                        # Aktualizuj progress bar co 50 zapisów
                        if success_count % 50 == 0:
                            pbar.set_postfix_str(f"Zapisano {success_count}")

                    except Exception as e:
                        logger.warning(f"⚠️ Błąd zapisu dla artykułu {data['article_id']}: {e}")
                        error_count += 1
                        try:
                            connection.rollback()
                        except:
                            pass
                        continue

            logger.info(f"✅ Zapisano kandydatów: {success_count} sukces, {error_count} błędów")
            return success_count

        except Exception as e:
            logger.error(f"❌ Błąd zapisywania kandydatów: {e}")
            return 0

    def show_detailed_statistics(self, candidates_data):
        """Pokaż szczegółowe statystyki similarity search"""

        print(f"\n📊 SZCZEGÓŁOWE STATYSTYKI SIMILARITY SEARCH")
        print("=" * 60)

        if not candidates_data:
            print("❌ Brak danych do analizy")
            return

        # Podstawowe statystyki
        total_articles = len(candidates_data)

        candidate_counts = [len(data['candidates']) for data in candidates_data]
        total_candidates = sum(candidate_counts)
        avg_candidates = total_candidates / total_articles if total_articles > 0 else 0
        max_candidates = max(candidate_counts) if candidate_counts else 0
        min_candidates = min(candidate_counts) if candidate_counts else 0

        print(f"📄 Artykuły przetworzonych: {total_articles}")
        print(f"🔗 Łączna liczba kandydatów: {total_candidates}")
        print(f"📈 Średnia kandydatów na artykuł: {avg_candidates:.1f}")
        print(f"📊 Min kandydatów: {min_candidates}")
        print(f"📊 Max kandydatów: {max_candidates}")

        # Statystyki podobieństwa
        all_scores = []
        high_similarity_count = 0  # > 0.8
        medium_similarity_count = 0  # 0.5-0.8
        low_similarity_count = 0  # < 0.5

        for data in candidates_data:
            for candidate in data['candidates']:
                score = candidate['score']
                all_scores.append(score)

                if score > 0.8:
                    high_similarity_count += 1
                elif score > 0.5:
                    medium_similarity_count += 1
                else:
                    low_similarity_count += 1

        if all_scores:
            all_scores = np.array(all_scores)
            print(f"\n⭐ ROZKŁAD PODOBIEŃSTW:")
            print(f"   📈 Średnie podobieństwo: {np.mean(all_scores):.3f}")
            print(f"   📊 Mediana podobieństwa: {np.median(all_scores):.3f}")
            print(f"   📊 Min podobieństwo: {np.min(all_scores):.3f}")
            print(f"   📊 Max podobieństwo: {np.max(all_scores):.3f}")
            print(f"   🔥 Wysokie (>0.8): {high_similarity_count}")
            print(f"   🟡 Średnie (0.5-0.8): {medium_similarity_count}")
            print(f"   🔵 Niskie (<0.5): {low_similarity_count}")

        # Top 10 par z największym podobieństwem
        print(f"\n🏆 TOP 10 NAJBARDZIEJ PODOBNYCH PAR:")
        top_pairs = []

        for data in candidates_data:
            if data['candidates']:
                best_candidate = max(data['candidates'], key=lambda x: x['score'])
                top_pairs.append({
                    'source': data['article_link'],
                    'target': best_candidate['link'],
                    'score': best_candidate['score']
                })

        top_pairs.sort(key=lambda x: x['score'], reverse=True)

        for i, pair in enumerate(top_pairs[:10], 1):
            source_short = pair['source'].split('/')[-1][:25] + '...' if len(pair['source'].split('/')[-1]) > 25 else pair['source'].split('/')[-1]
            target_short = pair['target'].split('/')[-1][:25] + '...' if len(pair['target'].split('/')[-1]) > 25 else pair['target'].split('/')[-1]
            print(f"{i:2d}. {pair['score']:.3f} | {source_short} → {target_short}")

    def run_full_similarity_search(self):
        """Uruchom kompletny similarity search - NAPRAWIONA WERSJA"""

        logger.info("🔍 URUCHAMIAM PEŁNY SIMILARITY SEARCH")
        logger.info("=" * 50)

        # Połącz z bazą
        connection = self.get_db_connection()
        if not connection:
            return False

        try:
            start_time = datetime.now()

            # 1. Załaduj artykuły z embeddingami
            if not self.load_all_articles_with_embeddings(connection):
                return False

            # Sprawdź czy mamy wystarczająco artykułów
            if len(self.articles_data) < 5:
                logger.error(f"❌ Za mało artykułów ({len(self.articles_data)}) do similarity search!")
                return False

            # 2. Oblicz macierz podobieństwa
            similarity_matrix = self.calculate_similarity_batch()
            if similarity_matrix is None:
                return False

            # 3. Znajdź podobne artykuły
            candidates_data = self.find_candidates_for_all_articles(similarity_matrix)
            if not candidates_data:
                return False

            # 4. Zapisz do bazy
            saved_count = self.save_all_candidates_to_db(connection, candidates_data)

            # 5. Pokaż statystyki
            self.show_detailed_statistics(candidates_data)

            # Podsumowanie
            end_time = datetime.now()
            duration = end_time - start_time

            logger.info(f"\n🎉 SIMILARITY SEARCH ZAKOŃCZONY!")
            logger.info("=" * 40)
            logger.info(f"⏱️ Czas trwania: {duration}")
            logger.info(f"📊 Przetworzono: {len(self.articles_data)} artykułów")
            logger.info(f"💾 Zapisano: {saved_count} zestawów kandydatów")

            if duration.total_seconds() > 0:
                rate = len(self.articles_data) / (duration.total_seconds() / 60)
                logger.info(f"⚡ Prędkość: {rate:.1f} artykułów/min")

            return saved_count > 0

        finally:
            connection.close()
            logger.info("🔒 Połączenie zamknięte")

# ============================================================================
# 📊 FUNKCJE POMOCNICZE
# ============================================================================

def check_current_similarity_status():
    """Sprawdź obecny status similarity search"""

    print(f"\n📊 OBECNY STATUS SIMILARITY SEARCH")
    print("=" * 45)

    try:
        secrets = get_secrets()
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )

        cursor = connection.cursor()

        # Artykuły z kandydatami
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE candidates IS NOT NULL
            AND candidates != '[]'
            AND candidates != 'null'
        """)

        with_candidates = cursor.fetchone()[0]

        # Łączna liczba artykułów z embeddingami
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE embedding_status = 'completed'
        """)

        total_embedded = cursor.fetchone()[0]

        print(f"📄 Artykuły z embeddingami: {total_embedded}")
        print(f"🔗 Artykuły z kandydatami: {with_candidates}")

        if total_embedded > 0:
            percent = (with_candidates / total_embedded) * 100
            print(f"📈 Pokrycie similarity: {percent:.1f}%")

            if percent >= 95:
                print(f"✅ GOTOWE do następnego etapu!")
            elif percent >= 50:
                print(f"🟡 Częściowo gotowe")
            else:
                print(f"🔴 Wymaga dokończenia")

        cursor.close()
        connection.close()

        return with_candidates, total_embedded

    except Exception as e:
        print(f"❌ Błąd sprawdzania statusu: {e}")
        return 0, 0

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja programu"""

    print("🔍 NAPRAWIONY SIMILARITY SEARCH")
    print("=" * 50)

    # Sprawdź obecny status
    with_candidates, total_embedded = check_current_similarity_status()

    # Menu opcji
    print(f"\n🎯 OPCJE:")
    print("1. Pełny similarity search (wszystkie artykuły)")
    print("2. Test na 50 artykułach")
    print("3. Tylko sprawdzenie statusu")

    if with_candidates > 0:
        print("4. Nadpisanie istniejących kandydatów")

    choice = input("\nWybierz opcję (1-4): ").strip()

    if choice == '1':
        # Pełny similarity search
        print(f"\n🚀 PEŁNY SIMILARITY SEARCH")
        print(f"📊 Będzie przetwarzać {total_embedded} artykułów")

        confirm = input("Kontynuować? (tak/nie): ").strip().lower()
        if confirm in ['tak', 'yes', 't', 'y', '']:
            processor = FixedSimilaritySearchProcessor()
            success = processor.run_full_similarity_search()

            if success:
                print(f"\n🎯 GOTOWE DO NASTĘPNEGO ETAPU!")
                print(f"💡 Uruchom: Skrypt 3 - Re-ranking Cohere")
        else:
            print("❌ Anulowano")

    elif choice == '2':
        # Test na 50 artykułach
        print(f"\n🧪 TEST SIMILARITY SEARCH (50 artykułów)")
        processor = FixedSimilaritySearchProcessor()
        processor.top_k = 10  # Mniej kandydatów do testu

        # Ograniczoną wersję można zrobić modyfikując zapytanie
        success = processor.run_full_similarity_search()

    elif choice == '3':
        # Tylko status
        print(f"✅ Status już wyświetlony powyżej")

    elif choice == '4' and with_candidates > 0:
        # Nadpisanie
        print(f"\n🔄 NADPISYWANIE KANDYDATÓW")
        print(f"⚠️ To nadpisze {with_candidates} istniejących zestawów kandydatów")
        confirm = input("Kontynuować? (tak/nie): ").strip().lower()

        if confirm in ['tak', 'yes', 't', 'y']:
            processor = FixedSimilaritySearchProcessor()
            processor.run_full_similarity_search()
        else:
            print("❌ Anulowano")

    else:
        print("❌ Nieprawidłowy wybór!")

if __name__ == "__main__":
    main()

✅ numpy i sklearn już zainstalowane
🔍 NAPRAWIONY SIMILARITY SEARCH

📊 OBECNY STATUS SIMILARITY SEARCH
📄 Artykuły z embeddingami: 174
🔗 Artykuły z kandydatami: 3
📈 Pokrycie similarity: 1.7%
🔴 Wymaga dokończenia

🎯 OPCJE:
1. Pełny similarity search (wszystkie artykuły)
2. Test na 50 artykułach
3. Tylko sprawdzenie statusu
4. Nadpisanie istniejących kandydatów

Wybierz opcję (1-4): 4

🔄 NADPISYWANIE KANDYDATÓW
⚠️ To nadpisze 3 istniejących zestawów kandydatów
Kontynuować? (tak/nie): tak


2025-06-03 09:04:07,482 - INFO - 🔍 URUCHAMIAM PEŁNY SIMILARITY SEARCH
2025-06-03 09:04:07,483 - INFO - ==================================================


✅ FixedSimilaritySearchProcessor zainicjalizowany
   🎯 Top K kandydatów: 20


2025-06-03 09:04:10,958 - INFO - ✅ Połączono z PostgreSQL
2025-06-03 09:04:10,960 - INFO - 📥 Ładowanie wszystkich artykułów z embeddingami...
2025-06-03 09:04:14,064 - INFO - 📊 Pobrано z bazy: 174 artykułów
2025-06-03 09:04:14,065 - INFO - 🔧 Przygotowywanie embeddingów...
Parsowanie embeddingów: 100%|██████████| 174/174 [00:00<00:00, 2006.05it/s]
2025-06-03 09:04:14,166 - INFO - ✅ Przygotowano macierz embeddingów: (174, 1536)
2025-06-03 09:04:14,167 - INFO - 📊 Prawidłowych artykułów: 174
2025-06-03 09:04:14,169 - INFO - ⚠️ Nieprawidłowych artykułów: 0
2025-06-03 09:04:14,173 - INFO - 🧮 Obliczanie macierzy podobieństwa...
2025-06-03 09:04:14,173 - INFO - 📊 Liczba artykułów: 174
2025-06-03 09:04:14,181 - INFO - ✅ Macierz podobieństwa obliczona w 0.01s
2025-06-03 09:04:14,183 - INFO - 📊 Rozmiar macierzy: (174, 174)
2025-06-03 09:04:14,184 - INFO - 🔍 Wyszukiwanie top 20 podobnych artykułów...
Wyszukiwanie kandydatów: 100%|██████████| 174/174 [00:00<00:00, 6674.13it/s, Avg: 20 kandydatów]
2


📊 SZCZEGÓŁOWE STATYSTYKI SIMILARITY SEARCH
📄 Artykuły przetworzonych: 174
🔗 Łączna liczba kandydatów: 3480
📈 Średnia kandydatów na artykuł: 20.0
📊 Min kandydatów: 20
📊 Max kandydatów: 20

⭐ ROZKŁAD PODOBIEŃSTW:
   📈 Średnie podobieństwo: 0.888
   📊 Mediana podobieństwa: 0.908
   📊 Min podobieństwo: 0.612
   📊 Max podobieństwo: 1.000
   🔥 Wysokie (>0.8): 2927
   🟡 Średnie (0.5-0.8): 553
   🔵 Niskie (<0.5): 0

🏆 TOP 10 NAJBARDZIEJ PODOBNYCH PAR:
 1. 1.000 | free-wege → dieta-pudelkowa-wege-harm...
 2. 1.000 | dieta-pudelkowa-wege-harm... → free-wege
 3. 1.000 | aktywny-weekend → trening-food
 4. 1.000 | co-do-pracy → trening-food
 5. 1.000 | dieta-pudelkowa → trening-food
 6. 1.000 | trening-food → dieta-pudelkowa
 7. 1.000 | dieta-w-prezencie-1621254... → wymiana-posilkow-w-cateri...
 8. 1.000 | wymiana-posilkow-w-cateri... → dieta-w-prezencie-1621254...
 9. 1.000 | jak-schudnac-z-brzucha → jak-szybko-schudnac-z-brz...
10. 1.000 | jak-szybko-schudnac-z-brz... → jak-schudnac-z-brzucha


## 5.1 Test Similarity Search

In [15]:
#!/usr/bin/env python3
"""
🔍 PROSTY TEST SIMILARITY SEARCH
Bez skomplikowanych cursorów - tylko podstawowa funkcjonalność
"""

import os
import psycopg2
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata
from tqdm import tqdm

# ============================================================================
# 🔑 KONFIGURACJA
# ============================================================================

def get_secrets():
    """Pobierz sekrety z Colab"""
    secrets = {}
    for name in ['POSTGRESQL_HOST', 'POSTGRESQL_DB', 'POSTGRESQL_USER',
                'POSTGRESQL_PASSWORD']:
        secrets[name] = userdata.get(name)
    secrets['POSTGRESQL_PORT'] = '5432'
    return secrets

# ============================================================================
# 🔍 PROSTY SIMILARITY SEARCH
# ============================================================================

def simple_similarity_search():
    """Prosty test similarity search"""

    print("🔍 PROSTY TEST SIMILARITY SEARCH")
    print("=" * 45)

    # Połącz z bazą
    secrets = get_secrets()

    try:
        connection = psycopg2.connect(
            host=secrets['POSTGRESQL_HOST'],
            port=secrets['POSTGRESQL_PORT'],
            database=secrets['POSTGRESQL_DB'],
            user=secrets['POSTGRESQL_USER'],
            password=secrets['POSTGRESQL_PASSWORD']
        )
        print("✅ PostgreSQL połączony")
    except Exception as e:
        print(f"❌ Błąd połączenia: {e}")
        return

    try:
        cursor = connection.cursor()

        # 1. Sprawdź ile artykułów jest gotowych
        print(f"\n📊 SPRAWDZAM GOTOWOŚĆ...")

        cursor.execute("""
            SELECT id, link, embedding, title_embedding
            FROM articles
            WHERE text_status = 'completed'
            AND embedding_status = 'completed'
            AND embedding IS NOT NULL
            AND title_embedding IS NOT NULL
            LIMIT 20
        """)

        articles = cursor.fetchall()
        print(f"✅ Znaleziono {len(articles)} artykułów do testu")

        if len(articles) < 5:
            print("❌ Za mało artykułów do testu!")
            return

        # 2. Przygotuj embeddingi
        print(f"\n🔧 PRZYGOTOWUJĘ EMBEDDINGI...")

        articles_data = []
        embeddings_list = []

        for article in tqdm(articles, desc="Parsing embeddingów"):
            try:
                article_id, link, embedding_str, title_embedding_str = article

                # Parse embedding
                embedding = json.loads(embedding_str)

                # Sprawdź rozmiar
                if len(embedding) != 1536:
                    print(f"⚠️ Błędny rozmiar embeddingu dla ID {article_id}: {len(embedding)}")
                    continue

                # Dodaj do list
                articles_data.append({
                    'id': article_id,
                    'link': link,
                    'embedding': embedding
                })

                embeddings_list.append(embedding)

            except Exception as e:
                print(f"❌ Błąd dla artykułu {article_id}: {e}")
                continue

        print(f"✅ Przygotowano {len(embeddings_list)} embeddingów")

        if len(embeddings_list) < 3:
            print("❌ Za mało prawidłowych embeddingów!")
            return

        # 3. Oblicz podobieństwo
        print(f"\n🧮 OBLICZAM PODOBIEŃSTWO...")

        embeddings_matrix = np.array(embeddings_list)
        print(f"📊 Macierz embeddingów: {embeddings_matrix.shape}")

        similarity_matrix = cosine_similarity(embeddings_matrix)
        print(f"✅ Macierz podobieństwa: {similarity_matrix.shape}")

        # 4. Znajdź kandydatów dla pierwszych 3 artykułów
        print(f"\n🔍 WYSZUKUJĘ KANDYDATÓW...")

        candidates_results = []

        for i in range(min(3, len(articles_data))):
            article = articles_data[i]
            similarity_scores = similarity_matrix[i]

            # Sortuj według podobieństwa
            similar_indices = np.argsort(similarity_scores)[::-1]

            # Znajdź top 5 kandydatów (pomijając siebie)
            candidates = []
            for idx in similar_indices:
                if idx == i:  # Pomiń siebie
                    continue

                if len(candidates) >= 5:
                    break

                similar_article = articles_data[idx]
                score = float(similarity_scores[idx])

                candidates.append({
                    'link': similar_article['link'],
                    'score': score
                })

            candidates_results.append({
                'article_id': article['id'],
                'article_link': article['link'],
                'candidates': candidates
            })

            print(f"   ✅ Artykuł {article['id']}: {len(candidates)} kandydatów")

        # 5. Pokaż wyniki
        print(f"\n📋 WYNIKI TESTU:")
        print("=" * 30)

        for result in candidates_results:
            print(f"\n🔗 Źródło: {result['article_link'][:50]}...")
            print(f"   Kandydaci:")

            for j, candidate in enumerate(result['candidates'], 1):
                score = candidate['score']
                link = candidate['link'][:40] + '...' if len(candidate['link']) > 40 else candidate['link']
                print(f"      {j}. {score:.3f} | {link}")

        # 6. Test zapisu do bazy
        print(f"\n💾 TEST ZAPISU DO BAZY...")

        success_count = 0

        for result in candidates_results:
            try:
                article_id = result['article_id']
                candidates_json = json.dumps(result['candidates'], ensure_ascii=False)

                cursor.execute("""
                    UPDATE articles
                    SET candidates = %s
                    WHERE id = %s
                """, (candidates_json, article_id))

                connection.commit()
                success_count += 1

            except Exception as e:
                print(f"❌ Błąd zapisu dla ID {article_id}: {e}")
                connection.rollback()

        print(f"✅ Zapisano kandydatów dla {success_count} artykułów")

        # 7. Weryfikacja zapisu
        print(f"\n✅ WERYFIKACJA ZAPISU...")

        cursor.execute("""
            SELECT id, candidates
            FROM articles
            WHERE candidates IS NOT NULL
            AND candidates != '[]'
            LIMIT 3
        """)

        saved_candidates = cursor.fetchall()

        for article_id, candidates_str in saved_candidates:
            try:
                candidates = json.loads(candidates_str)
                print(f"   ✅ ID {article_id}: {len(candidates)} kandydatów zapisanych")
            except:
                print(f"   ❌ ID {article_id}: błąd odczytu kandydatów")

        cursor.close()

        print(f"\n🎉 TEST ZAKOŃCZONY POMYŚLNIE!")
        print(f"🚀 Możesz teraz uruchomić pełny similarity search")

    except Exception as e:
        print(f"❌ Błąd testu: {e}")
        import traceback
        print(f"🔍 Traceback: {traceback.format_exc()}")

    finally:
        connection.close()
        print("🔒 Połączenie zamknięte")

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja"""

    # Sprawdź czy biblioteki są zainstalowane
    try:
        import numpy as np
        from sklearn.metrics.pairwise import cosine_similarity
        print("✅ Biblioteki gotowe")
    except ImportError:
        print("📦 Instaluję biblioteki...")
        os.system('pip install numpy scikit-learn')
        import numpy as np
        from sklearn.metrics.pairwise import cosine_similarity

    # Uruchom test
    simple_similarity_search()

if __name__ == "__main__":
    main()

✅ Biblioteki gotowe
🔍 PROSTY TEST SIMILARITY SEARCH
✅ PostgreSQL połączony

📊 SPRAWDZAM GOTOWOŚĆ...
✅ Znaleziono 20 artykułów do testu

🔧 PRZYGOTOWUJĘ EMBEDDINGI...


Parsing embeddingów: 100%|██████████| 20/20 [00:00<00:00, 1887.63it/s]

✅ Przygotowano 20 embeddingów

🧮 OBLICZAM PODOBIEŃSTWO...
📊 Macierz embeddingów: (20, 1536)
✅ Macierz podobieństwa: (20, 20)

🔍 WYSZUKUJĘ KANDYDATÓW...
   ✅ Artykuł 2: 5 kandydatów
   ✅ Artykuł 3: 5 kandydatów
   ✅ Artykuł 4: 5 kandydatów

📋 WYNIKI TESTU:

🔗 Źródło: https://cateringfoodharmony.pl/poznajmy-sie...
   Kandydaci:
      1. 0.983 | https://cateringfoodharmony.pl/promocje
      2. 0.967 | https://cateringfoodharmony.pl/free-harm...
      3. 0.963 | https://cateringfoodharmony.pl/wege-ryba
      4. 0.963 | https://cateringfoodharmony.pl/sport-har...
      5. 0.931 | https://cateringfoodharmony.pl/sirtfood

🔗 Źródło: https://cateringfoodharmony.pl/kontakt...
   Kandydaci:
      1. 0.790 | https://cateringfoodharmony.pl/rejony-do...
      2. 0.694 | https://cateringfoodharmony.pl/promocje
      3. 0.671 | https://cateringfoodharmony.pl/poznajmy-...
      4. 0.663 | https://cateringfoodharmony.pl/sirtfood
      5. 0.645 | https://cateringfoodharmony.pl/smoothie-...

🔗 Źródło: htt

✅ Zapisano kandydatów dla 3 artykułów

✅ WERYFIKACJA ZAPISU...
   ✅ ID 2: 5 kandydatów zapisanych
   ✅ ID 3: 5 kandydatów zapisanych
   ✅ ID 4: 5 kandydatów zapisanych

🎉 TEST ZAKOŃCZONY POMYŚLNIE!
🚀 Możesz teraz uruchomić pełny similarity search
🔒 Połączenie zamknięte


# **6. RERANKING COHERE**

In [ ]:
#!/usr/bin/env python3
"""
📊 SKRYPT 3: RE-RANKING COHERE
Filtrowanie i re-ranking kandydatów za pomocą Cohere API
"""

import os
import psycopg2
from psycopg2.extras import RealDictCursor
import json
import logging
import time
from datetime import datetime
from google.colab import userdata
from tqdm import tqdm

# Instalacja wymaganej biblioteki
try:
    import cohere
    print("✅ cohere już zainstalowany")
except ImportError:
    print("📦 Instaluję cohere...")
    os.system('pip install cohere')
    import cohere

# ============================================================================
# 📝 KONFIGURACJA LOGOWANIA
# ============================================================================

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)
logger = logging.getLogger(__name__)

# ============================================================================
# 🔑 KONFIGURACJA ŚRODOWISKA
# ============================================================================

def setup_environment():
    """Ustaw zmienne środowiskowe z ukrytych sekretów"""
    try:
        os.environ['POSTGRESQL_HOST'] = userdata.get('POSTGRESQL_HOST')
        os.environ['POSTGRESQL_PORT'] = '5432'
        os.environ['POSTGRESQL_USER'] = userdata.get('POSTGRESQL_USER')
        os.environ['POSTGRESQL_PASSWORD'] = userdata.get('POSTGRESQL_PASSWORD')
        os.environ['POSTGRESQL_DB'] = userdata.get('POSTGRESQL_DB')
        os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')

        print("✅ Zmienne środowiskowe ustawione")

        # Sprawdź czy klucze są ustawione
        if not os.environ.get('COHERE_API_KEY'):
            raise ValueError("Brak COHERE_API_KEY!")

        return True

    except Exception as e:
        print(f"❌ Błąd konfiguracji: {e}")
        return False

# ============================================================================
# 🧠 KLASA RE-RANKING PROCESSOR
# ============================================================================

class CohereRerankingProcessor:
    """Główna klasa do re-rankingu z Cohere"""

    def __init__(self):
        self.connection = None
        self.cohere_client = None
        self.max_candidates_to_rerank = 10  # Top 10 z similarity search
        self.top_n_output = 5  # Top 5 po re-rankingu
        self.rate_limit_delay = 2  # Sekund między requestami

        print("✅ CohereRerankingProcessor zainicjalizowany")
        print(f"   🎯 Max kandydatów do re-rankingu: {self.max_candidates_to_rerank}")
        print(f"   🏆 Top N wyników: {self.top_n_output}")
        print(f"   ⏱️ Delay między requestami: {self.rate_limit_delay}s")

    def get_db_connection(self):
        """Połączenie z PostgreSQL"""
        try:
            connection = psycopg2.connect(
                host=os.environ['POSTGRESQL_HOST'],
                port=os.environ['POSTGRESQL_PORT'],
                database=os.environ['POSTGRESQL_DB'],
                user=os.environ['POSTGRESQL_USER'],
                password=os.environ['POSTGRESQL_PASSWORD']
            )
            logger.info("✅ Połączono z PostgreSQL")
            return connection
        except Exception as err:
            logger.error(f"❌ Błąd połączenia z PostgreSQL: {err}")
            return None

    def initialize_cohere_client(self):
        """Inicjalizacja klienta Cohere"""
        try:
            cohere_api_key = os.environ.get('COHERE_API_KEY')
            if not cohere_api_key:
                logger.error("❌ Brak COHERE_API_KEY!")
                return False

            self.cohere_client = cohere.Client(cohere_api_key)
            logger.info("✅ Klient Cohere zainicjalizowany")
            return True

        except Exception as e:
            logger.error(f"❌ Błąd inicjalizacji Cohere: {e}")
            return False

    def fetch_articles_with_candidates(self, connection, batch_size=100):
        """Pobierz artykuły z kandydatami do re-rankingu"""

        logger.info("📥 Ładowanie artykułów z kandydatami...")

        try:
            cursor = connection.cursor()

            # Pobierz artykuły z kandydatami, ale bez re-rankingu
            cursor.execute("""
                SELECT id, link, candidates
                FROM articles
                WHERE text_status = 'completed'
                AND candidates IS NOT NULL
                AND candidates != '[]'
                AND candidates != 'null'
                AND (re_rank_content IS NULL OR re_rank_content = '[]' OR re_rank_content = 'null')
                ORDER BY id
                LIMIT %s
            """, (batch_size,))

            articles = cursor.fetchall()
            cursor.close()

            logger.info(f"📊 Znaleziono {len(articles)} artykułów z kandydatami do re-rankingu")
            return articles

        except Exception as e:
            logger.error(f"❌ Błąd pobierania artykułów: {e}")
            return []

    def extract_candidate_links(self, candidates_json):
        """Wyciągnij linki z JSON kandydatów"""
        try:
            if not candidates_json:
                return []

            candidates = json.loads(candidates_json)
            if not isinstance(candidates, list):
                return []

            # Wyciągnij linki (ograniczając do max_candidates_to_rerank)
            links = []
            for candidate in candidates[:self.max_candidates_to_rerank]:
                if isinstance(candidate, dict) and 'link' in candidate:
                    links.append(candidate['link'])

            return links

        except json.JSONDecodeError as e:
            logger.warning(f"⚠️ Błąd JSON w kandydatach: {e}")
            return []
        except Exception as e:
            logger.warning(f"⚠️ Błąd ekstraktowania kandydatów: {e}")
            return []

    def fetch_texts_for_links(self, connection, links):
        """Pobierz teksty dla podanych linków"""

        if not links:
            return {}

        try:
            cursor = connection.cursor()

            # Przygotuj zapytanie z placeholder dla każdego linka
            placeholders = ','.join(['%s'] * len(links))
            query = f"""
                SELECT link, text
                FROM articles
                WHERE link IN ({placeholders})
                AND text_status = 'completed'
            """

            cursor.execute(query, tuple(links))
            rows = cursor.fetchall()
            cursor.close()

            # Parse tekstów z JSON
            texts = {}
            for link, text_json in rows:
                try:
                    if text_json:
                        json_data = json.loads(text_json)
                        content = json_data.get('data', {}).get('content', '')

                        # Skróć treść do rozsądnej długości (Cohere ma limity)
                        if len(content) > 4000:
                            content = content[:4000] + '...'

                        texts[link] = content

                except json.JSONDecodeError:
                    logger.warning(f"⚠️ Błąd JSON dla link: {link}")
                    continue

            logger.info(f"📄 Pobrano teksty dla {len(texts)} linków z {len(links)} żądanych")
            return texts

        except Exception as e:
            logger.error(f"❌ Błąd pobierania tekstów: {e}")
            return {}

    def rerank_candidates_with_cohere(self, query_url, candidate_links, candidate_texts):
        """Re-rankuj kandydatów za pomocą Cohere API"""

        try:
            # Przygotuj dokumenty do re-rankingu
            documents = []
            valid_links = []

            for link in candidate_links:
                if link in candidate_texts and candidate_texts[link].strip():
                    documents.append(candidate_texts[link])
                    valid_links.append(link)

            if not documents:
                logger.warning(f"⚠️ Brak tekstów do re-rankingu dla {query_url}")
                return []

            if len(documents) < 2:
                logger.warning(f"⚠️ Za mało dokumentów do re-rankingu ({len(documents)}) dla {query_url}")
                return []

            logger.info(f"🔄 Re-ranking {len(documents)} dokumentów dla {query_url[:50]}...")

            # Wywołaj Cohere API
            response = self.cohere_client.rerank(
                model="rerank-multilingual-v3.0",
                query=query_url,  # Używamy URL jako query
                documents=documents,
                top_n=min(self.top_n_output, len(documents))
            )

            # Przygotuj wyniki
            reranked_results = []
            for result in response.results:
                if result.index < len(valid_links):
                    reranked_results.append({
                        'link': valid_links[result.index],
                        'relevance_score': float(result.relevance_score)
                    })

            logger.info(f"✅ Re-ranking ukończony: {len(reranked_results)} wyników")
            return reranked_results

        except Exception as e:
            logger.error(f"❌ Błąd Cohere re-ranking: {e}")
            return []

    def save_reranked_results(self, connection, article_id, reranked_results):
        """Zapisz wyniki re-rankingu do bazy"""

        try:
            # Konwertuj do JSON
            rerank_json = json.dumps(reranked_results, ensure_ascii=False)

            cursor = connection.cursor()
            cursor.execute("""
                UPDATE articles
                SET re_rank_content = %s
                WHERE id = %s
            """, (rerank_json, article_id))

            connection.commit()
            cursor.close()

            logger.info(f"✅ Zapisano re-ranking dla artykułu ID: {article_id}")
            return True

        except Exception as e:
            logger.error(f"❌ Błąd zapisu re-rankingu dla ID {article_id}: {e}")
            connection.rollback()
            return False

    def process_single_article(self, connection, article_data):
        """Przetwórz pojedynczy artykuł - re-ranking kandydatów"""

        article_id, query_url, candidates_json = article_data

        try:
            # Wyciągnij kandydatów
            candidate_links = self.extract_candidate_links(candidates_json)

            if not candidate_links:
                logger.warning(f"⚠️ Brak kandydatów dla artykułu ID: {article_id}")
                return False

            # Pobierz teksty kandydatów
            candidate_texts = self.fetch_texts_for_links(connection, candidate_links)

            if not candidate_texts:
                logger.warning(f"⚠️ Brak tekstów kandydatów dla artykułu ID: {article_id}")
                return False

            # Re-rankuj z Cohere
            reranked_results = self.rerank_candidates_with_cohere(
                query_url, candidate_links, candidate_texts
            )

            if not reranked_results:
                logger.warning(f"⚠️ Re-ranking nie zwrócił wyników dla artykułu ID: {article_id}")
                return False

            # Zapisz wyniki
            success = self.save_reranked_results(connection, article_id, reranked_results)

            return success

        except Exception as e:
            logger.error(f"❌ Błąd przetwarzania artykułu ID {article_id}: {e}")
            return False

    def process_batch_with_rate_limiting(self, connection, articles_batch):
        """Przetwórz batch artykułów z rate limitingiem"""

        success_count = 0
        error_count = 0

        logger.info(f"🚀 Przetwarzam batch {len(articles_batch)} artykułów...")

        # Progress bar
        pbar = tqdm(articles_batch,
                   desc="Re-ranking Cohere",
                   ncols=80,
                   bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]")

        for article in pbar:
            article_id = article[0]
            query_url = article[1][:50] + '...' if len(article[1]) > 50 else article[1]

            # Aktualizuj opis progress bara
            pbar.set_postfix_str(f"ID: {article_id}")

            # Przetwórz artykuł
            success = self.process_single_article(connection, article)

            if success:
                success_count += 1
            else:
                error_count += 1

            # Rate limiting - pauza między requestami
            if article != articles_batch[-1]:  # Nie czekaj po ostatnim
                time.sleep(self.rate_limit_delay)

        pbar.close()

        logger.info(f"✅ Batch zakończony: {success_count} sukces, {error_count} błędów")
        return success_count, error_count

    def run_full_reranking(self, batch_size=50, max_articles=None):
        """Uruchom pełny re-ranking wszystkich artykułów"""

        logger.info("📊 URUCHAMIAM PEŁNY RE-RANKING COHERE")
        logger.info("=" * 50)

        # Połącz z bazą
        connection = self.get_db_connection()
        if not connection:
            return False

        # Inicjalizuj Cohere
        if not self.initialize_cohere_client():
            connection.close()
            return False

        try:
            start_time = datetime.now()
            total_success = 0
            total_errors = 0
            processed_count = 0

            logger.info(f"🎯 Rozpoczynam re-ranking kandydatów...")

            while True:
                # Pobierz kolejny batch
                remaining_batch_size = batch_size
                if max_articles:
                    remaining = max_articles - processed_count
                    if remaining <= 0:
                        logger.info(f"✅ Osiągnięto limit {max_articles} artykułów")
                        break
                    remaining_batch_size = min(batch_size, remaining)

                articles_batch = self.fetch_articles_with_candidates(
                    connection, remaining_batch_size
                )

                if not articles_batch:
                    logger.info("✅ Wszystkie artykuły przetworzone!")
                    break

                # Przetwórz batch
                batch_success, batch_errors = self.process_batch_with_rate_limiting(
                    connection, articles_batch
                )

                # Aktualizuj statystyki
                total_success += batch_success
                total_errors += batch_errors
                processed_count += len(articles_batch)

                # Statystyki pośrednie
                elapsed = datetime.now() - start_time
                if elapsed.total_seconds() > 0:
                    rate = processed_count / (elapsed.total_seconds() / 60)
                    logger.info(f"📊 Postęp: {processed_count} artykułów, {rate:.1f} art/min")

                # Pauza między batchami
                if articles_batch and len(articles_batch) == remaining_batch_size:
                    logger.info("😴 Pauza 5s między batchami...")
                    time.sleep(5)

            # Podsumowanie końcowe
            end_time = datetime.now()
            duration = end_time - start_time

            logger.info(f"\n🎉 RE-RANKING ZAKOŃCZONY!")
            logger.info("=" * 40)
            logger.info(f"⏱️ Czas trwania: {duration}")
            logger.info(f"✅ Sukces: {total_success}")
            logger.info(f"❌ Błędy: {total_errors}")
            logger.info(f"📊 Łącznie: {processed_count}")

            if duration.total_seconds() > 0:
                rate = processed_count / (duration.total_seconds() / 60)
                logger.info(f"⚡ Prędkość: {rate:.1f} artykułów/min")

            return total_success > 0

        finally:
            connection.close()
            logger.info("🔒 Połączenie zamknięte")

# ============================================================================
# 📊 FUNKCJE POMOCNICZE
# ============================================================================

def check_reranking_status():
    """Sprawdź status re-rankingu"""

    print(f"\n📊 STATUS RE-RANKINGU")
    print("=" * 30)

    try:
        connection = psycopg2.connect(
            host=os.environ['POSTGRESQL_HOST'],
            port=os.environ['POSTGRESQL_PORT'],
            database=os.environ['POSTGRESQL_DB'],
            user=os.environ['POSTGRESQL_USER'],
            password=os.environ['POSTGRESQL_PASSWORD']
        )

        cursor = connection.cursor()

        # Artykuły z kandydatami
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE candidates IS NOT NULL
            AND candidates != '[]'
            AND candidates != 'null'
        """)
        with_candidates = cursor.fetchone()[0]

        # Artykuły z re-rankingiem
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE re_rank_content IS NOT NULL
            AND re_rank_content != '[]'
            AND re_rank_content != 'null'
        """)
        with_reranking = cursor.fetchone()[0]

        # Artykuły oczekujące na re-ranking
        cursor.execute("""
            SELECT COUNT(*) FROM articles
            WHERE candidates IS NOT NULL
            AND candidates != '[]'
            AND candidates != 'null'
            AND (re_rank_content IS NULL OR re_rank_content = '[]' OR re_rank_content = 'null')
        """)
        pending_reranking = cursor.fetchone()[0]

        print(f"🔗 Artykuły z kandydatami: {with_candidates}")
        print(f"📊 Artykuły z re-rankingiem: {with_reranking}")
        print(f"⏳ Oczekujące na re-ranking: {pending_reranking}")

        if with_candidates > 0:
            percent = (with_reranking / with_candidates) * 100
            print(f"📈 Pokrycie re-rankingu: {percent:.1f}%")

            if percent >= 95:
                print(f"✅ GOTOWE do następnego etapu!")
            elif percent >= 50:
                print(f"🟡 Częściowo gotowe")
            else:
                print(f"🔴 Wymaga dokończenia")

        cursor.close()
        connection.close()

        return with_candidates, with_reranking, pending_reranking

    except Exception as e:
        print(f"❌ Błąd sprawdzania statusu: {e}")
        return 0, 0, 0

# ============================================================================
# 🚀 GŁÓWNA FUNKCJA
# ============================================================================

def main():
    """Główna funkcja programu"""

    print("📊 RE-RANKING COHERE")
    print("=" * 40)

    # Konfiguracja środowiska
    if not setup_environment():
        print("❌ Błąd konfiguracji - sprawdź sekrety!")
        return

    # Sprawdź obecny status
    with_candidates, with_reranking, pending_reranking = check_reranking_status()

    # Menu opcji
    print(f"\n🎯 OPCJE:")
    print("1. Pełny re-ranking (wszystkie oczekujące artykuły)")
    print("2. Test na 10 artykułach")
    print("3. Re-ranking określonej liczby artykułów")
    print("4. Tylko sprawdzenie statusu")

    if with_reranking > 0:
        print("5. Nadpisanie istniejących re-rankingów")

    choice = input("\nWybierz opcję (1-5): ").strip()

    if choice == '1':
        # Pełny re-ranking
        print(f"\n🚀 PEŁNY RE-RANKING")
        print(f"📊 Będzie przetwarzać {pending_reranking} artykułów")

        if pending_reranking == 0:
            print("❌ Brak artykułów do re-rankingu!")
            return

        confirm = input("Kontynuować? (tak/nie): ").strip().lower()
        if confirm in ['tak', 'yes', 't', 'y', '']:
            processor = CohereRerankingProcessor()
            success = processor.run_full_reranking()

            if success:
                print(f"\n🎯 GOTOWE DO NASTĘPNEGO ETAPU!")
                print(f"💡 Uruchom: Skrypt 4 - Analiza i eksport")
        else:
            print("❌ Anulowano")

    elif choice == '2':
        # Test na 10 artykułach
        print(f"\n🧪 TEST RE-RANKING (10 artykułów)")
        processor = CohereRerankingProcessor()
        processor.top_n_output = 3  # Mniej wyników do testu
        success = processor.run_full_reranking(batch_size=10, max_articles=10)

    elif choice == '3':
        # Określona liczba artykułów
        try:
            max_articles = int(input("Ile artykułów? "))
            print(f"\n🎯 RE-RANKING {max_articles} ARTYKUŁÓW")
            processor = CohereRerankingProcessor()
            success = processor.run_full_reranking(max_articles=max_articles)
        except ValueError:
            print("❌ Nieprawidłowa liczba!")

    elif choice == '4':
        # Tylko status
        print(f"✅ Status już wyświetlony powyżej")

    elif choice == '5' and with_reranking > 0:
        # Nadpisanie
        print(f"\n🔄 NADPISYWANIE RE-RANKINGÓW")
        print(f"⚠️ To nadpisze {with_reranking} istniejących re-rankingów")
        confirm = input("Kontynuować? (tak/nie): ").strip().lower()

        if confirm in ['tak', 'yes', 't', 'y']:
            # Resetuj re-ranking status
            try:
                connection = psycopg2.connect(
                    host=os.environ['POSTGRESQL_HOST'],
                    port=os.environ['POSTGRESQL_PORT'],
                    database=os.environ['POSTGRESQL_DB'],
                    user=os.environ['POSTGRESQL_USER'],
                    password=os.environ['POSTGRESQL_PASSWORD']
                )

                cursor = connection.cursor()
                cursor.execute("UPDATE articles SET re_rank_content = NULL WHERE re_rank_content IS NOT NULL")
                connection.commit()
                cursor.close()
                connection.close()

                print(f"✅ Zresetowano re-rankingi")

                processor = CohereRerankingProcessor()
                processor.run_full_reranking()

            except Exception as e:
                print(f"❌ Błąd resetowania: {e}")
        else:
            print("❌ Anulowano")

    else:
        print("❌ Nieprawidłowy wybór!")

if __name__ == "__main__":
    main()

✅ cohere już zainstalowany
📊 RE-RANKING COHERE
✅ Zmienne środowiskowe ustawione

📊 STATUS RE-RANKINGU
🔗 Artykuły z kandydatami: 174
📊 Artykuły z re-rankingiem: 0
⏳ Oczekujące na re-ranking: 174
📈 Pokrycie re-rankingu: 0.0%
🔴 Wymaga dokończenia

🎯 OPCJE:
1. Pełny re-ranking (wszystkie oczekujące artykuły)
2. Test na 10 artykułach
3. Re-ranking określonej liczby artykułów
4. Tylko sprawdzenie statusu

Wybierz opcję (1-5): 1

🚀 PEŁNY RE-RANKING
📊 Będzie przetwarzać 174 artykułów
Kontynuować? (tak/nie): tak


2025-06-03 11:23:40,794 - INFO - 📊 URUCHAMIAM PEŁNY RE-RANKING COHERE
2025-06-03 11:23:40,795 - INFO - ==================================================


✅ CohereRerankingProcessor zainicjalizowany
   🎯 Max kandydatów do re-rankingu: 10
   🏆 Top N wyników: 5
   ⏱️ Delay między requestami: 2s


2025-06-03 11:23:42,008 - INFO - ✅ Połączono z PostgreSQL
2025-06-03 11:23:42,060 - INFO - ✅ Klient Cohere zainicjalizowany
2025-06-03 11:23:42,061 - INFO - 🎯 Rozpoczynam re-ranking kandydatów...
2025-06-03 11:23:42,062 - INFO - 📥 Ładowanie artykułów z kandydatami...
2025-06-03 11:23:43,267 - INFO - 📊 Znaleziono 50 artykułów z kandydatami do re-rankingu
2025-06-03 11:23:43,269 - INFO - 🚀 Przetwarzam batch 50 artykułów...
Re-ranking Cohere:   0%|                                        | 0/50 [00:00<?]2025-06-03 11:23:44,000 - INFO - 📄 Pobrano teksty dla 10 linków z 10 żądanych
2025-06-03 11:23:44,002 - INFO - 🔄 Re-ranking 10 dokumentów dla https://cateringfoodharmony.pl/...
2025-06-03 11:23:44,535 - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2025-06-03 11:23:44,538 - INFO - ✅ Re-ranking ukończony: 5 wyników
2025-06-03 11:23:45,020 - INFO - ✅ Zapisano re-ranking dla artykułu ID: 1
Re-ranking Cohere:   2%|▋                                   | 1/50 [00:03<

## 6.1 Analiza re-rankingu

In [ ]:
from google.colab import userdata
import os
import mysql.connector
from mysql.connector import Error
import json
import pandas as pd
from collections import Counter
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Konfiguracja środowiska
def setup_environment():
    os.environ['DB_HOST'] = userdata.get('DB_HOST')
    os.environ['DB_PORT'] = '3306'
    os.environ['DB_USER'] = userdata.get('DB_USER')
    os.environ['DB_PASS'] = userdata.get('DB_PASS')
    os.environ['DB_NAME'] = userdata.get('DB_NAME')
    os.environ['Linkowanie_wewnetrzne_master_baza'] = 'blog_linkowanie_wewnetrzne_master'
    print("✅ Zmienne środowiskowe ustawione")

setup_environment()

def get_db_connection():
    try:
        connection = mysql.connector.connect(
            host=os.getenv('DB_HOST'),
            port=os.getenv('DB_PORT'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASS'),
            database=os.getenv('DB_NAME')
        )
        logging.info("Connected to the database")
        return connection
    except Error as err:
        logging.error(f"Error: '{err}'")
        return None

def quick_rerank_analysis():
    """Szybka analiza wyników re-rankingu Cohere"""
    print("\n" + "="*60)
    print("🔍 SZYBKA ANALIZA WYNIKÓW RE-RANKING COHERE")
    print("="*60)

    connection = get_db_connection()
    if connection is None:
        print("❌ Błąd połączenia z bazą danych")
        return

    try:
        table_name = os.getenv('Linkowanie_wewnetrzne_master_baza')
        cursor = connection.cursor(dictionary=True)

        # 1. Podstawowe statystyki
        print("\n📊 PODSTAWOWE STATYSTYKI:")
        print("-" * 40)

        # Liczba artykułów z re-ranking
        cursor.execute(f"SELECT COUNT(*) as total FROM {table_name} WHERE re_rank_content IS NOT NULL AND re_rank_content != ''")
        total_reranked = cursor.fetchone()['total']

        # Łączna liczba artykułów
        cursor.execute(f"SELECT COUNT(*) as total FROM {table_name} WHERE text_status = 'completed'")
        total_articles = cursor.fetchone()['total']

        print(f"📄 Łączna liczba artykułów: {total_articles}")
        print(f"🔗 Artykuły z re-ranking: {total_reranked}")
        print(f"📈 Pokrycie: {(total_reranked/total_articles*100):.1f}%")

        # 2. Analiza linków i scores
        cursor.execute(f"SELECT re_rank_content FROM {table_name} WHERE re_rank_content IS NOT NULL AND re_rank_content != ''")
        rerank_data = cursor.fetchall()

        all_links = []
        all_scores = []
        links_per_article = []

        for row in rerank_data:
            try:
                data = json.loads(row['re_rank_content'])
                article_links = 0
                for item in data:
                    if 'link' in item and 'relevance_score' in item:
                        all_links.append(item['link'])
                        all_scores.append(item['relevance_score'])
                        article_links += 1
                links_per_article.append(article_links)
            except json.JSONDecodeError:
                continue

        print(f"\n🔗 STATYSTYKI LINKÓW:")
        print("-" * 40)
        print(f"📊 Łączna liczba wygenerowanych linków: {len(all_links)}")
        print(f"📊 Średnia linków na artykuł: {sum(links_per_article)/len(links_per_article):.1f}")
        print(f"📊 Max linków w artykule: {max(links_per_article) if links_per_article else 0}")
        print(f"📊 Min linków w artykule: {min(links_per_article) if links_per_article else 0}")

        # 3. Analiza relevance scores
        if all_scores:
            print(f"\n⭐ RELEVANCE SCORES:")
            print("-" * 40)
            print(f"📈 Średni score: {sum(all_scores)/len(all_scores):.3f}")
            print(f"📈 Najwyższy score: {max(all_scores):.3f}")
            print(f"📈 Najniższy score: {min(all_scores):.3f}")

            # Rozkład scores
            high_quality = sum(1 for s in all_scores if s >= 0.8)
            medium_quality = sum(1 for s in all_scores if 0.5 <= s < 0.8)
            low_quality = sum(1 for s in all_scores if s < 0.5)

            print(f"\n📊 JAKOŚĆ DOPASOWAŃ:")
            print("-" * 40)
            print(f"🟢 Wysokie (≥0.8): {high_quality} ({high_quality/len(all_scores)*100:.1f}%)")
            print(f"🟡 Średnie (0.5-0.8): {medium_quality} ({medium_quality/len(all_scores)*100:.1f}%)")
            print(f"🔴 Niskie (<0.5): {low_quality} ({low_quality/len(all_scores)*100:.1f}%)")

        # 4. Top 10 najczęściej linkowanych artykułów
        link_counter = Counter(all_links)
        top_links = link_counter.most_common(10)

        print(f"\n🏆 TOP 10 NAJCZĘŚCIEJ LINKOWANYCH:")
        print("-" * 40)
        for i, (link, count) in enumerate(top_links, 1):
            # Skrócenie URL dla czytelności
            short_link = link[:60] + "..." if len(link) > 60 else link
            print(f"{i:2d}. {short_link} ({count}x)")

        # 5. Sprawdzenie duplikatów
        unique_links = len(set(all_links))
        duplicate_percentage = (len(all_links) - unique_links) / len(all_links) * 100 if all_links else 0

        print(f"\n🔄 ANALIZA DUPLIKATÓW:")
        print("-" * 40)
        print(f"📊 Unikalne linki: {unique_links}")
        print(f"📊 Wszystkie linki: {len(all_links)}")
        print(f"📊 Procent duplikatów: {duplicate_percentage:.1f}%")

        print(f"\n✅ PODSUMOWANIE:")
        print("="*60)
        if total_reranked > 0:
            avg_score = sum(all_scores)/len(all_scores) if all_scores else 0
            print(f"🎯 System działa poprawnie!")
            print(f"📊 {total_reranked} artykułów ma wygenerowane linki")
            print(f"⭐ Średni relevance score: {avg_score:.3f}")
            print(f"🔗 Wygenerowano {len(all_links)} połączeń")

            if avg_score >= 0.7:
                print(f"✅ Jakość linków: WYSOKA")
            elif avg_score >= 0.5:
                print(f"🟡 Jakość linków: ŚREDNIA")
            else:
                print(f"⚠️ Jakość linków: NISKA - może warto dostroić parametry")
        else:
            print(f"❌ Brak danych re-ranking - sprawdź czy Cohere działał poprawnie")

    except Exception as e:
        print(f"❌ Błąd podczas analizy: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        connection.close()
        print(f"\n🔒 Połączenie z bazą zamknięte")

if __name__ == "__main__":
    quick_rerank_analysis()